<a href="https://colab.research.google.com/github/IanMcDevitt/Interview/blob/main/backtestRLlevel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this work-in-progress project, I am developing a high-frequency trading (HFT) model utilizing an Actor-Critic framework with Proximal Policy Optimization (PPO). This project aims to create a sophisticated market-making strategy that incorporates directional-trend position holding. Key features include an evolutionary algorithm for model training, custom engineering of reward and state for decision-making accuracy, and a specialized backtesting system for model evaluation. This effort demonstrates my expertise in applying deep learning and reinforcement learning techniques to financial trading, emphasizing adaptive strategies and precision in navigating the cryptocurrency market.



In [ ]:
!pip install pandas
!pip install matplotlib
!pip install uuid
!pip install numpy
!pip install queue
!pip install math
!pip install glob
!pip install time
!pip install matplotlib.pyplot
!pip install matplotlib.patches
!pip install matplotlib.colors
!pip install tqdm
!pip install pytorch
!pip install torchvision
!pip install scipy
!pip install scikit-learn
!pip install gym
!pip install keras
!pip install deque




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  Preparing metadata (setup.py) ... done
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6479 sha256=1add64483d2059e7e1018319d9fbfc645abc64650bb427f1c5ad11d1c4646fbc
  Stored in directory: /root/.cache/pip/wheels/ed/08/9e/f0a977dfe55051a07e21af89200125d65f1efa60cbac61ed88
Successfully built uuid


ERROR: Could not find a version that satisfies the requirement queue (from versions: none)
ERROR: No matching distribution found for queue
ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math
ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot
ERROR: Could not find a version that satisfies the requirement matplotlib.patches (from versions: none)
ERROR: No matching distribution found for matplotlib.patches
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not 

In [ ]:
# At the start of your Jupyter Notebook
%matplotlib inline
# or
%matplotlib notebook


In [ ]:
import numpy as np
# Example hyperparameters
actor_critic_hyperparams = {
    'state_size': 66,#324
    'action_size': 8,
    'shared_layers_config': [
        # Shared layers are defined within the class
    ],
    'actor_layers_config': [
        {'in_features': 128, 'out_features': 256},
        {'in_features': 256, 'out_features': 256},
        {'in_features': 256, 'out_features': 128}
    ],
    'critic_layers_config': [
        {'in_features': 128, 'out_features': 256},
        {'in_features': 256, 'out_features': 256},
        {'in_features': 256, 'out_features': 128}
    ],
'action_scale': np.array([
    0.00035,  # Bid Price Adjustment
    0.00035,  # Ask Price Adjustment
    0.0255,  # Buy Order Size
    0.0255,  # Sell Order Size
    0.5,     # Hold
    0.35,    # Target Inventory Level
    0.5,     # Cancellation Intensity
    0.5      # Cancellation Strategy
]),
'action_bias': np.array([
    -0.00025,       # Bid Price Adjustment
    0.00025,       # Ask Price Adjustment
    0.001,  # Buy Order Size
    0.001,  # Sell Order Size
    0.5,    # Hold
    0.0,       # Target Inventory Level
    0.5,     # Cancellation Intensity
    0.5      # Cancellation Strategy
]),

    'log_std_min': -20,
    'log_std_max': 2,
    'init_nu': 15  # Initial nu value for the Student's t-distribution
}
ppo_hyperparams = {
    'clip_param': 0.2,  # Clipping parameter for PPO's objective function
    'ppo_epoch': 1,     # Number of epochs for PPO updates
    'mini_batch_size': 128,  # Size of mini-batches used in PPO updates
    'value_loss_coef': 0.5, # Coefficient for the value loss term
    'entropy_coef': 0.1,   # Coefficient for the entropy bonus 0.001
    'lr': 3e-4,             # Learning rate for the optimizer
    'eps': 1e-5,            # Epsilon value for the optimizer
    'max_grad_norm': 1.0,   # Maximum norm for gradient clipping
    'target_entropy': -0.1 # Example target entropy value for your HFT scenario
}


environment_training_settings = {
    'buffer_size': 2048,              # Size of the rollout buffer
    'state_shape': (1,66),            # Shape of the state space
    'action_shape': (1,8),
    'mini_batch_size': 128, # Shape of the action space
    'gamma': 0.99,                    # Discount factor for future rewards
    'lambda_gae': 0.95,               # Lambda parameter for Generalized Advantage Estimation
    'total_timesteps': 100000,        # Total number of timesteps for the training process
    'evaluation_interval': 5000,      # Interval (in timesteps) at which the model is evaluated
    'num_evaluation_episodes': 10     # Number of episodes to run during each evaluation
}

evolutionary_training_config = {
    'age_threshold': 6,
    'performance_threshold': -0.2,
    'improvement_rate_threshold': 0.01,
    'relative_ranking_threshold': 0.25,
    'diversity_threshold': 0.5,
    'baseline_avg_age': 10,
    'baseline_avg_performance': 0,
    'age_std_dev_factor': 1,
    'performance_std_dev_factor': 1,
    'population_size': 20,
    'num_generations': 50
}


In [ ]:
EVOLUTION_INTERVAL = 2 * 60 * 60  # 2 hours in seconds
TRAINING_FREQUENCY = 4  # Train 4 times within the evolution interval

import torch
import torch.optim as optim
import logging
from datetime import datetime
import time
from collections import deque
from threading import Lock
from scipy.stats import mstats
import torch
import torch.nn.functional as F
from torch.distributions import StudentT
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import gym
from gym import spaces
import torch.nn as nn
import copy
import numpy as np
from scipy.stats import skew
from scipy.stats import kurtosis


import time
from datetime import datetime, timedelta


from tqdm import tqdm
import os
import queue

from copy import deepcopy


class ModelWithAge:
    def __init__(self, actor_critic, lob_instance, ppo, rollout_buffer, environment, volatility, trading_manager):
        self.actor_critic = actor_critic  # The model (neural network)
        self.lob_instance = lob_instance  # Instance of the Limit Order Book (LOB)
        self.ppo = ppo  # Proximal Policy Optimization (PPO) algorithm instance
        self.rollout_buffer = rollout_buffer  # Rollout buffer for storing training data
        self.environment = environment  # Trading environment
        self.volatility = volatility  # Volatility of the model
        self.decision_time = None
        self.trading_manager = trading_manager

        self.age = 0  # Age of the model
        self.performance = 0  # Current performance of the model
        self.performance_history = []  # History of the model's performance

    def update_performance(self, new_performance):
        """
        Update the model's performance and add the new performance to the history.

        Args:
            new_performance (float): The new performance metric of the model.
        """
        self.performance = self.environment.portfolio_manager.get_latest_portfolio_value() - 500
        # self.performance_history.append(new_performance)

    def act(self, state):
        """
        Use the model to determine an action based on the given state.

        Args:
            state (array): The current state of the environment.

        Returns:
            action: The action determined by the model.
            log_prob: The log probability of the action.
            value: The value estimate for the state.
        """
        return self.actor_critic.act(state)

    def train(self, env, hyperparams):
        """
        Train the model using data collected in the environment.

        Args:
            env: The environment to interact with.
            hyperparams (dict): A dictionary of hyperparameters for training.
        """
        # Training logic using self.actor_critic, self.ppo, and self.rollout_buffer
        # ...

    def reset(self):
        # Reset the rollout buffer for fresh training data
        self.rollout_buffer.reset()

        # Reset the model's inventory and cash balance to their initial states
        self.inventory = 0
        self.cash_balance = self.initial_capital

        # # Clear any pending actions that were accumulated in the previous episode
        # self.pending_actions.clear()

        # Reset the reward-related metrics
        self.reward.reset()
    # Other methods as needed...

# Additional methods, like PPO, RolloutBuffer, and neural network definitions would be here


    # ... Any other necessary methods ...
class EvolutionaryModel:
    def __init__(self,  actor_critic_hyperparams, ppo_hyperparams, rollout_buffer_params, initial_lob_state, evolutionary_training_config, volatility, portfolio_manager, capital, trading_manager):
        self.actor_critic_hyperparams = actor_critic_hyperparams
        self.ppo_hyperparams = ppo_hyperparams
        self.rollout_buffer_params = rollout_buffer_params
        self.initial_lob_state = initial_lob_state
        self.volatility = volatility
        self.portfolio_manager = portfolio_manager
        self.capital = capital
        self.evolutionary_training_config = evolutionary_training_config
        self.trading_manager = trading_manager
        self.population = self.initialize_population()

    def initialize_population(self):
        population = []
        for _ in range(self.evolutionary_training_config['population_size']):
            actor_critic = self.create_random_actor_critic_model(self.actor_critic_hyperparams)
            #self.inject_adaptive_noise_into_model(actor_critic, )

            lob_instance = copy.deepcopy(self.initial_lob_state)
            ppo = PPO(actor_critic, self.ppo_hyperparams)
            rollout_buffer = RolloutBuffer(self.rollout_buffer_params)
            portfolio_manager = copy.deepcopy(self.portfolio_manager)
            trading_manager = copy.deepcopy(self.trading_manager)
            new_volatility = copy.deepcopy(self.volatility)
            trading_environment = ContinuousTradingEnvironment(new_volatility, portfolio_manager, self.capital, trading_manager)

            model_with_age = ModelWithAge(actor_critic, lob_instance, ppo, rollout_buffer, trading_environment, new_volatility, trading_manager)
            population.append(model_with_age)
        return population

    def clone(self):
        # Create a deep copy of the model along with its LOB
        cloned_lob = deepcopy(self.lob)
        cloned_model = deepcopy(self.actor_critic)
        return ModelWithAge(cloned_model, cloned_lob)

    @staticmethod
    def create_random_actor_critic_model(hyperparams):
        # Initialize a new ActorCriticNetwork instance with random weights
        random_model = ActorCriticNetwork(
            state_size=hyperparams['state_size'],
            action_size=hyperparams['action_size'],
            shared_layers_config=hyperparams['shared_layers_config'],
            actor_layers_config=hyperparams['actor_layers_config'],
            critic_layers_config=hyperparams['critic_layers_config'],
            action_scale=hyperparams['action_scale'],
            action_bias=hyperparams['action_bias'],
            log_std_min=hyperparams.get('log_std_min', -2),
            log_std_max=hyperparams.get('log_std_max', 1),
            init_nu=hyperparams.get('init_nu', 7)
        )
        random_model.eval()
        # Randomize the weights and biases of the network
        for layer in random_model.modules():
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    nn.init.uniform_(layer.bias, -0.1, 0.1)

        return random_model

    def evolve_population(self):
        avg_age = sum(model.age for model in self.population) / len(self.population)
        avg_performance = sum(model.performance for model in self.population) / len(self.population)

        print(f"Average Age: {avg_age}, Average Performance: {avg_performance}")

        dynamic_age_threshold = self.adjust_age_threshold_based_on_population_trends(self.population, self.evolutionary_training_config)
        dynamic_performance_threshold = self.adjust_performance_threshold_based_on_population_trends(self.population, self.evolutionary_training_config)

        print(f"Dynamic Age Threshold: {dynamic_age_threshold}, Dynamic Performance Threshold: {dynamic_performance_threshold}")

        # Pruning models based on age and performance
        self.population = [model for model in self.population if not self.should_prune(model, avg_age, avg_performance, dynamic_age_threshold, dynamic_performance_threshold)]
        print(f"Population size after pruning: {len(self.population)}")

        scores = [model.performance for model in self.population]
        # Maintaining diversity
        # self.population = self.maintain_diversity(self.population, scores, self.evolutionary_training_config['diversity_threshold'])
        # print(f"Population size after maintaining diversity: {len(self.population)}")

        sorted_population = sorted(zip(scores, self.population), key=lambda pair: pair[0], reverse=True)
        top_performers = [model for _, model in sorted_population[:len(sorted_population) // 2]]
        new_population = top_performers.copy()

        print(f"Selected top performers. New population size: {len(new_population)}")

        while len(new_population) < self.evolutionary_training_config['population_size']:
            parent1, parent2 = random.sample(top_performers, 2)
            child_actor_critic = self.two_point_crossover(parent1.actor_critic, parent2.actor_critic)
            self.mutate(child_actor_critic)

            child_lob_instance = copy.deepcopy(self.initial_lob)
            child_ppo = PPO(child_actor_critic, self.ppo_hyperparams)
            child_rollout_buffer = RolloutBuffer(self.rollout_buffer_params)
            child_portfolio_manager = copy.deepcopy(self.portfolio_manager)
            child_volatility = copy.deepcopy(self.volatility)
            child_trading_manager = copy.deepcopy(self.trading_manager)
            child_trading_environment = ContinuousTradingEnvironment(child_volatility, child_portfolio_manager, self.capital, child_trading_manager)
            child_model = ModelWithAge(child_actor_critic, child_lob_instance, child_ppo, child_rollout_buffer, child_trading_environment, child_volatility, child_trading_manager)
            new_population.append(child_model)

            print(f"Added new child model to population. Total population size: {len(new_population)}")

        self.population = new_population
        print(f"Final population size after evolution: {len(self.population)}")
        # Additional methods as needed for training, performance evaluation, etc.

    # Additional methods as needed for training, performance evaluation, etc.

    def calculate_rate_of_improvement(self,model):
        if len(model.performance_history) < 2:
            return 0
        return model.performance_history[-1] - model.performance_history[-2]

    # Function to calculate relative ranking of a model in the population
    def calculate_relative_ranking(self,model, population):
        scores = [m.performance for m in population]
        return sum(1 for score in scores if model.performance > score) / len(scores)

    # Function to decide whether to prune a model
    def should_prune(self, model, avg_age, avg_performance, population, config):
        age_difference = model.age - avg_age
        performance_difference = model.performance - avg_performance
        improvement_rate = self.calculate_rate_of_improvement(model)
        relative_ranking = self.calculate_relative_ranking(model, population)

        age_threshold = config['age_threshold']
        performance_threshold = config['performance_threshold']
        improvement_rate_threshold = config['improvement_rate_threshold']
        relative_ranking_threshold = config['relative_ranking_threshold']

        return (age_difference > age_threshold and
                performance_difference < -performance_threshold and
                improvement_rate < improvement_rate_threshold and
                relative_ranking < relative_ranking_threshold)

    # Evolutionary Functions
    def two_point_crossover(self, parent1, parent2):
        child = copy.deepcopy(parent1)
        for param1, param2 in zip(child.parameters(), parent2.parameters()):
            if len(param1.data.size()) == 2:  # Weight matrices
                num_rows = param1.data.size(0)
                points = sorted(random.sample(range(num_rows), 2))
                param1.data[points[0]:points[1]] = param2.data[points[0]:points[1]]
        return child

    def mutate(self, model):
        for param in model.parameters():
            if len(param.size()) == 2:  # Weight matrices
                for i in range(param.size(0)):
                    for j in range(param.size(1)):
                        if random.uniform(0, 1) < 0.1:  # Mutation probability
                            param.data[i][j] += random.uniform(-0.1, 0.1)  # Mutation amount

    def inject_adaptive_noise_into_model(self, model, base_noise_std=0.1):
        with torch.no_grad():
            for param in model.parameters():
                param_std = param.data.std().item()
                scaled_noise_std = base_noise_std * (1 / (1 + param_std))  # Example scaling based on std deviation of the parameter
                noise = torch.randn_like(param) * scaled_noise_std
                param.data += noise


    def calculate_genetic_distance(self, model1, model2):
        distance = 0
        for param1, param2 in zip(model1.parameters(), model2.parameters()):
            distance += torch.norm(param1.data - param2.data, p=2).item()  # Euclidean distance
        return distance

    def calculate_population_diversity(self):
        total_distance = 0
        count = 0
        for i in range(len(self.population)):
            for j in range(i + 1, len(self.population)):
                total_distance += self.calculate_genetic_distance(self.population[i].actor_critic, self.population[j].actor_critic)
                count += 1
        return total_distance / count if count > 0 else 0

    def maintain_diversity(self, population, scores, diversity_threshold):
        diversity_score = self.calculate_population_diversity(population)
        if diversity_score < diversity_threshold:
            for i in range(len(population) - len(scores) // 2, len(population)):
                hyperparams = population[i].ppo.hyperparams  # Assuming hyperparams are stored in the ppo object of each model
                population[i] = EvolutionaryModel.create_random_actor_critic_model(hyperparams)
        return population

    def adjust_age_threshold_based_on_population_trends(self, population, config):
        ages = [model.age for model in population]
        avg_age = sum(ages) / len(ages)
        age_std_dev = (sum((age - avg_age) ** 2 for age in ages) / len(ages)) ** 0.5
        baseline_age_threshold = config['baseline_avg_age']
        age_std_dev_factor = config['age_std_dev_factor']

        # Adjust the age threshold based on the average age and standard deviation
        # Older populations might need a higher threshold
        if avg_age > 5:
            return baseline_age_threshold + age_std_dev_factor * age_std_dev
        else:
            return baseline_age_threshold - age_std_dev_factor * age_std_dev


    def adjust_performance_threshold_based_on_population_trends(self, population, config):
        performances = [model.performance for model in population]
        avg_performance = sum(performances) / len(performances)
        performance_std_dev = (sum((performance - avg_performance) ** 2 for performance in performances) / len(performances)) ** 0.5
        baseline_performance_threshold = config['baseline_avg_performance']
        performance_std_dev_factor = config['performance_std_dev_factor']

        # Adjust the performance threshold based on the average performance and standard deviation
        # Higher performing populations might need a tighter threshold
        if avg_performance > 200:
            return baseline_performance_threshold - performance_std_dev_factor * performance_std_dev
        else:
            return baseline_performance_threshold + performance_std_dev_factor * performance_std_dev

        def select_top_performers(self, num_top_performers):
            # Sort the population based on model performance in descending order
            sorted_population = sorted(self.population, key=lambda model: model.performance, reverse=True)
            # Return the top performers
            return sorted_population[:num_top_performers]









class RateLimiter:
    def __init__(self, max_num_orders, max_ratecount, ratecount_decay, decay_interval):
        self.max_num_orders = max_num_orders
        self.max_ratecount = max_ratecount
        self.ratecount_decay = ratecount_decay
        # self.max_total_orders = max_total_orders
        #self.max_buy_ratio = max_buy_ratio
        #self.max_sell_ratio = max_sell_ratio
        self.total_orders_placed = 0

        self.decay_interval = decay_interval
        self.rate_limit_window_reset_time = None
        self.current_ratecount = 0
        self.last_action_time = None
        self.lock = Lock()

    def update_rate_limit_reset_time(self):
        if self.current_ratecount > 0:
            time_to_full_decay = self.current_ratecount / self.ratecount_decay
            self.rate_limit_window_reset_time = self.last_action_time + time_to_full_decay
        else:
            self.rate_limit_window_reset_time = None


    # def simulate_time_step(self, current_time):
    #     with self.lock:
    #         elapsed_time = current_time - self.last_action_time
    #         self.current_ratecount = max(0, self.current_ratecount - self.ratecount_decay * elapsed_time)
    #         self.last_action_time = current_time
    #         self.update_rate_limit_reset_time()

    def simulate_time_step(self, local_timestamp):
        #with self.lock:
        if self.last_action_time is None:
            # For the first action, set the last action time to the current timestamp
            self.last_action_time = local_timestamp
        else:
            # Calculate the elapsed time since the last action
            elapsed_time = local_timestamp - self.last_action_time

            # Apply the decay to the rate count
            # The decay is based on the elapsed time and the ratecount_decay parameter
            decay_amount = self.ratecount_decay * elapsed_time
            self.current_ratecount = max(0, self.current_ratecount - decay_amount)

            # Update the last action time to the current timestamp
            self.last_action_time = local_timestamp

        # Update the time for when the rate limit window is reset
        self.update_rate_limit_reset_time()



    # ... other methods ...

    # def get_time_until_rate_limit_reset(self):
    #     if self.rate_limit_window_reset_time:
    #         return max(0, self.rate_limit_window_reset_time - time.time())
    #     return 0

    def get_time_until_rate_limit_reset(self, local_timestamp):
        """
        Calculate the time until the rate limit is reset, based on the provided local timestamp.

        :param local_timestamp: The simulated current time in the backtesting environment.
        :return: Time in seconds until the rate limit resets. If no reset is scheduled, returns 0.
        """
        #with self.lock:
        if self.rate_limit_window_reset_time:
            return max(0, self.rate_limit_window_reset_time - local_timestamp)
        return 0

    def get_usage_percentage(self):
        # Calculate the usage percentage
        usage_percentage = (self.current_ratecount / self.max_ratecount) * 100
        return usage_percentage

    def can_execute_order(self, local_timestamp):
        #with self.lock:
        self.simulate_time_step(local_timestamp)

        if self.current_ratecount >= self.max_ratecount:
            return False, "EOrder:Rate limit exceeded"

        if self.total_orders_placed >= self.max_num_orders:
            return False, "EOrder:Max number of orders exceeded"

        return True, ""




    def _calculate_cancellation_penalty(self, order_age):
          if order_age < 5:
              return 8
          elif order_age < 10:
              return 6
          elif order_age < 15:
              return 5
          elif order_age < 45:
              return 4
          elif order_age < 90:
              return 2
          elif order_age < 300:
              return 1
          else:
              return 0

    def update_on_order_placement(self, local_timestamp):
        with self.lock:
            self.simulate_time_step(local_timestamp)
            self.current_ratecount += 1
            self.total_orders_placed += 1  # Increment total orders


    def can_cancel_order(self, order_placement_time, local_timestamp):
        order_age = local_timestamp - order_placement_time

        #with self.lock:
        self.simulate_time_step(local_timestamp)
        penalty = self._calculate_cancellation_penalty(order_age)

        if self.current_ratecount + penalty >= self.max_ratecount:
            return False, "EOrder:Rate limit exceeded"

        self.current_ratecount += penalty
        self.last_action_time = local_timestamp

        return True, ""

    # def can_cancel_order(self, order_placement_time):
    #     current_time = time.time()
    #     order_age = current_time - order_placement_time

    #     with self.lock:
    #         self.simulate_time_step(current_time)
    #         penalty = self._calculate_cancellation_penalty(order_age)

    #         if self.current_ratecount + penalty >= self.max_ratecount:
    #             return False, "EOrder:Rate limit exceeded"

    #         self.current_ratecount += penalty
    #         self.last_action_time = current_time

    #         return True, ""

class Swish(nn.Module):
    def __init__(self):
        super(Swish, self).__init__()
        self.beta = nn.Parameter(torch.ones(1))

    def forward(self, x):
        return x * torch.sigmoid(self.beta * x)

class Mish(nn.Module):
    def __init__(self):
        super(Mish, self).__init__()

    def forward(self, x):
        # Mish function: x * tanh(softplus(x)) = x * tanh(ln(1 + e^x))
        return x * torch.tanh(F.softplus(x))


class ActorCriticNetwork(nn.Module):
    def __init__(self, state_size, action_size, shared_layers_config, actor_layers_config, critic_layers_config, action_scale, action_bias, log_std_min=-2, log_std_max=2, init_nu=15):
        super(ActorCriticNetwork, self).__init__()
        # Shared layers
        # Shared layers
        self.shared_layers = self._build_layers([
            {'in_features': state_size, 'out_features': 128},
            {'in_features': 128, 'out_features': 256},
            {'in_features': 256, 'out_features': 256},
            {'in_features': 256, 'out_features': 128},
            {'in_features': 128, 'out_features': 128}

        ])

        # Actor layers
        self.actor_layers = self._build_layers(actor_layers_config)
        self.actor_mean = nn.Linear(128, action_size)
        self.actor_log_std = nn.Parameter(torch.zeros(1, action_size))
        self.actor_log_std = nn.Parameter(torch.zeros(1, action_size).fill_(-0.5))  # Initialize closer to the middle of your range


        # Critic layers
        self.critic_layers = self._build_layers(critic_layers_config)
        self.critic_output = nn.Linear(128, 1)
        self.actor_nu = nn.Parameter(torch.tensor([np.log(init_nu)]))

        # Action scale and bias
        # Set action scale and bias
        # self.action_scale = torch.tensor(action_scale, dtype=torch.float32)
        # self.action_bias = torch.tensor(action_bias, dtype=torch.float32)

        # # Ensure these are registered as part of the model's parameters
        # self.register_buffer('action_scale', self.action_scale)
        # self.register_buffer('action_bias', self.action_bias)
        self.register_buffer('action_scale', torch.tensor(action_scale, dtype=torch.float32))
        self.register_buffer('action_bias', torch.tensor(action_bias, dtype=torch.float32))



        # Log std bounds
        self.log_std_min = log_std_min
        self.log_std_max = log_std_max

    def _build_layers(self, layer_config):
        layers = []
        for idx, layer_details in enumerate(layer_config):
            layers.append(nn.Linear(layer_details['in_features'], layer_details['out_features']))
            if idx < len(layer_config) - 1:
                layers.append(nn.BatchNorm1d(layer_details['out_features']))
                layers.append(Mish())  # Replace ReLU with Swish
                if layer_details.get('dropout', 0) > 0:
                    layers.append(nn.Dropout(layer_details['dropout']))
        return nn.Sequential(*layers)
    def forward(self, state):
        # Check for NaN in input state
        assert not torch.isnan(state).any(), "NaN detected in input state"

        # Shared feature extraction
        shared_output = self.shared_layers(state)
        assert not torch.isnan(shared_output).any(), "NaN detected in shared_output after shared_layers"

        # Actor path
        actor_output = self.actor_layers(shared_output)
        assert not torch.isnan(actor_output).any(), "NaN detected in actor_output after actor_layers"

        mean = self.actor_mean(actor_output)
        assert not torch.isnan(mean).any(), "NaN detected in mean after actor_mean"

        # Ensure std is positive and within reasonable limits
        log_std = self.actor_log_std.clamp(self.log_std_min, self.log_std_max)
        std = torch.exp(log_std)
        assert not torch.isnan(std).any(), "NaN detected in std after exponentiation"

        # Ensure nu is positive and greater than 1 to avoid undefined behavior
        #nu = torch.exp(self.actor_nu).clamp(min=5, max=30)
        nu = torch.exp(self.actor_nu.clamp(min=np.log(1.1), max=np.log(30)))
        assert not torch.isnan(nu).any(), "NaN detected in nu after processing"

        # Critic path
        critic_output = self.critic_layers(shared_output)
        assert not torch.isnan(critic_output).any(), "NaN detected in critic_output after critic_layers"

        value = self.critic_output(critic_output)
        assert not torch.isnan(value).any(), "NaN detected in value after critic_output"

        # Final check before returning
        assert not torch.isnan(mean).any(), "NaN detected in mean before return"
        assert not torch.isnan(std).any(), "NaN detected in std before return"
        assert not torch.isnan(nu).any(), "NaN detected in nu before return"
        assert not torch.isnan(value).any(), "NaN detected in value before return"

        return mean, std, nu, value

    # def forward(self, state):
    #     # Flatten the state tensor to 2D (batch size by features)
    #     flattened_state = state.view(state.size(0), -1)

    #     # Shared feature extraction
    #     shared_output = self.shared_layers(flattened_state)

    #     # Actor path
    #     actor_output = self.actor_layers(shared_output)
    #     mean = self.actor_mean(actor_output)

    #     # Ensure std is positive and within reasonable limits
    #     log_std = self.actor_log_std.clamp(self.log_std_min, self.log_std_max)
    #     std = log_std.exp()

    #     # Ensure nu is positive and greater than 1 to avoid undefined behavior
    #     nu = torch.exp(self.actor_nu).clamp(min=1.1, max=30)

    #     # Critic path
    #     critic_output = self.critic_layers(shared_output)
    #     value = self.critic_output(critic_output)

    #     return mean, std, nu, value



    def act(self, state):
        # Pass state through the network
        # flattened_state = state.view(state.size(0), -1)
        # print(flattened_state.shape)
        # Flatten the state tensor to 2D (batch size by features)
        shared_output = self.shared_layers(state)
        actor_output = self.actor_layers(shared_output)

        # Get parameters for the Student's t-distribution
        mean = self.actor_mean(actor_output)

        # Clamping the logarithm of the standard deviation and then exponentiating it
        log_std = self.actor_log_std.clamp(self.log_std_min, self.log_std_max)
        # std = log_std.exp()
        std = torch.exp(log_std)

        # Ensure nu is positive and greater than 1 to avoid undefined behavior
        # nu = torch.exp(self.actor_nu).clamp(min=5, max=30)
        nu = torch.exp(self.actor_nu.clamp(min=np.log(1.1), max=np.log(30)))


        # Create Student's t-distribution
        t_dist = torch.distributions.StudentT(df=nu, loc=mean, scale=std)

        # Sample action and compute log probability
        raw_action = t_dist.rsample()  # Raw action sampling
        log_prob = t_dist.log_prob(raw_action).sum(-1, keepdim=True)  # Log probability

        # Scale and bias the raw actions
        scaled_action = torch.tanh(raw_action) * self.action_scale + self.action_bias

        # Get the value prediction from the critic network
        value = self.critic_output(self.critic_layers(shared_output))

        # Return the action, log probability, and state value estimate
        return scaled_action, log_prob, value





class PPO:
    def __init__(self, actor_critic, ppo_hyperparams, target_entropy=.2, entropy_coef_adjust_rate=0.01, alpha=0.1):
        self.actor_critic = actor_critic
        self.clip_param = ppo_hyperparams['clip_param']
        self.ppo_epoch = ppo_hyperparams['ppo_epoch']
        self.mini_batch_size = ppo_hyperparams['mini_batch_size']
        self.value_loss_coef = ppo_hyperparams['value_loss_coef']
        self.entropy_coef = ppo_hyperparams['entropy_coef']  # Initial entropy coefficient
        self.max_grad_norm = ppo_hyperparams['max_grad_norm']
        self.optimizer = optim.Adam(actor_critic.parameters(), lr=ppo_hyperparams['lr'], eps=ppo_hyperparams['eps'])
        self.target_entropy = target_entropy
        self.initial_entropy_coef = self.entropy_coef  # Store initial value
        self.entropy_coef_adjust_rate = entropy_coef_adjust_rate
        self.alpha = alpha  # EMA smoothing factor
        # Initialize EMA for average entropy
        self.ema_entropy = 0

    def update_ema_entropy(self, current_entropy):
            self.ema_entropy = self.alpha * current_entropy + (1 - self.alpha) * self.ema_entropy

    def sigmoid_adjustment(self):
        if self.target_entropy is not None:
            current_deviation = self.ema_entropy - self.target_entropy
            k = 1.0  # Sensitivity parameter
            adjustment = 1 / (1 + np.exp(-k * current_deviation))
            new_entropy_coef = self.initial_entropy_coef * adjustment
            # Implementing a smooth update with a maximum change rate
            max_change_rate = 0.05  # Maximum change of 5% per update
            change = new_entropy_coef - self.entropy_coef
            change = np.clip(change, -self.entropy_coef * max_change_rate, self.entropy_coef * max_change_rate)
            self.entropy_coef += change
            self.entropy_coef = max(min(self.entropy_coef, 1.0), 0.01)


    def compute_losses(self, sample):

        states = sample['states']
        actions = sample['actions']
        old_log_probs = sample['log_probs']
        returns = sample['returns']
        advantages = sample['advantages']
        #states, actions, returns, advantages, old_action_log_probs, old_nu = batch

        # Get current policy outputs given the batch states
        states = states.squeeze(1)  # This removes the second dimension

        print("States shape:", states.shape)
        action_means, action_std, action_nu, state_values = self.actor_critic(states)

        # Create Student's t-distribution for current policy
        current_dist = StudentT(df=action_nu, loc=action_means, scale=action_std)

        # Compute log probabilities of batch actions
        current_action_log_probs = current_dist.log_prob(actions).sum(-1, keepdim=True)

        # Compute ratio between new and old probabilities
        ratios = torch.exp(current_action_log_probs - old_log_probs)

        # Compute actor (policy) loss
        surr1 = ratios * advantages
        surr2 = torch.clamp(ratios, 1.0 - self.clip_param, 1.0 + self.clip_param) * advantages
        actor_loss = -torch.min(surr1, surr2).mean()

        # Compute critic (value) loss
        value_loss = F.mse_loss(state_values, returns)

        # Compute entropy for exploration bonus
        entropy = current_dist.entropy().mean()

        # Compute the total loss with the dynamically adjusted entropy coefficient
        total_loss = actor_loss + self.value_loss_coef * value_loss - self.entropy_coef * entropy

        return total_loss, actor_loss, value_loss, entropy


    # def compute_gae(self, next_value, rewards, masks, values, gamma=0.99, tau=0.95):
    #     values = values + [next_value]
    #     gae = 0
    #     returns = []
    #     for step in reversed(range(len(rewards))):
    #         delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
    #         gae = delta + gamma * tau * masks[step] * gae
    #         returns.insert(0, gae + values[step])
    #     return returns

    def mini_batch_generator(self, rollouts, advantages, returns):
        batch_size = len(rollouts)
        ids = torch.randperm(batch_size)
        for i in range(0, batch_size, self.mini_batch_size):
            yield rollouts[ids[i:i+self.mini_batch_size]], advantages[ids[i:i+self.mini_batch_size]], returns[ids[i:i+self.mini_batch_size]]

    def adjust_entropy_bonus(self, avg_entropy):
        if self.target_entropy is not None:
            if avg_entropy < self.target_entropy:
                self.entropy_coef *= (1 + self.entropy_coef_adjust_rate)
            else:
                self.entropy_coef *= (1 - self.entropy_coef_adjust_rate)
            # Keep the coefficient within reasonable bounds
            self.entropy_coef = max(min(self.entropy_coef, self.initial_entropy_coef), 0.01)

    def update(self, batches):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        actor_loss_accumulator = []
        critic_loss_accumulator = []
        entropy_accumulator = []
        for batch in batches:
            # Print the overall structure of the batch
            print("Batch structure:")
            for k, v in batch.items():
                print(f"{k}: {v.shape}")

            # Move batch data to the appropriate device (GPU if available)
            batch = {k: v.to(device) for k, v in batch.items()}

            total_loss, actor_loss, critic_loss, entropy = self.compute_losses(batch)
            print("Processed a batch.")

            # Accumulate losses and entropy for averaging
            actor_loss_accumulator.append(actor_loss.item())
            critic_loss_accumulator.append(critic_loss.item())
            entropy_accumulator.append(entropy.item())

            # Optimization step for each batch
            self.optimizer.zero_grad()
            total_loss.backward()
            if self.max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(self.actor_critic.parameters(), self.max_grad_norm)
            self.optimizer.step()

        avg_entropy = np.mean(entropy_accumulator)
        # Update the EMA of entropy after each update cycle
        self.update_ema_entropy(avg_entropy)
        # Adjust the entropy coefficient based on the EMA and target entropy
        self.sigmoid_adjustment()

        avg_actor_loss = sum(actor_loss_accumulator) / len(actor_loss_accumulator)
        avg_critic_loss = sum(critic_loss_accumulator) / len(critic_loss_accumulator)
        # avg_entropy = sum(entropy_accumulator) / len(entropy_accumulator)

        # self.adjust_entropy_bonus(avg_entropy)

        return avg_actor_loss, avg_critic_loss, avg_entropy


class RolloutBuffer:
    def __init__(self, environment_training_settings ):
        self.buffer_size = environment_training_settings['buffer_size']
        self.state_shape = environment_training_settings['state_shape']
        self.action_shape = environment_training_settings['action_shape']
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.reset()


    def reset(self):
        # Initialize buffers as empty lists for dynamic appending
        self.states = []
        self.actions = []
        self.rewards = []
        self.dones = []
        self.log_probs = []
        self.values = []
        self.returns = []
        self.advantages = []

    def add(self, state, action, reward, done, log_prob, value):
        # Directly append new experience data to the buffers
        self.states.append(state.detach().cpu().numpy() if isinstance(state, torch.Tensor) else state)
        self.actions.append(action.detach().cpu().numpy() if isinstance(action, torch.Tensor) else action)
        self.rewards.append(reward.detach().cpu().numpy() if isinstance(reward, torch.Tensor) else reward)
        self.dones.append(done.detach().cpu().numpy() if isinstance(done, torch.Tensor) else done)
        self.log_probs.append(log_prob.detach().cpu().numpy() if isinstance(log_prob, torch.Tensor) else log_prob)
        self.values.append(value.detach().cpu().numpy() if isinstance(value, torch.Tensor) else value)

    def compute_returns_and_advantage(self, last_value, gamma, lambda_gae):
        # Convert lists to numpy arrays for efficient computation
        self.rewards = np.array(self.rewards)
        self.values = np.array(self.values)
        self.dones = np.array(self.dones)

        # Initialize returns and advantages arrays
        self.returns = np.zeros_like(self.rewards)
        self.advantages = np.zeros_like(self.rewards)

        last_advantage = 0
        for t in reversed(range(len(self.rewards))):
            if t == len(self.rewards) - 1:
                next_non_terminal = 1.0 - self.dones[t]
                next_value = last_value
            else:
                next_non_terminal = 1.0 - self.dones[t + 1]
                next_value = self.values[t + 1]

            delta = self.rewards[t] + gamma * next_value * next_non_terminal - self.values[t]
            last_advantage = delta + gamma * lambda_gae * next_non_terminal * last_advantage
            self.returns[t] = last_advantage + self.values[t]
            self.advantages[t] = last_advantage

        # Normalize advantages
        advantages_mean = np.mean(self.advantages)
        advantages_std = np.std(self.advantages) + 1e-8
        self.advantages = (self.advantages - advantages_mean) / advantages_std

    def get_sequential_mini_batches(self, mini_batch_size):
        num_samples = len(self.states)
        for start in range(0, num_samples, mini_batch_size):
            indices = slice(start, start + mini_batch_size)
            yield self._create_batch(indices)

    def get_random_mini_batches(self, mini_batch_size):
        num_samples = len(self.states)
        indices = np.random.permutation(num_samples)
        for start in range(0, num_samples, mini_batch_size):
            batch_indices = indices[start:start + mini_batch_size]
            yield self._create_batch(batch_indices)

    def _create_batch(self, indices):
        # Convert lists to numpy arrays only when creating batches
        return {
            'states': torch.tensor(np.array(self.states)[indices], dtype=torch.float32).to(self.device),
            'actions': torch.tensor(np.array(self.actions)[indices], dtype=torch.float32).to(self.device),
            'rewards': torch.tensor(np.array(self.rewards)[indices], dtype=torch.float32).to(self.device),
            'dones': torch.tensor(np.array(self.dones)[indices], dtype=torch.bool).to(self.device),
            'log_probs': torch.tensor(np.array(self.log_probs)[indices], dtype=torch.float32).to(self.device),
            'values': torch.tensor(np.array(self.values)[indices], dtype=torch.float32).to(self.device),
            'returns': torch.tensor(np.array(self.returns)[indices], dtype=torch.float32).to(self.device),
            'advantages': torch.tensor(np.array(self.advantages)[indices], dtype=torch.float32).to(self.device),
        }
    # def reset(self):
    #     # Resetting buffer arrays with appropriate shapes and types
    #     self.states = torch.zeros((self.buffer_size, *self.state_shape), dtype=torch.float32, device=self.device)
    #     self.actions = torch.zeros((self.buffer_size, *self.action_shape), dtype=torch.float32, device=self.device)
    #     self.rewards = torch.zeros(self.buffer_size, dtype=torch.float32, device=self.device)
    #     self.dones = torch.zeros(self.buffer_size, dtype=torch.bool, device=self.device)
    #     self.log_probs = torch.zeros(self.buffer_size, dtype=torch.float32, device=self.device)
    #     self.values = torch.zeros(self.buffer_size, dtype=torch.float32, device=self.device)
    #     self.returns = torch.zeros(self.buffer_size, dtype=torch.float32, device=self.device)
    #     self.advantages = torch.zeros(self.buffer_size, dtype=torch.float32, device=self.device)
    #     self.ptr = 0
    # def reset(self):
    #     # Use numpy arrays for storing experiences
    #     self.states = np.zeros((self.buffer_size, *self.state_shape), dtype=np.float32)
    #     self.actions = np.zeros((self.buffer_size, *self.action_shape), dtype=np.float32)
    #     self.rewards = np.zeros(self.buffer_size, dtype=np.float32)
    #     self.dones = np.zeros(self.buffer_size, dtype=bool)
    #     self.log_probs = np.zeros(self.buffer_size, dtype=np.float32)
    #     self.values = np.zeros(self.buffer_size, dtype=np.float32)
    #     self.returns = np.zeros(self.buffer_size, dtype=np.float32)
    #     self.advantages = np.zeros(self.buffer_size, dtype=np.float32)
    #     self.ptr = 0


    # # def add(self, state, action, reward, done, log_prob, value):
    # #     # Adding a new experience to the buffer
    # #     if self.ptr >= self.buffer_size:
    # #         raise IndexError('RolloutBuffer is full, cannot add more samples.')
    # #     self.states[self.ptr] = torch.tensor(state, device=self.device)
    # #     self.actions[self.ptr] = torch.tensor(action, device=self.device)
    # #     self.rewards[self.ptr] = reward
    # #     self.dones[self.ptr] = done
    # #     self.log_probs[self.ptr] = log_prob
    # #     self.values[self.ptr] = value
    # #     self.ptr += 1

    # def add(self, state, action, reward, done, log_prob, value):
    #     # Adding a new experience to the buffer, automatically detaching tensors if necessary
    #     if self.ptr >= self.buffer_size:
    #         raise IndexError('RolloutBuffer is full, cannot add more samples.')

    #     # Automatically detach and convert to NumPy if input is a tensor
    #     self.states[self.ptr] = state.detach().cpu().numpy() if isinstance(state, torch.Tensor) else state
    #     self.actions[self.ptr] = action.detach().cpu().numpy() if isinstance(action, torch.Tensor) else action
    #     self.rewards[self.ptr] = reward.detach().cpu().numpy() if isinstance(reward, torch.Tensor) else reward
    #     self.dones[self.ptr] = done.detach().cpu().numpy() if isinstance(done, torch.Tensor) else done
    #     self.log_probs[self.ptr] = log_prob.detach().cpu().numpy() if isinstance(log_prob, torch.Tensor) else log_prob
    #     self.values[self.ptr] = value.detach().cpu().numpy() if isinstance(value, torch.Tensor) else value

    #     self.ptr += 1

    # # def compute_returns_and_advantage(self, last_value, gamma, lambda_gae):
    # #     # Computing returns and advantages using GAE
    # #     last_advantage = 0
    # #     for i in reversed(range(self.buffer_size)):
    # #         if i == self.buffer_size - 1:
    # #             next_non_terminal = 1.0 - self.dones[i]
    # #             next_value = last_value
    # #         else:
    # #             next_non_terminal = 1.0 - self.dones[i + 1]
    # #             next_value = self.values[i + 1]

    # #         delta = self.rewards[i] + gamma * next_value * next_non_terminal - self.values[i]
    # #         last_advantage = delta + gamma * lambda_gae * next_non_terminal * last_advantage
    # #         self.returns[i] = last_advantage + self.values[i]
    # #         self.advantages[i] = last_advantage

    # #     # Normalize the advantages
    # #     self.advantages = (self.advantages - self.advantages.mean()) / (self.advantages.std() + 1e-8)
    # def compute_returns_and_advantage(self, last_value, gamma, lambda_gae):
    #     # Start modifications here
    #     last_advantage = 0
    #     # Loop through the buffer backwards, but only up to self.ptr
    #     for i in reversed(range(self.ptr)):
    #         if i == self.ptr - 1:
    #             next_non_terminal = 1.0 - self.dones[i]
    #             next_value = last_value
    #         else:
    #             next_non_terminal = 1.0 - self.dones[i + 1]
    #             next_value = self.values[i + 1]

    #         delta = self.rewards[i] + gamma * next_value * next_non_terminal - self.values[i]
    #         last_advantage = delta + gamma * lambda_gae * next_non_terminal * last_advantage
    #         self.returns[i] = last_advantage + self.values[i]
    #         self.advantages[i] = last_advantage

    #     # Normalize the advantages, but only for the filled part of the buffer
    #     advantages_slice = self.advantages[:self.ptr]
    #     advantages_mean = advantages_slice.mean()
    #     advantages_std = advantages_slice.std() + 1e-8
    #     self.advantages[:self.ptr] = (advantages_slice - advantages_mean) / advantages_std
    # # def get_mini_batches(self, mini_batch_size):
    # #     # Generating mini-batches for PPO updates
    # #     batch_start = torch.randperm(self.ptr)
    # #     for start in range(0, self.ptr, mini_batch_size):
    # #         batch_indices = batch_start[start:start+mini_batch_size]
    # #         yield {
    # #             'states': self.states[batch_indices],
    # #             'actions': self.actions[batch_indices],
    # #             'rewards': self.rewards[batch_indices],
    # #             'dones': self.dones[batch_indices],
    # #             'log_probs': self.log_probs[batch_indices],
    # #             'values': self.values[batch_indices],
    # #             'returns': self.returns[batch_indices],
    # #             'advantages': self.advantages[batch_indices],
    # #         }
    # def get_sequential_mini_batches(self, mini_batch_size):
    #     # Sequentially generate mini-batches
    #     for start in range(0, self.ptr, mini_batch_size):
    #         end = start + mini_batch_size
    #         yield self._create_batch(slice(start, end))

    # def get_random_mini_batches(self, mini_batch_size):
    #     # Randomly generate mini-batches
    #     batch_indices = np.random.permutation(self.ptr)
    #     for start in range(0, self.ptr, mini_batch_size):
    #         end = start + mini_batch_size
    #         selected_indices = batch_indices[start:end]
    #         yield self._create_batch(selected_indices)

    # def _create_batch(self, indices):
    #     # Helper method to create a batch from indices
    #     if isinstance(indices, slice):
    #         batch_indices = range(indices.start or 0, min(indices.stop, self.ptr), indices.step or 1)
    #     else:
    #         batch_indices = indices
    #     return {
    #         'states': torch.tensor(self.states[batch_indices], dtype=torch.float32),
    #         'actions': torch.tensor(self.actions[batch_indices], dtype=torch.float32),
    #         'rewards': torch.tensor(self.rewards[batch_indices], dtype=torch.float32),
    #         'dones': torch.tensor(self.dones[batch_indices], dtype=torch.bool),
    #         'log_probs': torch.tensor(self.log_probs[batch_indices], dtype=torch.float32),
    #         'values': torch.tensor(self.values[batch_indices], dtype=torch.float32),
    #         'returns': torch.tensor(self.returns[batch_indices], dtype=torch.float32),
    #         'advantages': torch.tensor(self.advantages[batch_indices], dtype=torch.float32),
    #     }
    # # def get_mini_batches(self, mini_batch_size):
    # #     # Generating mini-batches with numpy arrays converted to PyTorch tensors
    # #     batch_start = np.random.permutation(self.ptr)
    # #     for start in range(0, self.ptr, mini_batch_size):
    # #         batch_indices = batch_start[start:start+mini_batch_size]
    # #         yield {
    # #             'states': torch.tensor(self.states[batch_indices], device=self.device),
    # #             'actions': torch.tensor(self.actions[batch_indices], device=self.device),
    # #             'rewards': torch.tensor(self.rewards[batch_indices], device=self.device),
    # #             'dones': torch.tensor(self.dones[batch_indices], dtype=torch.bool, device=self.device),
    # #             'log_probs': torch.tensor(self.log_probs[batch_indices], device=self.device),
    # #             'values': torch.tensor(self.values[batch_indices], device=self.device),
    # #             'returns': torch.tensor(self.returns[batch_indices], device=self.device),
    # #             'advantages': torch.tensor(self.advantages[batch_indices], device=self.device),
    # #         }


#     for epoch in range(num_epochs):
#         logging.info(f'Starting Epoch: {epoch + 1}/{num_epochs}')

#         # Initialize metrics
#         episode_rewards = []
#         episode_lengths = []
#         all_drawdowns = []
#         all_sharpe_ratios = []

#         # Start training
#         state = env.reset()
#         episode_reward = 0
#         timestep = 0
#         progress_bar = tqdm(total=hyperparams['total_timesteps'])

#         while timestep < hyperparams['total_timesteps']:
#             # Sample an action from the policy network
#             action, log_prob, value = actor_critic.act(state)
#             action = action.cpu().numpy()  # Ensure action is a numpy array to be passed to env.step()

#             # Step through the environment using the sampled action
#             next_state, reward, done, _ = env.step(action)
#             rollouts.add(state, action, reward, log_prob, value, done)

#             episode_reward += reward
#             state = next_state
#             timestep += 1
#             progress_bar.update(1)

#             if done:
#                 episode_rewards.append(episode_reward)
#                 episode_lengths.append(timestep)  # Assuming timestep tracks the current step per episode

#                 # Log episode details and calculate performance metrics
#                 drawdown = calculate_drawdown(episode_rewards)
#                 sharpe_ratio = calculate_sharpe_ratio(episode_rewards)
#                 all_drawdowns.append(drawdown)
#                 all_sharpe_ratios.append(sharpe_ratio)

#                 logging.info(f'Episode finished. Reward: {episode_reward:.2f}, Length: {timestep}')
#                 logging.info(f'Episode drawdown: {drawdown:.4f}, Sharpe Ratio: {sharpe_ratio:.4f}')

#                 episode_reward = 0
#                 state = env.reset()

#         # Perform update if enough data is collected
#             if done or len(rollouts.states) >= hyperparams['mini_batch_size'] * hyperparams['ppo_epoch']:
#                 last_value = actor_critic.critic_forward(state)
#                 rollouts.compute_returns_and_advantage(last_value, hyperparams['gamma'], hyperparams['gae_lambda'])
#                 avg_actor_loss, avg_critic_loss, avg_entropy_loss = ppo.update(rollouts)
#                 rollouts.reset()

#                 logging.info(f'Update at timestep {timestep}, Actor Loss: {avg_actor_loss:.4f}, Critic Loss: {avg_critic_loss:.4f}, Entropy: {avg_entropy_loss:.4f}')

#                 if done:
#                     # Get the next starting point based on elapsed time
#                     next_index = env.find_next_tick_index(tick_data, current_index, elapsed_time)
#                     state = tick_data[next_index]['state']
#                     timestep = next_index

#             # Save and evaluate periodically
#             if timestep % hyperparams['checkpoint_interval'] == 0:
#                 checkpoint_path = f'ppo_checkpoint_{timestep}.pth'
#                 torch.save(actor_critic.state_dict(), checkpoint_path)
#                 logging.info(f'Model checkpoint saved at timestep {timestep}')

#             if timestep % hyperparams['evaluation_interval'] == 0:
#                 avg_reward = evaluate(actor_critic, env, hyperparams['num_evaluation_episodes'])
#                 logging.info(f'Evaluation at timestep {timestep}, Average Reward: {avg_reward:.2f}')
#                 progress_bar.set_postfix(avg_reward=avg_reward)

#         progress_bar.close()
#         logging.info(f'Training completed after {timestep} timesteps')
#         logging.info(f'Average reward: {np.mean(episode_rewards):.2f} +/- {np.std(episode_rewards):.2f}')
#         logging.info(f'Average episode length: {np.mean(episode_lengths)}')
#         logging.info(f'Max drawdown during training: {min(all_drawdowns):.4f}')
#         logging.info(f'Average Sharpe Ratio: {mstats.gmean(all_sharpe_ratios):.4f}')


# def evaluate(actor_critic_network, env, num_episodes, deterministic=False):
#     total_rewards = 0
#     for episode in range(num_episodes):
#         state = env.reset()
#         done = False
#         episode_reward = 0
#         while not done:
#             # Get the action from the network
#             if deterministic:
#                 # Assuming the network has a method to get the deterministic action
#                 action, _ = actor_critic_network.get_deterministic_action(state)
#             else:
#                 # Or get the stochastic action if deterministic is False
#                 action, _ = actor_critic_network.act(state)

#             action = action.cpu().numpy()  # Convert to numpy array if necessary
#             state, reward, done, _ = env.step(action)
#             episode_reward += reward

#         total_rewards += episode_reward
#         logging.info(f'Episode: {episode}, Reward: {episode_reward}')

#     average_reward = total_rewards / num_episodes
#     logging.info(f'Average reward: {average_reward}')
#     return average_reward



class ContinuousTradingEnvironment(gym.Env):
    """
    A continuous market making environment for reinforcement learning using tick data.

    Attributes:
        action_space: The space of possible actions. This is continuous and could represent
                      the size and price of the orders you want to place or cancel.
        observation_space: The space of possible states, which could include features derived from
                           tick data such as smoothed prices, inventory levels, recent trade volumes, etc.
        current_step: Current step in the simulation.
        done: Indicator for when the environment should be reset.
        state: The current state of the environment.
        tick_data: The tick data for the trading environment.
        inventory_level: The current level of inventory held.
        cash_balance: The current cash balance from trading activities.
    """
    def __init__(self, volatility, portfolio_manager, capital, trading_manager ):
        super(ContinuousTradingEnvironment, self).__init__()

        # Define the range for bid and ask price adjustments relative to some reference (e.g., mid-price)
        self.bid_price_adjustment_range = [-0.0006, 0.0001]  # +/- 1% for bid price adjustment
        self.ask_price_adjustment_range = [-0.0001, 0.0006]  # +/- 1% for ask price adjustment

        # Define the range for the size of buy and sell orders
        self.buy_order_size_range = [-0.001, 0.05]  # 1% to 10% for buy orders
        self.sell_order_size_range = [-0.001, 0.05]  # 1% to 10% for sell orders

        # Define the range for order cancellation intensity and strategy
        self.cancellation_intensity_range = [0, 1]  # Range for how many orders to cancel
        self.cancellation_strategy_range = [0, 1]  # Range for what type of orders to cancel

        # Define the range for the hold action (0: not holding, 1: holding)
        self.hold_range = [0, 1]

        # Define the range for the target inventory level
        self.target_inventory_level_range = [-0.35, 0.35]  # -50% to +50% of some reference value

        # Define the continuous action space boundaries
        self.action_space = gym.spaces.Box(
            low=np.array([
                self.bid_price_adjustment_range[0],
                self.ask_price_adjustment_range[0],
                self.buy_order_size_range[0],
                self.sell_order_size_range[0],
                self.hold_range[0],
                self.target_inventory_level_range[0],
                self.cancellation_intensity_range[0],
                self.cancellation_strategy_range[0],
            ]),
            high=np.array([
                self.bid_price_adjustment_range[1],
                self.ask_price_adjustment_range[1],
                self.buy_order_size_range[1],
                self.sell_order_size_range[1],
                self.hold_range[1],
                self.target_inventory_level_range[1],
                self.cancellation_intensity_range[1],
                self.cancellation_strategy_range[1],
            ]),
            dtype=np.float32
        )

    # Other initialization steps...


        # Define observation space
        # Adjust the shape based on the number of features in your state representation
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(66,),  # Example size, should match your state representation
            dtype=np.float32
        )
        #self.split_order_threshold = 0.5  # Example threshold value
        self.initial_capital = capital
        # Initialize environment state
        # self.tick_data = tick_data
        self.one_hour_window = timedelta(hours=1)  # 1-hour window for data
        self.price_ticks = deque(maxlen=35)  # Store (timestamp, price) tuples
        self.tick_window = deque(maxlen=35)  # Additional window for tick data processing
        self.last_ingestion_timestamp = None
        self.last_exchange_timestamp = None
        # self.latency_data = latency_data
        #self.current_step = 0
        self.done = False
        self.inventory_level = 0
        self.cash_balance = self.initial_capital
        self.RELATIVE_ORDER_SIZE_THRESHOLD = 0.00005  # 0.005% of total balance

        self.own_trades = []  # A list to keep track of the agent's trades
        self.time_discount_factor = 0.99  # Example value for time discounting
        self.trade_cost = 0.01  # Example fixed cost per trade, could also be a percentage
        self.spread_cost = 0.01  # Spread cost as a percentage of the trade price
        self.commission_rate = 0.002  # Commission rate per trade
        # Initialize other attributes such as rate limiter
        # max_num_orders, max_ratecount, ratecount_decay, decay_interval
        self.rate_limiter = RateLimiter(max_num_orders=70, max_ratecount=55, ratecount_decay=-3.75, decay_interval=1) #the real mximum orders is 225 and the max ratecount is 180
        self.optimal_time_window = 252  # Example for a one-year window in trading days
        self.max_allowed_drawdown = 0.003  # .3% drawdown
        #self.volatility_window = 30  # Example for a one-month window in trading days
        #self.portfolio_values = []  # To keep track of portfolio values for drawdown calculation
        #self.liquidity_reward_scaling_factor = 1.0  # Adjust as necessary
        self.inventory = 0  # Current inventory level
        self.inventory_target = 0  # Desired inventory level, which can be adaptive
        self.max_inventory_risk = 0.3  # Maximum inventory risk tolerance (example: 10% of capital)
        #self.volatility_window = 30  # Window for volatility calculation
        #self.latency_manager = LatencyManager()
        # self.model_computation_time_tracker = ModelComputationTimeTracker()
        self.volatility_manager = volatility
        self.portfolio_manager = portfolio_manager
        self.order_manager = OrderManager(self.portfolio_manager)

        self.orderbook = OrderBookManager()
        self.trade_update_manager = trading_manager
        self.market_data_transformation = MarketStateCalculator(self.rate_limiter, self.volatility_manager, self.order_manager, self.initial_capital)

        self.reward_manager = Reward(self.volatility_manager, self.cash_balance, self.max_inventory_risk, self.order_manager, self.market_data_transformation, self.rate_limiter)


        self.last_exchange_timestamp = None
        self.last_ingestion_timestamp = None
        self.last_order_execution_timestamp = None
        self.model_calculation_time = None
        '''
         self.best_bid_price = None
        self.best_ask_price = None
        self.book_depth = None
        self.bid_ask_spread = None
        self.volume_imbalance = None
        self.order_flow = None
        self.last_traded_price = None
        self.last_trade_volume = None
        self.rolling_trade_volume = None
        self.time_since_last_trade = None
        self.volatility = None
        self.inventory_level = None
        self.unrealized_pnl = None
        self.accumulated_profit_loss = None
        self.time_since_last_action = None
        self.historical_actions = None
        self.api_call_count = None
        self.rate_limit_window_reset_time = None
        self.var = None
        self.liquidity = None

        '''

        # Reset the environment to start
        # self.reset()


    # def reset(self):
    #     """
    #     Reset the environment to an initial state.
    #     """
    #     self.current_step = 0
    #     self.done = False
    #     self.inventory_level = 0
    #     self.cash_balance = 0
    #     self.state = self._get_initial_state()
    #     return self.state


    def reset(self):
        # Reset the environment to an initial state
        self.current_step = 0
        self.done = False

        self.inventory = 0
        self.cash_balance = self.initial_capital
        self.pending_actions.clear()
        self.current_time = 0
        self.order_book_window.clear()
        self.state = self._get_initial_state()
        return self.state

    # def _ingest_data(self, data):
    #     # Record the current time as ingestion timestamp
    #     self.last_ingestion_timestamp = time.time()

    #     # Record the exchange timestamp from the data
    #     self.last_exchange_timestamp = data['exchange_timestamp']

    # def _get_initial_state(self):
    #     """
    #     Returns an initial state for the environment.
    #     """
    #     # Initialize state, e.g., using the first few ticks or a fixed starting point
    #     self.latency_feature = self._calculate_latency_feature(self.current_step)

    #     return np.zeros(self.observation_space.shape)


    # def check_if_done_post_penalty(self):
    #     """
    #     Check if the environment should terminate after applying a rate limit penalty.
    #     """
    #     # Example condition: if cash balance falls below a threshold
    #     if self.cash_balance < self.minimum_cash_threshold:
    #         return True  # Terminate the episode
    #     # Add any additional terminal checks here as needed
    #     return False  # Continue the episode

    def calculate_order_size(self, market_depth, trade_imbalance, recent_trade_volume, current_price, target_inventory_level, order_size_adjustment):
        # Calculate the inventory gap based on the target inventory level
        inventory_gap = (self.max_inventory_capacity * target_inventory_level) - self.current_inventory

        # Maximum order size based on a percentage of capital
        max_order_size_as_percentage_of_capital = 0.05 * self.initial_capital / current_price

        # Target order size considering inventory gap and capital constraints
        target_order_size = min(abs(inventory_gap), max_order_size_as_percentage_of_capital)

        # Minimum order size based on the current price
        min_order_size = 0.0001 * current_price
        target_order_size = max(target_order_size, min_order_size)

        # Adjust the order size based on the action's size adjustment
        adjustment_scale = (order_size_adjustment + 1) / 2  # Normalize from [-1, 1] to [0, 1]
        adjusted_order_size = min_order_size + (max_order_size_as_percentage_of_capital - min_order_size) * adjustment_scale

        # Final order size is the lesser of the risk-adjusted size and the adjusted size
        final_order_size = min(adjusted_order_size, target_order_size)

        # Determine the direction of the order: buy, sell, or hold
        order_direction = 1 if inventory_gap > 0 else -1 if inventory_gap < 0 else 0

        # Implement the hold logic: if direction is hold, set the order size to zero
        if order_direction == 0:
            final_order_size = 0

        # Apply the direction to the final order size
        final_order_size *= order_direction

    #     return final_order_size



    def process_cancellation(self, model, intensity, strategy, current_simulation_time):
        # Access active orders from OrderManager
        active_orders = model.environment.order_manager.active_orders

        # Determine the number of orders to cancel
        num_orders_to_cancel = int(intensity * len(active_orders))

        # Advanced order selection based on strategy
        orders_to_cancel = self.select_orders_for_cancellation( strategy, num_orders_to_cancel, active_orders)

        # Cancel the selected orders
        for order in orders_to_cancel:
            self.cancel_order(model, order, current_simulation_time)
            # Update the rate limiter for each canceled order
            order_age = current_simulation_time - order.time_placed
            model.environment.rate_limiter.can_cancel_order(order_age, current_simulation_time)

# Other methods remain the same


    def select_orders_for_cancellation(self, strategy, num_orders_to_cancel, active_orders):
        if strategy <= 0.80:
            print("Executing cancellation logic: Selecting least profitable orders")
            return self.select_least_profitable_orders(num_orders_to_cancel, active_orders)
        else:
            print("Executing cancellation logic: Selecting orders furthest from market")
            return self.select_orders_furthest_from_market(num_orders_to_cancel, active_orders)
        # else:
            #return self.select_mix_of_old_and_new_orders( num_orders_to_cancel, active_orders, current_time)

    # Implement the remaining selection methods similarly, using 'active_orders' parameter

    def cancel_order(self, model, order, current_simulation_time):
        # Logic to cancel the order in the model's LOB instance and OrderManager
        model.lob_instance.remove_order(order.order_id)
        model.environment.order_manager.cancel_order(order.order_id, current_simulation_time)

    def select_least_profitable_orders(self, num_orders_to_cancel, active_orders):
        if len(active_orders) <= 2:
            print("Not enough orders to cancel while maintaining at least one order per side.")
            return []

        # Split orders by side
        buy_orders = [order for order in active_orders if order.side == 'buy']
        sell_orders = [order for order in active_orders if order.side == 'sell']

        # Ensure not to cancel all orders of one side
        if len(buy_orders) == 1 or len(sell_orders) == 1:
            num_orders_to_cancel = min(num_orders_to_cancel, len(active_orders) - 2)

        current_market_price = self.market_data_transformation.cached_mid_price
        orders_with_profitability = [(order, (current_market_price - order.price) * order.size) for order in active_orders]
        orders_sorted_by_profitability = sorted(orders_with_profitability, key=lambda x: x[1])
        least_profitable_orders = [order for order, _ in orders_sorted_by_profitability[:num_orders_to_cancel]]

        return least_profitable_orders

    def select_orders_furthest_from_market(self, num_orders_to_cancel, active_orders):
        if len(active_orders) <= 2:
            print("Not enough orders to cancel while maintaining at least one order per side.")
            return []

        # Split orders by side
        buy_orders = [order for order in active_orders if order.is_buy == 'buy']
        sell_orders = [order for order in active_orders if order.is_buy == 'sell']

        # Ensure not to cancel all orders of one side
        if len(buy_orders) == 1 or len(sell_orders) == 1:
            num_orders_to_cancel = min(num_orders_to_cancel, len(active_orders) - 2)

        current_mid_price = self.market_data_transformation.cached_mid_price
        orders_with_distance = [(order, abs(order.price - current_mid_price)) for order in active_orders]
        orders_sorted_by_distance = sorted(orders_with_distance, key=lambda x: x[1], reverse=True)
        furthest_orders = [order for order, _ in orders_sorted_by_distance[:num_orders_to_cancel]]

        return furthest_orders



    def select_mix_of_old_and_new_orders(self, num_orders_to_cancel, active_orders):
        old_orders = [order for order in active_orders if self.calculate_order_age(order) > 300]
        # strategically_important_orders = self.identify_strategically_important_orders(old_orders)
        return old_orders


    # def calculate_order_age(self, order):
    #     #current_time = datetime.utcnow()

    #     print(f"Debug: Current time: {current_time}, Order placed time: {order.time_placed}, Order age: {order_age} seconds")

    #     return (current_time - order.time_placed)

    def calculate_recent_price_change(self):
        if len(self.price_data) < 2:
            return 0
        return (self.price_data[-1] - self.price_data[-2]) / self.price_data[-2] * 100



    def _calculate_pnl(self):
        # Calculate the profit or loss since the last action
        # Assuming we have a self.last_trade_price and self.current_trade_price
        pnl = (self.current_trade_price - self.last_trade_price) * self.inventory_level
        return pnl





    # def render(self, mode='human', close=False):
    #     """
    #     Render the environment for human viewing.

    #     The rendering can be in different modes; if 'human', it prints the current state's
    #     summary to the console. If 'system', it logs these details to a file. Additional
    #     modes like 'graphical' can be added for more complex visualizations, such as using
    #     matplotlib for plotting the portfolio value over time.
    #     """
    #     if close:
    #         # Close any existing renderings
    #         plt.close() if self.fig else None
    #         return

    #     if mode == 'human':
    #         # Print out a summary of the current state for human understanding
    #         print(f"Step: {self.current_step}")
    #         print(f"Current Portfolio Value: {self.cash_balance + self.inventory_level * self.current_price}")
    #         print(f"Current Inventory Level: {self.inventory_level}")
    #         print(f"Current Cash Balance: {self.cash_balance}")
    #         # Optionally, add more detailed summaries such as recent trades, etc.

    #     elif mode == 'system':
    #         # For system mode, you could log these details to a file instead
    #         with open('render_log.txt', 'a') as f:
    #             f.write(f"Step: {self.current_step}, Portfolio Value: {self.cash_balance + self.inventory_level * self.current_price}\n")

    #     elif mode == 'graphical':
    #         # For graphical mode, render the state using matplotlib or another plotting library
    #         if not hasattr(self, 'fig') or self.fig is None:
    #             self.fig, self.ax = plt.subplots(1, 1)
    #             plt.ion()
    #             plt.show()

    #         # Update the plot for the current step
    #         self.ax.clear()
    #         self.ax.plot(self.portfolio_values, label='Portfolio Value')
    #         self.ax.set_title(f"Current Step: {self.current_step}")
    #         self.ax.set_xlabel('Step')
    #         self.ax.set_ylabel('Portfolio Value')
    #         self.ax.legend()
    #         plt.draw()
    #         plt.pause(1)  # Pause briefly to allow the plot to be updated


class Order:
    def __init__(self, order_id, size, price, placement_time, is_buy):
        self.order_id = order_id
        self.size = size
        self.price = price
        self.time_placed = placement_time
        self.is_buy = is_buy
        self.filled = 0
        self.canceled = False  # Track if the order was canceled


class OrderManager:
    def __init__(self, portfolio_manager):
        self.active_orders = []  # Track active orders
        self.cancelled_orders = deque()  # Use deque for cancelled orders
        self.cancellation_window = timedelta(minutes=6)  # 6-minute window for cancellations

        self.target_inventory = 0  # Target inventory level to maintain

        self.inventory = 0  # Track the current inventory level
        self.average_buy_price = 0  # Average buy price of the inventory
        self.average_sell_price = 0  # Average sell price of the inventory
        self.realized_pnl = 0  # Realized PnL
        self.unrealized_pnl = 0  # Unrealized PnL
        self.maker_fee_percentage = 0.0016  # Maker fee of 0.16%
        self.portfolio_manager = portfolio_manager

    def update_target_inventory(self, target):
        self.target_inventory = target

    def get_target_inventory(self):
        return self.target_inventory

    def cancel_order(self, order_id, current_time_microseconds):
            print(f"Attempting to cancel order with order_id: {order_id}. Received current_time (UNIX timestamp in microseconds): {current_time_microseconds}")

            order = self.find_order_by_id(order_id)
            if order:
                order.canceled = True  # Mark the order as canceled
                self.active_orders.remove(order)
                # Record the cancellation with its timestamp (in microseconds)
                self.cancelled_orders.append({'order_id': order_id, 'cancel_time': current_time_microseconds})

            # Remove cancellations older than the cancellation window
            # Convert self.cancellation_window to microseconds for comparison
            cancellation_window_microseconds = self.cancellation_window.total_seconds() * 1e6
            print(f'Cancel_order function cancellation_window:{cancellation_window_microseconds}')

            while self.cancelled_orders and current_time_microseconds - self.cancelled_orders[0]['cancel_time'] > cancellation_window_microseconds:
                self.cancelled_orders.popleft()


    def calculate_recent_cancellations_weight(self, current_time_microseconds, decay_rate):
        weight = 0
        for cancellation in self.cancelled_orders:
            time_since_cancellation_microseconds = current_time_microseconds - cancellation['cancel_time']
            time_since_cancellation_seconds = time_since_cancellation_microseconds / 1e6
            decayed_weight = np.exp(-decay_rate * time_since_cancellation_seconds)
            weight += decayed_weight
        return weight


    def calculate_cancellation_penalty(self, order, recent_cancellations_weight):
        cancellation_penalty_coefficient = 10  # Example coefficient, adjust as needed
        # Apply a higher penalty for a higher weight of recent cancellations
        penalty = cancellation_penalty_coefficient * recent_cancellations_weight
        # Cap the penalty to avoid excessively high values

        max_penalty = 50 # Example maximum penalty value
        return min(penalty, max_penalty)

    # def calculate_time_in_queue_reward(self ,order, current_time):
    #     time_in_queue = current_time - order.placement_time
    #     # Use a non-linear function for reward calculation (e.g., logarithmic)
    #     time_in_queue_coefficient = 0.05  # Example coefficient, adjust as needed
    #     reward = time_in_queue_coefficient * np.log(1 + time_in_queue)
    #     # Cap the reward to avoid excessively high values
    #     max_reward = 100  # Example maximum reward value
    #     return min(reward, max_reward)


    def calculate_time_in_queue_reward(self, order, current_time_microseconds):
        print(f"Debug: current_time (microseconds): {current_time_microseconds}, order.placement_time (microseconds): {order.placement_time}")

        time_in_queue_seconds = (current_time_microseconds - order.placement_time) / 1e6
        print(f"Debug: time_in_queue (in seconds): {time_in_queue_seconds}")

        time_in_queue_coefficient = 0.7
        reward = time_in_queue_coefficient * np.log(1 + time_in_queue_seconds)
        print(f"Debug: Calculated reward before capping: {reward}")

        max_reward = 100
        final_reward = min(reward, max_reward)
        print(f"Debug: Final reward after capping: {final_reward}")
        sys.exit()
        return final_reward

    def calculate_order_stability_reward(self, order, current_time, recent_cancellations_weight):
        time_in_queue_reward = self.calculate_time_in_queue_reward(order, current_time)
        cancellation_penalty = self.calculate_cancellation_penalty(order, recent_cancellations_weight)

        total_reward = time_in_queue_reward - cancellation_penalty
        return total_reward



    def place_order(self, order_id, size, price, is_buy, time):

        order = Order(order_id, size, price, time, is_buy)
        self.active_orders.append(order)
        return order

    def remove_order(self, order_to_remove):
        self.active_orders = [order for order in self.active_orders if order.order_id != order_to_remove.order_id]

    #order_id, matched_volume, matched_price, is_buy

    def process_order_execution(self, order_id, filled_size, current_market_price):
        order = self.find_order_by_id(order_id)
        if order:
            previous_filled = order.filled
            order.filled += filled_size
            filled_this_execution = filled_size - previous_filled
            self.update_inventory(order, filled_this_execution,current_market_price)
            self.update_realized_pnl(order, filled_this_execution, current_market_price)
            if order.filled == order.size:
                self.active_orders.remove(order)
            self.update_positions_and_pnl(current_market_price)

    def update_realized_pnl(self, order, filled_size, current_market_price):
        maker_fee = current_market_price * filled_size * self.maker_fee_percentage

        if order.is_buy == 'buy':
            # Buying to close a short position or adding to a long position.
            if self.inventory < 0:
                # Closing or reducing a short position.
                pnl_change = (self.average_sell_price - current_market_price) * filled_size
            else:
                # Adding to a long position doesn't realize PnL.
                # Realized PnL occurs when selling, so no PnL change here for buys that don't close shorts.
                pnl_change = 0
        else:
            # Selling to open a short position or closing a long position.
            if self.inventory > 0:
                # Closing or reducing a long position.
                pnl_change = (current_market_price - self.average_buy_price) * filled_size
            else:
                # Opening or adding to a short position.
                # For simplicity, we'll not calculate PnL change here since it's realized when the position is closed.
                # However, if you want to track unrealized PnL for short positions, additional logic would be needed.
                pnl_change = 0

        # Adjust the PnL change by subtracting the maker fee.
        if pnl_change != 0:
            pnl_change -= maker_fee

        # Update the realized PnL.
        self.realized_pnl = pnl_change
        # Update the portfolio value based on the PnL change.
        self.portfolio_manager.update_portfolio_value(pnl_change)

    def update_inventory(self, order, filled_size, current_market_price):
        # Calculate the cost of the filled portion of the order
        cost = filled_size * current_market_price

        if order.is_buy:
            # If buying, increase inventory
            new_inventory = self.inventory + filled_size
            if self.inventory >= 0:
                # If previously long or flat, adjust average buy price
                total_cost = self.average_buy_price * self.inventory + cost
                self.average_buy_price = total_cost / new_inventory if new_inventory != 0 else 0
            else:
                # If previously short, cover the short position
                self.average_sell_price = (self.average_sell_price * -self.inventory - cost) / new_inventory if new_inventory != 0 else 0
        else:
            # If selling, decrease inventory
            new_inventory = self.inventory - filled_size
            if self.inventory <= 0:
                # If previously short or flat, adjust average sell price
                total_cost = self.average_sell_price * -self.inventory + cost
                self.average_sell_price = total_cost / -new_inventory if new_inventory != 0 else 0
            else:
                # If previously long, reduce the long position
                self.average_buy_price = (self.average_buy_price * self.inventory - cost) / new_inventory if new_inventory != 0 else 0

        self.inventory = new_inventory


    # def update_inventory(self, order, filled_size, current_market_price):
    #     if order.is_buy:
    #         total_buy_value = self.average_buy_price * max(self.inventory, 0) + current_market_price * filled_size
    #         new_buy_inventory = max(self.inventory, 0) + filled_size
    #         self.average_buy_price = total_buy_value / new_buy_inventory if new_buy_inventory else 0
    #     else:
    #         total_sell_value = self.average_sell_price * max(-self.inventory, 0) + current_market_price * filled_size
    #         new_sell_inventory = max(-self.inventory, 0) + filled_size
    #         self.average_sell_price = total_sell_value / new_sell_inventory if new_sell_inventory else 0

    #     self.inventory += filled_size if order.is_buy else -filled_size
    # def update_inventory(self, order, filled_size, current_market_price):
    #     if order.is_buy:
    #         if self.inventory >= 0:
    #             # Buying to open or add to a long position
    #             total_buy_value = self.average_buy_price * self.inventory + current_market_price * filled_size
    #             self.inventory += filled_size
    #             self.average_buy_price = total_buy_value / self.inventory if self.inventory > 0 else 0
    #         else:
    #             # Buying to cover a short position or flipping to long
    #             if filled_size <= abs(self.inventory):
    #                 # Partially covering a short position
    #                 self.inventory += filled_size
    #             else:
    #                 # Flipping from short to long
    #                 over_covered = filled_size - abs(self.inventory)
    #                 self.inventory = over_covered
    #                 self.average_buy_price = current_market_price  # New long position
    #                 # Reset average sell price as we no longer have a short position
    #                 self.average_sell_price = 0
    #     else:
    #         # Selling
    #         if self.inventory > 0:
    #             # Selling part of a long position or flipping to short
    #             if filled_size <= self.inventory:
    #                 self.inventory -= filled_size
    #             else:
    #                 # Flipping from long to short
    #                 over_sold = filled_size - self.inventory
    #                 self.inventory = -over_sold  # New short position
    #                 self.average_sell_price = current_market_price  # New short position
    #                 # Reset average buy price as we no longer have a long position
    #                 self.average_buy_price = 0
    #         else:
    #             # Adding to a short position or opening a new short position
    #             total_sell_value = self.average_sell_price * abs(self.inventory) + current_market_price * filled_size
    #             self.inventory -= filled_size
    #             self.average_sell_price = total_sell_value / abs(self.inventory) if self.inventory < 0 else 0

        # No need to update positions and PnL here



    def update_positions_and_pnl(self, current_market_price):
        if self.inventory > 0:
            self.unrealized_pnl = (current_market_price - self.average_buy_price) * self.inventory
        elif self.inventory < 0:
            self.unrealized_pnl = (self.average_sell_price - current_market_price) * (-self.inventory)
        else:
            self.unrealized_pnl = 0


    def calculate_total_active_orders_value(self):
        total_order_value = 0
        for order in self.active_orders:
            if not order.canceled:
                order_value = order.price * order.size
                total_order_value += order_value
        return total_order_value

    def calculate_combined_inventory_and_orders_value(self):
        inventory_value = self.calculate_inventory_value()
        active_orders_value = self.calculate_total_active_orders_value()
        return inventory_value + active_orders_value

    def calculate_inventory_value(self):
        if self.inventory > 0:  # Long position
            inventory_value = self.inventory * self.average_buy_price
        elif self.inventory < 0:  # Short position
            inventory_value = abs(self.inventory) * self.average_sell_price
        else:  # No position
            inventory_value = 0
        return inventory_value

    # def calculate_inventory_percentage(self, aum):
    #     inventory_value = self.calculate_inventory_value()
    #     if aum == 0:
    #         return 0
    #     return (inventory_value / aum) * 100



    def find_order_by_id(self, order_id):
        for order in self.active_orders:
            if order.order_id == order_id:
                return order
        return None


    def get_average_price(self):
        # Assuming you're tracking average price in some way
        return self.average_price

    def get_realized_pnl(self):
        return self.realized_pnl

    def get_average_buy_price(self):
        return self.average_buy_price

    def get_average_sell_price(self):
        return self.average_sell_price

    def get_best_and_worst_bid_with_total_volume(self):
        """
        Get the highest and lowest bid prices among active orders, along with the total volume of the highest bid.
        """
        bid_orders = [(order.price, order.size) for order in self.active_orders if order.is_buy and not order.canceled]
        if not bid_orders:
            return (None, None, None)  # No bids

        # Aggregate volumes by price
        bid_volumes_by_price = {}
        for price, volume in bid_orders:
            bid_volumes_by_price[price] = bid_volumes_by_price.get(price, 0) + volume

        if bid_volumes_by_price:
            best_bid_price = max(bid_volumes_by_price.keys())
            worst_bid_price = min(bid_volumes_by_price.keys())
            best_bid_volume = bid_volumes_by_price[best_bid_price]
        else:
            return (None, None, None)

        return best_bid_price, worst_bid_price, best_bid_volume


    def get_best_and_worst_ask_with_total_volume(self):
        """
        Get the lowest and highest ask prices among active orders, along with the total volume of the lowest ask.
        """
        ask_orders = [(order.price, order.size) for order in self.active_orders if not order.is_buy and not order.canceled]
        if not ask_orders:
            return (None, None, None)  # No asks

        # Aggregate volumes by price
        ask_volumes_by_price = {}
        for price, volume in ask_orders:
            ask_volumes_by_price[price] = ask_volumes_by_price.get(price, 0) + volume

        if ask_volumes_by_price:
            best_ask_price = min(ask_volumes_by_price.keys())
            worst_ask_price = max(ask_volumes_by_price.keys())
            best_ask_volume = ask_volumes_by_price[best_ask_price]
        else:
            return (None, None, None)

        return best_ask_price, worst_ask_price, best_ask_volume





    # ... Additional methods as needed ...

    # ... Additional methods as needed ...


class PortfolioManager:
    def __init__(self, initial_capital):
        self.initial_capital = initial_capital
        self.portfolio_values = deque()  # Store (timestamp, portfolio value) tuples
        self.realized_pnl = 0
        self.current_capital = initial_capital
        self.time_window = timedelta(minutes=30)

    def update_portfolio_value(self, pnl_change):
        """
        Update the portfolio value with the realized PnL change.
        """
        current_time = datetime.now()
        self.realized_pnl += pnl_change
        self.current_capital += pnl_change
        new_portfolio_value = self.get_latest_portfolio_value() + pnl_change
        self.portfolio_values.append((current_time, new_portfolio_value))

        # Remove old data outside the specified time window
        while self.portfolio_values and self.portfolio_values[0][0] < current_time - self.time_window:
            self.portfolio_values.popleft()

    def get_latest_portfolio_value(self):
        """
        Get the most recent portfolio value.
        """
        return self.portfolio_values[-1][1] if self.portfolio_values else self.initial_capital

    def get_realized_pnl(self):
        """
        Get the total realized PnL.
        """
        return self.realized_pnl
    def get_AUM(self):
        """
        Calculate and return the current AUM.
        """
        # Implement the logic to calculate AUM
        # For simplicity, let's assume AUM is equal to current capital in this example
        return self.current_capital

class VolatilityManager:
    def __init__(self, one_hour_window=timedelta(hours=1),one_minute_window=timedelta(minutes=1), seven_minute_window=timedelta(minutes=7),thirty_five_minute_window=timedelta(minutes=35)):
        self.price_ticks = deque()  # Store (timestamp, price) tuples
        self.one_hour_window = one_hour_window  # Time window for price ticks
        self.mad_window_sizes = [timedelta(minutes=1), timedelta(minutes=7), timedelta(minutes=60)]
        self.mad_volatilities = {}  # Store the latest MAD volatility measures


        # New functionality for DEMA
        self.one_minute_window = one_minute_window
        self.seven_minute_window = seven_minute_window
        self.thirty_five_minute_window = thirty_five_minute_window
        self.one_minute_trades = deque()
        self.seven_minute_trades = deque()
        self.thirty_five_minute_trades = deque()



    def update_tick_data(self, new_trade_data, current_time):
        print("new_trade_data:", new_trade_data)

        # Check if new_trade_data is a pandas DataFrame
        if isinstance(new_trade_data, pd.DataFrame):
            for index, trade in new_trade_data.iterrows():
                price = trade.get('price')
                timestamp = trade.get('local_timestamp', current_time)
                if pd.notnull(price) and pd.notnull(timestamp):
                    self.price_ticks.append((timestamp, price))
                else:
                    print("Trade data format not recognized or data is incomplete")

        # Check if new_trade_data is a pandas Series (single trade)
        elif isinstance(new_trade_data, pd.Series):
            price = new_trade_data.get('price')
            timestamp = new_trade_data.get('local_timestamp', current_time)
            if pd.notnull(price) and pd.notnull(timestamp):
                self.price_ticks.append((timestamp, price))
            else:
                print("Trade data format not recognized or data is incomplete")

        # Check if new_trade_data is a single dictionary (single trade)
        elif isinstance(new_trade_data, dict):
            price = new_trade_data.get('price')
            timestamp = new_trade_data.get('local_timestamp', current_time)
            if price is not None and timestamp is not None:
                self.price_ticks.append((timestamp, price))
            else:
                print("Trade data format not recognized or data is incomplete")

        # Check if new_trade_data is a list (multiple trades)
        elif isinstance(new_trade_data, list):
            for trade in new_trade_data:
                price = trade.get('price')
                timestamp = trade.get('local_timestamp', current_time)
                if price is not None and timestamp is not None:
                    self.price_ticks.append((timestamp, price))
                else:
                    print("Trade data format not recognized or data is incomplete")

        # Handle unexpected data types
        else:
            print("Unexpected data type for new_trade_data")
    #    return

        #self.price_ticks.append((timestamp, price))
        # Print statements for debugging
        print("Debugging update_tick_data method:")
        print(f"Timestamp (float): {timestamp}")
        print(f"Type of timestamp: {type(timestamp)}")
        print(f"One hour window (timedelta): {self.one_hour_window}")
        print(f"Type of one_hour_window: {type(self.one_hour_window)}")

        # Convert one_hour_window from timedelta to microseconds
        one_hour_window_microseconds = self.one_hour_window.total_seconds() * 1e6  # 1e6 microseconds in one second

        # Remove ticks outside the one-hour window
        while self.price_ticks and self.price_ticks[0][0] < timestamp - one_hour_window_microseconds:
            self.price_ticks.popleft()

# ,
#         # Remove old data outside the specified time window
#         while self.price_ticks and self.price_ticks[0][0] < timestamp - self.one_hour_window:
#             self.price_ticks.popleft()

        # Update MAD volatility measures
        #self.update_mad_volatility_measures(timestamp)
         # Update the one_minute_trades and five_minute_trades queues
        self._update_ema_trades(new_trade_data, current_time, self.one_minute_trades, self.one_minute_window)
        self._update_ema_trades(new_trade_data, current_time, self.seven_minute_trades, self.seven_minute_window)

        self._update_ema_trades(new_trade_data, current_time, self.thirty_five_minute_trades, self.thirty_five_minute_window)


    # def _update_ema_trades(self, new_trade_data, current_time, trade_queue, time_window):
    #     """
    #     Update the trade queue with new data and remove outdated trades.
    #     """
    #     if isinstance(new_trade_data, dict):
    #         price = new_trade_data.get('price')
    #         timestamp = new_trade_data.get('local_timestamp', current_time)
    #     elif isinstance(new_trade_data, list) and len(new_trade_data) >= 2:
    #         price_index = 4
    #         timestamp_index = 2
    #         price = new_trade_data[price_index]
    #         timestamp = new_trade_data[timestamp_index]
    #     else:
    #         return

    #     trade_queue.append((timestamp, price))
    #     while trade_queue and trade_queue[0][0] < timestamp - time_window.total_seconds() * 1e6:
    #         trade_queue.popleft()

    def _update_ema_trades(self, new_trade_data, current_time, trade_queue, time_window):
        """
        Update the trade queue with new data and remove outdated trades.
        """
        print(f"Updating EMA trades with new_trade_data: {new_trade_data}")

        # Check if new_trade_data is a list of dictionaries
        if isinstance(new_trade_data, list) and all(isinstance(item, dict) for item in new_trade_data):
            for trade in new_trade_data:
                price = trade.get('price')
                timestamp = trade.get('local_timestamp', current_time)
                print(f"Adding trade to queue: Timestamp: {timestamp}, Price: {price}")
                if price is not None and timestamp is not None:
                    # Convert timestamp to the appropriate format if necessary
                    # Assuming timestamp is already in the correct format
                    trade_queue.append((timestamp, price))
                else:
                    print("Some trades in the list have incomplete data")
        # Check if new_trade_data is a pandas Series
        elif isinstance(new_trade_data, pd.Series):
            price = new_trade_data.get('price')
            timestamp = new_trade_data.get('local_timestamp', current_time)
            if price is not None and timestamp is not None:
                trade_queue.append((timestamp, price))
            else:
                print("Trade data in pandas Series is incomplete")
        else:
            print("new_trade_data format not recognized or data is incomplete")
            return

        # print(f"Adding trade to queue: Timestamp: {timestamp}, Price: {price}")
        # trade_queue.append((timestamp, price))

        print(f"Trade queue before removal: {list(trade_queue)}")
        while trade_queue and trade_queue[0][0] < timestamp - time_window.total_seconds() * 1e6:
            removed_trade = trade_queue.popleft()
            print(f"Removed outdated trade: {removed_trade}")

        print(f"Trade queue after removal: {list(trade_queue)}")



    def calculate_ema(self, trade_queue, window_size):
        if not trade_queue:
            return 0
        if len(trade_queue) < window_size:
            # Return the last available EMA if insufficient data
            return trade_queue[-1][1]

        alpha = 2 / (window_size + 1)
        ema = trade_queue[0][1]  # Initial EMA value is the first price
        for timestamp, price in trade_queue:
            ema = (price * alpha) + (ema * (1 - alpha))
        return ema

    def calculate_dema(self, trade_queue, window_size):
        ema = self.calculate_ema(trade_queue, window_size)
        if ema == 0:
            return 0

        ema_of_ema = self.calculate_ema_of_ema(trade_queue, window_size)
        dema = 2 * ema - ema_of_ema
        return dema

    def calculate_ema_of_ema(self, trade_queue, window_size):
        if not trade_queue:
            return 0
        if len(trade_queue) < window_size:
            # Return the last available EMA of EMA if insufficient data
            return trade_queue[-1][1]

        alpha = 2 / (window_size + 1)
        ema_values = self.calculate_ema(trade_queue, window_size, ema_only=True)
        ema_of_ema = ema_values[0]
        for ema in ema_values:
            ema_of_ema = (ema * alpha) + (ema_of_ema * (1 - alpha))
        return ema_of_ema

    def calculate_ema(self, trade_queue, window_size, ema_only=False):
        if not trade_queue:
            return 0

        alpha = 2 / (window_size + 1)
        ema_values = []
        ema = trade_queue[0][1]  # Initial EMA value is the first price
        for timestamp, price in trade_queue:
            ema = (price * alpha) + (ema * (1 - alpha))
            ema_values.append(ema)

        return ema_values if ema_only else ema


    def update_mad_volatility_measures(self, current_time):
        for window_size in self.mad_window_sizes:
            mad_volatility = self.calculate_mad_volatility(window_size, current_time)
            window_minutes = int(window_size.total_seconds() / 60)
            self.mad_volatilities[f"{window_minutes}m"] = mad_volatility
            print(f"MAD volatility for {window_minutes} minutes: {mad_volatility}")

    def calculate_mad_volatility(self, window_size, current_time):
        """
        Calculate the MAD volatility for the specified window size.
        """

        # Convert window_size from timedelta to microseconds
        window_size_in_units = window_size.total_seconds() * 1e6  # Convert to microseconds if necessary

        # Filter prices within the window
        window_prices = [price for (time, price) in self.price_ticks if current_time - time <= window_size_in_units]
        print(f"Window prices: {window_prices}")
        print(f"price ticks: {self.price_ticks}")


        # # Calculate
        # window_prices = [price for time, price in self.price_ticks if time >= current_time - window_size]

        if len(window_prices) < 2:
            return 0

        return self.preprocess_feature_to_single_value(window_prices)


    def preprocess_feature_to_single_value(self, feature):
        median = np.median(feature)
        mad = np.median([np.abs(val - median) for val in feature])
        print(f"MAD: {mad}")

        if mad != 0:
            normalized_feature = [0.6745 * (val - median) / mad for val in feature]
            return np.median(normalized_feature)
        else:
            return 0

    def get_mad_volatility_measures(self):
        return self.mad_volatilities

    def calculate_hourly_volatility(self):
        """
        Calculate volatility based on the last hour's price data.
        """
        if not self.price_ticks or len(self.price_ticks) < 2:
            return None  # Not enough data to calculate volatility

        # Extract prices from the tick data
        recent_prices = np.array([price for _, price in self.price_ticks])
        returns = np.diff(np.log(recent_prices))
        hourly_volatility = np.std(returns)
        return hourly_volatility

    def get_hourly_volatility(self):
        return self.calculate_hourly_volatility()

    def get_seven_minute_dema(self):
        """
        Calculate and return the 5-minute Double Exponential Moving Average (DEMA).
        """
        if not self.seven_minute_trades:
            return None  # Return None if there is no trade data

        window_size = 1000  # The window size for 5-minute DEMA
        return self.calculate_dema(self.seven_minute_trades, window_size)
    def get_one_minute_dema(self):
        """
        Calculate and return the one-minute Double Exponential Moving Average (DEMA).
        """
        if not self.one_minute_trades:
            return None  # Return None if there is no trade data

        # Determine an appropriate window size for the one-minute DEMA calculation
        # This could be based on the number of trades you expect in one minute, for example:
        window_size = 1000  # Adjust this based on your expected number of trades per minute
        return self.calculate_dema(self.one_minute_trades, window_size)

    def get_thirty_five_minute_dema(self):
        """
        Calculate and return the 35-minute Double Exponential Moving Average (DEMA).
        """
        if not self.thirty_five_minute_trades:
            return None  # Return None if there is no trade data

        # Determine an appropriate window size for the 35-minute DEMA calculation
        # This could be based on the number of trades you expect in 35 minutes, for example:
        window_size = 10000  # Example, adjust based on your expected number of trades per minute
        return self.calculate_dema(self.thirty_five_minute_trades, window_size)





class Reward:
    pnl_bounds = (-5000, 5000)  # Example range for PnL
    drawdown_penalty_bounds = (-500, 0)  # Drawdowns can only penalize, not reward
    liquidity_reward_bounds = (0, 500)  # Rewards for liquidity provision
    inventory_penalty_bounds = (-500, 0)  # Penalties for inventory mismanagement
    queue_position_reward_bounds = (0, 500)  # Rewards for good queue positioning
    market_share_reward_bounds = (0, 100)  # Rewards for increased market share
    rate_limit_penalty_bounds = (-2000, 0)  # Significant penalty for hitting rate limit
    def __init__(self, volatility_manager, initial_capital, max_inventory_risk, order_manager, market_state_calculator, rate_limiter):

        self.volatility_manager = volatility_manager
        self.initial_capital = initial_capital
        self.max_inventory_risk = max_inventory_risk
        #self.max_rate_limit_penalty = max_rate_limit_penalty
        self.order_manager = order_manager  # Instance of OrderManager
        self.decay_rate = 0.1  # Example decay rate for cancellation weight calculation
        self.time_in_queue_coefficient = 0.7
        self.cancellation_penalty_coefficient = 2
        self.max_cancellation_penalty = 50
        self.max_time_in_queue_reward = 2
        self.market_state_calculator = market_state_calculator

        self.drawdon_max = .007
        # Initialize weights for each reward component
        self.weights = {
            'pnl': 0.20,
            'realized_pnl': 0.35,
            'drawdown': 0.20,
            'liquidity': 0.10,
            'inventory': 0.15,
            'queue_position': 0.15,
           # 'market_share': 0.05
        }
        self.drawdown_threshold = 0.003  # Default threshold for drawdown penalty
        self.penalty_multiplier = 10  # Default multiplier for the drawdown penalty
        self.five_min_window = timedelta(minutes=5)  # 1-hour window
        #self.price_ticks = deque()  # Store (timestamp, price) tuples
        self.volatility = 0
        self.volatility_threshold = 0.013
        self.window_size = 500 # drawdown window
        self.rate_limiter = rate_limiter
        self.realized_pnl = 0  # Initialize realized PnL
        self.realized_pnl_multiplier = 10
        self.rate_limit_hits = 0  # Counter for rate limit breaches
        self.initial_rate_limit_penalty = -1000 # Initial penalty for hitting the rate limit
        #self.max_rate_limit_penalty = max_rate_limit_penalty
        # self.order_window_size = order_window_size
        self.filled_limit_orders_window = deque()  # Initialize deque for filled limit orders
        self.total_limit_orders_window = deque()  # Initialize deque for total limit orders

        # self.filled_limit_orders_window = deque(maxlen=order_window_size)
        # self.total_limit_orders_window = deque(maxlen=order_window_size)
        self.liquidity_reward_scaling_factor = 1.0
        self.inventory_penalty_scaling_factor = 2  # Adjustable scaling factor for inventory penalty

        #self.portfolio_values = [initial_capital] * window_size
        # Initialize other necessary attributes
        self.max_inventory_risk = max_inventory_risk
        self.inventory_target = 0  # Updated dynamically
        self.current_inventory = 0  # This should be updated with the actual inventory level


    # Reward function components
    # ... [Other methods for PnL, liquidity reward, etc.] ...
    # def _calculate_queue_position_reward(self):
    #     # Assuming you have a method to get the current state of the order book and your order details
    #     order_details = self.get_order_details()
    #     order_book_state = self.get_order_book_state()

    #     # Weights for each queue position metric
    #     time_weight = 0.5  # Weight for the time since order placement
    #     volume_weight = 0.3  # Weight for the relative volume
    #     events_weight = 0.2  # Weight for the order book events

    #     # Calculate metrics that contribute to the queue position
    #     order_time_since_placement = self.calculate_time_since_placement(order_details.timestamp)
    #     relative_volume = self.calculate_relative_volume(order_details.volume, order_book_state.total_volume_at_price)
    #     order_book_events = self.calculate_order_book_events(order_book_state.recent_changes)

    #     # Combining the metrics into a queue position score with weights
    #     queue_position_score = (
    #         time_weight * order_time_since_placement +
    #         volume_weight * (1 - relative_volume) +
    #         events_weight * order_book_events
    #     )

    #     # Normalize the combined score to be between 0 and 1
    #     normalized_queue_position_score = self.normalize_metric(queue_position_score, min_value=0, max_value=time_weight + volume_weight + events_weight)

    #     # Invert the score for the reward: a higher score (closer to the front of the queue) yields a higher reward
    #     queue_position_reward = 1 - normalized_queue_position_score

    #     return queue_position_reward
    def calculate_recent_cancellations_weight(self, current_time_microseconds, decay_rate):
        weight = 0
        for cancellation in self.order_manager.cancelled_orders:
            time_since_cancellation_microseconds = current_time_microseconds - cancellation['cancel_time']
            print(f'time_since_cancellation_microseconds: {time_since_cancellation_microseconds}')
            #time_since_cancellation_seconds = time_since_cancellation_microseconds / 1e6
            decayed_weight = np.exp(-decay_rate * time_since_cancellation_microseconds)
            weight += decayed_weight
        return weight


    def calculate_cancellation_penalty(self, recent_cancellations_weight):
        penalty = self.cancellation_penalty_coefficient * recent_cancellations_weight
        return min(penalty, self.max_cancellation_penalty)

    # def calculate_time_in_queue_reward(self, order, current_time):
    #     time_in_queue = current_time - order.time_placed
    #     reward = self.time_in_queue_coefficient * np.log(1 + time_in_queue)
    #     return min(reward, self.max_time_in_queue_reward)

    def calculate_time_in_queue_reward(self, order, current_time):
        # Log the types and values for debugging
        print(f"Current time (type: {type(current_time)}): {current_time}")
        print(f"Order time placed (type: {type(order.time_placed)}): {order.time_placed}")

        # Convert Unix timestamp in microseconds to seconds
        current_time_seconds = current_time / 1e6
        order_time_placed_seconds = order.time_placed / 1e6

        # Calculate the time in queue in seconds
        time_in_queue_seconds = current_time_seconds - order_time_placed_seconds
        print(f"Time in queue (seconds): {time_in_queue_seconds}")

        # Convert seconds to hours or your desired unit of time
        #time_in_queue_hours = time_in_queue_seconds / 3600
        reward = self.time_in_queue_coefficient * np.log(1 + time_in_queue_seconds)
        reward = min(reward, self.max_time_in_queue_reward)
        print(f"Calculated reward: {reward}")

        return reward

    def calculate_order_stability_reward(self, order, current_time):
        recent_cancellations_weight = self.calculate_recent_cancellations_weight(current_time,  0.01)
        time_in_queue_reward = self.calculate_time_in_queue_reward(order, current_time)
        cancellation_penalty = self.calculate_cancellation_penalty(recent_cancellations_weight)
        total_reward = time_in_queue_reward - cancellation_penalty
        return total_reward

    def _calculate_queue_position_reward(self, current_time):
        """
        Calculate rewards for queue position stability for each active order.
        """
        # current_time = datetime.now()
        total_queue_reward = 0
        for order in self.order_manager.active_orders:
            order_stability_reward = self.calculate_order_stability_reward(order, current_time)
            total_queue_reward += order_stability_reward
        return total_queue_reward

    def apply_penalty_for_rate_limit(self):
        """
        Apply a progressively increasing penalty for each rate limit breach.

        """
        usage_percentage = self.rate_limiter.get_usage_percentage()

        # Define the maximum penalty
        max_penalty = -1 # Maximum penalty

        # Calculate penalty based on usage
        penalty = (usage_percentage / 100) * max_penalty
        # penalty = np.clip(penalty, max_penalty, 0)  # Ensure the penalty does not exceed the maximum

        return penalty









    # Volatility calculations
    # def _calculate_price_volatility(self):
    #     log_returns = np.diff(np.log(self.recent_prices))
    #     price_volatility = np.std(log_returns)
    #     return price_volatility

    # def _calculate_volume_volatility(self):
    #     volume_changes = np.diff(self.recent_volumes) / self.recent_volumes[:-1]
    #     volume_volatility = np.std(volume_changes)
    #     return volume_volatility

    # def _calculate_spread_volatility(self):
    #     spread_changes = np.diff(self.recent_spreads) / self.recent_spreads[:-1]
    #     spread_volatility = np.std(spread_changes)
    #     return spread_volatility

    # def _calculate_multifactor_volatility(self):
    #     weights = {'price': 0.6, 'volume': 0.2, 'spread': 0.2}
    #     multifactor_volatility = (weights['price'] * self._calculate_price_volatility() +
    #                               weights['volume'] * self._calculate_volume_volatility() +
    #                               weights['spread'] * self._calculate_spread_volatility())
    #     return multifactor_volatility

    # Inventory-related calculations
    def _calculate_inventory_deviation(self):
        current_inventory = self.order_manager.inventory
        deviation = abs(current_inventory - self.order_manager.target_inventory)
        print(f"Current inventory: {current_inventory}")
        print(f"Inventory target: {self.order_manager.target_inventory}")
        print(f"Inventory deviation: {deviation}")
        return deviation

    def _calculate_liquidity_factor(self):
        # Retrieve the latest sophisticated liquidity value
        liquidity = self.market_state_calculator.liquidity
        if liquidity is not None and liquidity != 0:
            liquidity_factor = 1 / liquidity
        else:
            liquidity_factor = 0  # Handle cases where liquidity is not available or zero
        return liquidity_factor

    def _calculate_adaptive_inventory_penalty(self):
        deviation = self._calculate_inventory_deviation()
        #liquidity_factor = self._calculate_liquidity_factor()

        # Amplify penalty using a non-linear function like quadratic
        #amplified_deviation = deviation ** 2

        # Apply the scaling factor and liquidity factor
        inventory_penalty =   deviation
        return -inventory_penalty


    def set_inventory_target(self, target):
        self.inventory_target = target

    # def _adjust_inventory_target(self, volatility):
    #     base_target = self._base_inventory_target()
    #     adjusted_target = base_target * (1 - volatility)
    #     adjusted_target = max(min(adjusted_target, base_target), 0)
    #     return adjusted_target

    def update_order_window(self, filled_orders, total_orders):
        """
        Update the windows of filled and total limit orders with the latest data and timestamps.
        """
        current_time = datetime.now()  # Get the current time

        # Append the new data along with the timestamp
        self.filled_limit_orders_window.append((current_time, filled_orders))
        self.total_limit_orders_window.append((current_time, total_orders))

        # Remove old data outside the specified time window from filled orders
        while self.filled_limit_orders_window and self.filled_limit_orders_window[0][0] < current_time - self.five_min_window:
            self.filled_limit_orders_window.popleft()

        # Remove old data outside the specified time window from total orders
        while self.total_limit_orders_window and self.total_limit_orders_window[0][0] < current_time - self.five_min_window:
            self.total_limit_orders_window.popleft()

    def _calculate_liquidity_reward(self):
        """
        Calculate the liquidity reward based on filled limit orders and their proximity to the mid-price,
        normalized by the proximity of all limit orders.
        """
        filled_proximity_rewards = []
        total_proximity_values = []

        for order in self.filled_limit_orders_window:
            filled_proximity_reward = self._calculate_proximity_reward(order)
            filled_proximity_rewards.append(filled_proximity_reward)

        for order in self.total_limit_orders_window:
            total_proximity_value = self._calculate_proximity_reward(order, filled=False)
            total_proximity_values.append(total_proximity_value)

        # Aggregate proximity rewards and normalize
        if filled_proximity_rewards and total_proximity_values:
            liquidity_reward = sum(filled_proximity_rewards) / sum(total_proximity_values) if sum(total_proximity_values) > 0 else 0
        else:
            liquidity_reward = 0

        return liquidity_reward

    def _calculate_decay_rate(self):
        # Higher volatility can lead to a higher decay rate,
        # meaning distance from the mid-price becomes more significant
        base_decay_rate = 0.1  # Base decay rate
        volatility_factor = self.volatility / self.volatility_threshold  # self.volatility_threshold defined elsewhere
        decay_rate = base_decay_rate * volatility_factor
        return max(0.01, min(decay_rate, 1))  # Ensure decay rate is within reasonable bounds

    def _calculate_proximity_reward(self, order, filled=True):
        """
        Calculate the proximity reward for an order.
        """
        # Calculate the distance from the mid-price
        distance_from_mid_price = abs(order['price'] - self.mid_price)

        # Define a decay rate based on market volatility or other factors
        decay_rate = self._calculate_decay_rate()  # Implement this method

        # Calculate proximity score
        proximity_score = np.exp(-decay_rate * distance_from_mid_price)

        if filled:
            # Adjust the score based on filled proportion of the order
            filled_proportion = order['filled_quantity'] / order['total_quantity']
            adjusted_proximity_score = proximity_score * filled_proportion
            # Calculate final reward for the order
            reward = adjusted_proximity_score * order['filled_quantity']
            return reward
        else:
            # For total orders, consider the order quantity instead of filled quantity
            return proximity_score * order['total_quantity']



    def _calculate_hourly_volatility(self):
        """
        Retrieve the 60-minute MAD volatility from the VolatilityManager.
        """
        mad_volatilities = self.volatility_manager.price_ticks
        return mad_volatilities  # Retrieve the 60-minute MAD volatility


    # Method to calculate the current rolling drawdown
    def _calculate_adjusted_drawdown(self):
        """
        Calculate the adjusted drawdown considering market conditions.
        """
        ##print(market_volatility)
        # if not isinstance(market_volatility, (float, int)):
        #     raise ValueError(f"Unexpected market_volatility type: {type(market_volatility)}. Expected float or int.")

        if not self.order_manager.portfolio_manager.portfolio_values:
            return 0

        peak_value = max(value for _, value in self.order_manager.portfolio_manager.portfolio_values)
        current_value = self.order_manager.portfolio_manager.get_latest_portfolio_value()

        # Calculate standard deviation of volatility measurements
        # std_dev = np.std(market_volatility)
        # print(std_dev)

        raw_drawdown = (peak_value - current_value) / peak_value
        adjusted_drawdown = raw_drawdown * 10
        # sys.exit()
        return adjusted_drawdown
        # raw_drawdown = (peak_value - current_value) / peak_value if peak_value else 0

        # adjusted_drawdown = raw_drawdown * market_volatility
        # return adjusted_drawdown


    def _non_linear_drawdown_penalty(self):
        """
        Calculate a non-linear penalty for drawdowns exceeding a threshold.
        """
        penalty = self._calculate_adjusted_drawdown()
        # if adjusted_drawdown > self.drawdown_threshold:
        #     exp_argument = self.penalty_multiplier * (adjusted_drawdown - self.drawdown_threshold)
        #     exp_argument_capped = max(exp_argument, -1)  # Example cap to prevent -inf
        #     penalty = -np.exp(exp_argument_capped)
        # else:
        #     penalty = 0
        return -penalty



    def _calculate_position_penalty(self):

        total_position = self.order_manager.calculate_combined_inventory_and_orders_value() #inventory_value + active_orders_value
        position_ratio = total_position / self.order_manager.portfolio_manager.get_AUM()

        penalty_threshold = 0.50  # 50% of AUM
        if position_ratio > penalty_threshold:
            # Calculate penalty based on how much the threshold is exceeded
            penalty = (position_ratio - penalty_threshold) * 20
            return penalty
        else:
            # No penalty if below threshold
            return 0

    def reset_reward_function(self):
        # Reset realized PnL
        self.realized_pnl = 0

        # Reinitialize the portfolio values with the initial capital
        self.portfolio_values.clear()
        self.portfolio_values.append((datetime.now(), self.initial_capital))

        # Reset the rate limit hit counter
        self.rate_limit_hits = 0

        # Reset other reward components as necessary
        # For example, resetting metrics related to market share, liquidity, etc.
        self.liquidity_rewards = 0
        self.inventory_penalties = 0
        # self.queue_position_rewards = 0
        # self.market_share_rewards = 0
        self.drawdown_penalties = 0

        # Any other state variables that contribute to the reward calculation
        # should be reset or reinitialized here as well



    # # The main reward function that combines all components
    # def _get_reward(self,current_time):
    #     # Fetch PnL from OrderManager
    #     self.pnl = self.order_manager.unrealized_pnl  # Unrealized PnL
    #     self.realized_pnl = self.order_manager.get_realized_pnl()  # Assuming this method exists

    #     # Calculate and store reward components
    #     self.drawdown_penalty = self._non_linear_drawdown_penalty()
    #     self.liquidity_reward = self._calculate_liquidity_reward()
    #     self.inventory_penalty = self._calculate_adaptive_inventory_penalty()
    #     #self.queue_position_reward = self._calculate_queue_position_reward()
    #     self.queue_position_reward = self._calculate_queue_position_reward(current_time)

    #     # self.market_share_reward = self._calculate_market_share_reward()
    #     self.rate_limit_penalty = self.apply_penalty_for_rate_limit()
    #     self.calculate_position_penalty = self._calculate_position_penalty()

    #     # Combine the components into a final reward
    #     total_reward = (
    #         self.weights['pnl'] * self.pnl * 20 +
    #         self.weights['realized_pnl'] * self.realized_pnl * 500 +
    #         self.weights['drawdown'] * self.drawdown_penalty +
    #         self.weights['liquidity'] * self.liquidity_reward +
    #         self.weights['inventory'] * self.inventory_penalty +
    #         self.weights['queue_position'] * self.queue_position_reward +
    #         self.calculate_position_penalty +
    #         # self.weights['market_share'] * self.market_share_reward +
    #         self.rate_limit_penalty
    #     )

    #     return total_reward
    # The main reward function that combines all components
    def _get_reward(self, current_time):
        # Fetch PnL from OrderManager
        self.pnl = self.order_manager.unrealized_pnl  # Unrealized PnL
        self.realized_pnl = self.order_manager.get_realized_pnl()  # Assuming this method exists

        # Calculate and store reward components
        self.drawdown_penalty = self._non_linear_drawdown_penalty()
        self.liquidity_reward = self._calculate_liquidity_reward()
        self.inventory_penalty = self._calculate_adaptive_inventory_penalty()
        self.queue_position_reward = self._calculate_queue_position_reward(current_time)
        self.rate_limit_penalty = self.apply_penalty_for_rate_limit()
        self.position_penalty = self._calculate_position_penalty()

        # Debug prints for each component
        print(f"Unrealized PnL: {self.pnl}")
        print(f"Realized PnL: {self.realized_pnl}")
        print(f"Drawdown Penalty: {self.drawdown_penalty}")
        print(f"Liquidity Reward: {self.liquidity_reward}")
        print(f"Inventory Penalty: {self.inventory_penalty}")
        print(f"Queue Position Reward: {self.queue_position_reward}")
        print(f"Rate Limit Penalty: {self.rate_limit_penalty}")
        print(f"Position Penalty: {self.position_penalty}")
        print(f"directional reward: {self.market_state_calculator.alignment_reward}")

        # Combine the components into a final reward
        total_reward = (
            self.weights['pnl'] * self.pnl * 10 +
            self.weights['realized_pnl'] * self.realized_pnl * 200 +
            self.weights['drawdown'] * self.drawdown_penalty +
            self.weights['liquidity'] * self.liquidity_reward +
            self.weights['inventory'] * self.inventory_penalty / 100+
            self.weights['queue_position'] * self.queue_position_reward  +
            self.market_state_calculator.alignment_reward +
            self.position_penalty +
            self.rate_limit_penalty
        )

        # Debug print for total reward
        print(f"Total Reward: {total_reward}")

        return total_reward


    # Getter methods for each reward component
    def get_pnl(self):
        return self.pnl

    def get_realized_pnl(self):
        return self.realized_pnl

    def get_drawdown_penalty(self):
        return self.drawdown_penalty

    def get_liquidity_reward(self):
        return self.liquidity_reward

    def get_inventory_penalty(self):
        return self.inventory_penalty

    def get_queue_position_reward(self):
        return self.queue_position_reward

    def get_rate_limit_penalty(self):
        return self.rate_limit_penalty



# class OrderBookCache:
#     def __init__(self, max_length=100):
#         self.order_book_data_cache = deque(maxlen=max_length)
#         self.trades_data_cache = deque(maxlen=max_length)
#         self.exchange_timestamps = deque(maxlen=max_length)
#         self.local_timestamps = deque(maxlen=max_length)

#         self.latest_order_book_data = np.zeros((20, 4))  # Example dimensions for order book
#         self.latest_trades_data = np.zeros((10, 4))  # Example dimensions for trades data

#     def add_order_book_data(self, order_book_data, exchange_timestamp, local_timestamp=None):
#         if local_timestamp is None:
#             local_timestamp = time.time()
#         self.latest_order_book_data = order_book_data
#         self.order_book_data_cache.append({'order_book': order_book_data, 'timestamp': exchange_timestamp, 'local_timestamp': local_timestamp})
#         self.exchange_timestamps.append(exchange_timestamp)
#         self.local_timestamps.append(local_timestamp)

#     def update_with_order_book_handler_data(self, order_book_handler, exchange_timestamp, local_timestamp=None):
#         if local_timestamp is None:
#             local_timestamp = time.time()
#         order_book_array = order_book_handler.get_order_book_array()
#         self.add_order_book_data(order_book_array, exchange_timestamp, local_timestamp)


#     def add_trades_data(self, trades_data, exchange_timestamp, local_timestamp=None):
#         if local_timestamp is None:
#             local_timestamp = time.time()
#         self.latest_trades_data = trades_data
#         self.trades_data_cache.append({'trades': trades_data, 'timestamp': exchange_timestamp, 'local_timestamp': local_timestamp})
#         self.exchange_timestamps.append(exchange_timestamp)
#         self.local_timestamps.append(local_timestamp)

#     def get_latest_order_book_data(self):
#         return self.latest_order_book_data

#     def get_latest_trades_data(self):
#         return self.latest_trades_data

#     def copy_to_3d_arrays(self):
#         # Adjust the dimensions as necessary
#         num_order_book_slices = len(self.order_book_data_cache)
#         num_trades_slices = len(self.trades_data_cache)

#         order_book_3d = np.zeros((20, 4, num_order_book_slices))
#         trades_data_3d = np.zeros((10, 4, num_trades_slices))
#         exchange_ts_array = np.array(self.exchange_timestamps)
#         local_ts_array = np.array(self.local_timestamps)

#         for i, data in enumerate(self.order_book_data_cache):
#             order_book_3d[:, :, i] = data['order_book']
#         for i, data in enumerate(self.trades_data_cache):
#             trades_data_3d[:, :, i] = data['trades']

#         return order_book_3d, trades_data_3d, exchange_ts_array, local_ts_array
# # Usage example
# cache = OrderBookCache(max_length=100)
# cache.add_order_book_data(new_order_book_data, exchange_timestamp=time.time(), local_timestamp=time.time())
# cache.add_trades_data(new_trades_data, exchange_timestamp=time.time(), local_timestamp=time.time())
# # ... add more data over time ...

# # When needed, copy the cached data to 3D arrays for analysis
# order_book_3d, trades_data_3d, exchange_ts, local_

class OrderBookManager:
    def __init__(self, depth=10, max_updates=2):
        self.bid_order_book = []
        self.ask_order_book = []
        self.depth = depth
        self.order_book_history = deque(maxlen=max_updates)

    def update_order_book(self, order_book_data):
        bid_updates = order_book_data.get('bids', [])
        ask_updates = order_book_data.get('asks', [])

        self._update_order_book_side(self.bid_order_book, bid_updates, 'buy')
        self._update_order_book_side(self.ask_order_book, ask_updates, 'sell')

        current_state = self.get_top_bids_and_asks()
        self.order_book_history.append(current_state)

    def _update_order_book_side(self, order_book, updates, side):
        # Update or append orders
        for price, volume in updates:
            self._update_or_append_order(order_book, price, volume, side)

        # Sort and trim the order book
        if side == 'buy':
            # Sort bids in descending order and keep the top 'self.depth' bids
            order_book.sort(key=lambda x: x[0], reverse=True)
            order_book[:] = order_book[:self.depth]
        elif side == 'sell':
            # Sort asks in ascending order and keep the top 'self.depth' asks
            order_book.sort(key=lambda x: x[0])
            order_book[:] = order_book[:self.depth]

    def _update_or_append_order(self, order_book, price, volume, side):
        for i, (p, v) in enumerate(order_book):
            if p == price:
                if volume > 0:
                    # Replace the volume of the existing order
                    order_book[i] = (price, volume)
                else:
                    # Remove the order if the new volume is zero
                    del order_book[i]
                return
        if volume > 0:
            # Append a new order if the price is not found and volume is non-zero
            order_book.append((price, volume))

    def get_top_bids_and_asks(self):
        top_bids = sorted(self.bid_order_book, key=lambda x: x[0], reverse=True)[:self.depth]
        top_asks = sorted(self.ask_order_book, key=lambda x: x[0])[:self.depth]
        return top_bids, top_asks

    def get_order_book_history(self):
        return list(self.order_book_history)

    def update_from_lob_instance(self, lob_instance):
        bids, asks = lob_instance.get_current_book()

        # Replace the current bid and ask order books with the new data
        self.bid_order_book = sorted(bids, key=lambda x: x[0], reverse=True)[:self.depth]
        self.ask_order_book = sorted(asks, key=lambda x: x[0])[:self.depth]

        # Create a new snapshot of the top bids and asks
        current_state = (self.bid_order_book, self.ask_order_book)

        # Append the new snapshot to the order book history
        self.order_book_history.append(current_state)

        # Optional: Debugging information

class TradeUpdateManager:
    def __init__(self, max_updates=15):
        self.trade_history = deque(maxlen=max_updates)

    def update_trades(self, trades_data):
        """
        Update the trades history with the latest trades data.

        :param trades_data: A list of trade updates. Each update is expected to be a dictionary
                            with keys like 'price', 'volume', and 'timestamp'.
        """
        # for trade in trades_data:
        self.trade_history.append(trades_data)

    def format_and_append_trade_data(self, data, current_unix_timestamp):
        # Check if data is a list containing dictionaries for trades
        #current_unix_timestamp = current_unix_timestamp * 1e6
        if isinstance(data, list) and all(isinstance(item, dict) for item in data):
            # Assume the first item in the list represents the trade data we want to format
            trade_data = data[0]

            formatted_data = {
                'event': 2.0,  # Assuming 'event' is always 2 for trade data
                'local_timestamp': current_unix_timestamp,  # Use the current UNIX timestamp
                'side': 1.0 if trade_data['side'] == 'buy' else -1.0,
                'price': trade_data['price'],
                'volume': trade_data['volume']
            }

            # Convert to pd.Series for consistency
            formatted_series = pd.Series(formatted_data)

            # Append the formatted data to the trade history
            self.trade_history.append(formatted_series)
        else:
            print("Input data format not recognized or not supported.")
            sys.exit()


    def get_latest_trades(self):
        """
        Get the latest trades from the history.

        :return: A list of the latest trade updates.
        """
        return list(self.trade_history)


class MarketStateCalculator:
    def __init__(self, rate_limiter, volatility_manager, order_manager, initial_capital):
        # Inventory and price metrics
        self.inventory_level = 0
        self.average_buy_price = 0
        self.average_sell_price = 0
        self.initial_capital = initial_capital
        # Placeholders for data arrays
        self.trades_data = np.array([])
        #self.order_book = np.zeros((20, 4), dtype=np.float64)

        # Historical and other metrics
        self.historical_returns = np.array([])
        self.api_call_count = 0
        self.rate_limit_window_reset_time = None

        self.liquidity = 0
        # Cached metrics

        self.cached_trades_data = 0.0
        self.cached_order_book = 0.0
        self.cached_vwap = 0.0
        self.cached_price_impact = 0.0
        self.cached_var = 0.0
        self.cached_order_flow_ratio = 0.0
        self.cached_order_flow = 0.0
        self.cached_trading_volumes = 0.0
        self.cached_weighted_price_volatility = 0.0
        self.cached_time_weighted_volume_variability = 0.0
        self.cached_bid_depth = 0.0
        self.cached_order_flow_rate = 0.0
        self.cached_ask_depth = 0.0
        self.cached_total_bid_volume = 0.0
        self.cached_market_depth = 0.0
        self.cached_bid_ask_spread = 0.0
        self.cached_ema_buy = 0.0
        self.cached_ema_sell = 0.0
        self.cached_cv_buy = 0.0
        self.cached_cv_sell = 0.0
        self.cached_bid_skewness = 0.0
        self.cached_ask_skewness = 0.0
        self.cached_bid_slope = 0.0
        self.cached_ask_slope = 0.0
        self.cached_best_ask_volume = 0.0
        self.cached_best_bid_volume = 0.0
        self.cached_best_bid_price = 0.0
        self.cached_best_ask_price = 0.0
        self.cached_half_spread = 0.0
        self.cached_mid_price = 0.0
        self.cached_total_ask_volume = 0.0
        self.cached_top_two_bid_levels_volume = 0.0
        self.cached_top_two_ask_levels_volume = 0.0
        self.cached_buy_order_flow_rate = 0.0
        self.cached_sell_order_flow_rate = 0.0
        self.alignment_reward = 0.0

        self.volatility_manager = volatility_manager

        # Last trade metrics
        self.last_traded_price = 0
        self.last_trade_volume = 0
        self.rolling_trade_volume = deque(maxlen=9)
        self.last_trade_timestamp = 0
        self.time_since_last_trade = 0

        # State vector preparation
        # self.previous_state = np.zeros(num_features)
        # self.state_mean = np.zeros(num_features)
        # self.state_std = np.ones(num_features)

        # External dependencies
        self.rate_limiter = rate_limiter
        self.order_manager = order_manager

        # Data update flags
        self.is_trade_data_updated = False
        self.is_order_book_updated = False

    # Other methods for calculation and updates...


        '''
         def __init__(self):
        self.inventory_level = 0  # Current inventory level
        self.average_buy_price = 0  # Average buy price of the inventory
        # Cached metrics (initialized as needed)
        self.cached_bid_depth = 0
        self.cached_ask_depth = 0
        self.cached_market_imbalance = 0
        self.cached_market_depth = 0
        self.cached_bid_ask_spread = 0
        self.cached_vwap = 0
        self.cached_price_impact = 0
        self.cached_var = 0
        self.cached_order_flow_ratio = 0
        self.cached_order_flow = 0
        self.cached_trading_volumes = 0
        self.cached_weighted_price_volatility = 0
        self.cached_time_weighted_volume_variability = 0
        # Flags for data updates
        self.is_trade_data_updated = False
        self.is_order_book_updated = False
        '''

    def get_current_position(self):
        return self.order_manager.inventory

    def get_average_buy_price(self):
        return self.order_manager.average_buy_price
    def get_average_sell_price(self):
        return self.order_manager.average_sell_price

    def get_unrealized_pnl(self):
        return self.order_manager.unrealized_pnl


    # def calculate_vwap(self):
    #     if self.cached_trades_data is not None:
    #         return np.average(self.cached_trades_data[:, 0], weights=self.cached_trades_data[:, 1])
    #     return None

    def calculate_vwap_multi_snapshot(self, trades_data_snapshots):
        print("trades_data_snapshots:", trades_data_snapshots)  # Print the contents of trades_data_snapshots
        vwap_numerator = 0
        total_volume = 0

        # Iterate through each trade snapshot (pandas Series)
        for trade_data in trades_data_snapshots:
            # Extract the 'price' and 'volume' from each Series
            price = trade_data.get('price')
            volume = trade_data.get('volume')

            # Update the vwap numerator and total volume
            if price is not None and volume is not None:
                vwap_numerator += price * volume
                total_volume += volume

        # Calculate the VWAP if total_volume is greater than 0
        if total_volume > 0:
            vwap = vwap_numerator / total_volume
        else:
            print("Total volume is 0, unable to calculate VWAP.")
            vwap = None

        return vwap

    # def calculate_price_impact(self):
    #     if self.cached_trades_data is not None:
    #         price_changes = np.diff(self.cached_trades_data[:, 0])
    #         volumes = self.cached_trades_data[1:, 1]
    #         return price_changes / volumes
    #     return None
    def calculate_price_impact_multi_snapshot(self, trades_data_snapshots):
        price_impacts = []
        for i in range(1, len(trades_data_snapshots)):
            if isinstance(trades_data_snapshots[i], pd.Series) and isinstance(trades_data_snapshots[i - 1], pd.Series):
                price_change = trades_data_snapshots[i]['price'] - trades_data_snapshots[i - 1]['price']
                volume = trades_data_snapshots[i]['volume']
                price_impacts.append(price_change / volume if volume != 0 else 0)
            else:
                price_impacts.append(None)
        return price_impacts


    # ... Additional metrics calculations like order flow ratio, book depth, skewness, etc. ...
    # def calculate_order_flow_ratio(self):
    #     if self.cached_trades_data is not None:
    #         buy_volume = np.sum(self.cached_trades_data[self.cached_trades_data[:, 3] == 1, 1])
    #         sell_volume = np.sum(self.cached_trades_data[self.cached_trades_data[:, 3] == -1, 1])
    #         order_flow_ratio = buy_volume / sell_volume if sell_volume != 0 else float('inf')
    #         return order_flow_ratio
    #     return None

    def calculate_trade_flow_ratio(self, trades_data_snapshots):
        total_buy_volume = 0
        total_sell_volume = 0

        for trade_data in trades_data_snapshots:
            if 'side' in trade_data and 'volume' in trade_data:
                trade_side = trade_data['side']
                trade_volume = trade_data['volume']

                if trade_side == 1:  # Buy trade
                    total_buy_volume += trade_volume
                elif trade_side == -1:  # Sell trade
                    total_sell_volume += trade_volume

        # Calculate the trade flow ratio
        if total_sell_volume != 0:
            trade_flow_ratio = total_buy_volume / total_sell_volume
        else:
            trade_flow_ratio = 999999 # Avoid division by zero

        return trade_flow_ratio










    # def calculate_book_depth(self):
    #     if self.cached_order_book is not None:
    #         bid_depth = np.count_nonzero(self.cached_order_book[:, 1])  # Count non-zero bid volumes
    #         ask_depth = np.count_nonzero(self.cached_order_book[:, 3])  # Count non-zero ask volumes
    #         return bid_depth, ask_depth
    #     return None, None

    def calculate_most_recent_book_depth(self, order_book_snapshots):
        # Directly use the most recent snapshot, assuming it's always present
        most_recent_snapshot = order_book_snapshots[-1]
        bids, asks = most_recent_snapshot

        # Directly calculate depth and volumes, assuming bids and asks are always present
        bid_depth = len(bids) if bids else 0
        ask_depth = len(asks) if asks else 0
        total_depth = bid_depth + ask_depth
        best_bid_volume = bids[0][1] if bids else 0
        best_ask_volume = asks[0][1] if asks else 0

        return {
            'total_depth': total_depth,
            'bid_depth': bid_depth,
            'ask_depth': ask_depth,
            'best_bid_volume': best_bid_volume,
            'best_ask_volume': best_ask_volume
        }



    # def calculate_historical_returns(self, prices):
    #     returns = np.diff(prices) / prices[:-1]
    #     return returns

    # # Function to calculate Value at Risk (VaR)
    # def calculate_var(self, confidence_level=0.95):
    #     if len(self.historical_returns) > 0:
    #         sorted_returns = np.sort(self.historical_returns)
    #         index = int((1 - confidence_level) * len(sorted_returns))
    #         var = sorted_returns[index]
    #         return var
    #     return None



    def calculate_top_two_volumes(self,order_book_snapshots):
        most_recent_snapshot = order_book_snapshots[-1]
        bids, asks = most_recent_snapshot

        # Calculate the volume for the top two levels of bids and asks
        # Ensure there are at least two bids and two asks before accessing
        bid_volume_top_two = sum(bid[1] for bid in bids[:2]) if len(bids) >= 2 else sum(bid[1] for bid in bids)
        ask_volume_top_two = sum(ask[1] for ask in asks[:2]) if len(asks) >= 2 else sum(ask[1] for ask in asks)

        return bid_volume_top_two, ask_volume_top_two




    # def calculate_bid_ask_spread(self):
    #     if self.cached_order_book is not None:
    #         return self.cached_order_book[0, 2] - self.cached_order_book[0, 0]  # ask - bid
    #     return None
    def calculate_most_recent_bid_ask_spread(self, order_book_snapshots):
        # Directly use the most recent snapshot, assuming it's always present
        most_recent_snapshot = order_book_snapshots[-1]
        bids, asks = most_recent_snapshot

        # Assume bids and asks are always present in the most recent snapshot
        best_bid = bids[0][0] if bids else 0
        best_ask = asks[0][0] if asks else 0
        spread = best_ask - best_bid if bids and asks else 0
        half_spread = spread / 2 if spread is not None else 0
        mid_price = (best_bid + best_ask) / 2 if bids and asks else 0

        return {
            'spread': spread,
            'best_bid': best_bid,
            'best_ask': best_ask,
            'half_spread': half_spread,
            'mid_price': mid_price
    }




    def calculate_order_flow(self):
        if self.cached_trades_data is not None:
            return np.sum(self.cached_trades_data[:, 1])  # Sum of trade volumes
        return None

    def calculate_trading_volumes(self):
        # Example: Sum of trade volumes over the last N trades
        if self.cached_trades_data is not None:
            return np.sum(self.cached_trades_data[:, 1][-10:])  # Last 10 trades
        return None

    # def calculate_weighted_price_volatility(self):
    #     # Volatility measure, weighted by volume
    #     if self.cached_trades_data is not None:
    #         price_changes = np.diff(self.cached_trades_data[:, 0])
    #         volumes = self.cached_trades_data[1:, 1]
    #         return np.sum(np.abs(price_changes) * volumes)
    #     return None

    def calculate_weighted_price_volatility_multi_snapshot(self, trades_data_snapshots):
        volatilities = []
        for i in range(1, len(trades_data_snapshots)):
            if isinstance(trades_data_snapshots[i], pd.Series) and isinstance(trades_data_snapshots[i - 1], pd.Series):
                price_change = trades_data_snapshots[i]['price'] - trades_data_snapshots[i - 1]['price']
                volume = trades_data_snapshots[i]['volume']
                volatility = np.abs(price_change) * volume
                volatilities.append(volatility)
            else:
                volatilities.append(None)
        return volatilities




    # def calculate_time_weighted_volume_variability(self):
    #     # Variability in volume over time
    #     if self.cached_trades_data is not None:
    #         average_volume = np.mean(self.cached_trades_data[:, 1])
    #         return np.sum(np.abs(self.cached_trades_data[:, 1] - average_volume))
    #     return None
    def calculate_time_weighted_volume_variability_multi_snapshot(self, trades_data_snapshots):
        variabilities = []
        for i in range(1, len(trades_data_snapshots)):
            if isinstance(trades_data_snapshots[i], pd.Series) and isinstance(trades_data_snapshots[i - 1], pd.Series):
                time_diff = trades_data_snapshots[i]['local_timestamp'] - trades_data_snapshots[i - 1]['local_timestamp']
                volume_change = trades_data_snapshots[i]['volume'] - trades_data_snapshots[i - 1]['volume']
                weighted_variability = np.abs(volume_change) / time_diff if time_diff != 0 else 0
                variabilities.append(weighted_variability)
            else:
                variabilities.append(None)
        return variabilities





    # def calculate_market_imbalance(self):
    #     if self.cached_order_book is not None:
    #         # Assuming order book format: [bid price, bid volume, ask price, ask volume]
    #         total_bid_volume = np.sum(self.cached_order_book[:, 1])  # Sum of all bid volumes
    #         total_ask_volume = np.sum(self.cached_order_book[:, 3])  # Sum of all ask volumes

    #         market_imbalance = total_bid_volume - total_ask_volume
    #         return market_imbalance
        # return 0  # Return zero if there's no order book data available
    def calculate_market_imbalance_for_recent_snapshot(self, order_book_snapshots):
        # Directly use the most recent snapshot, assuming it's always present and valid
        recent_order_book = order_book_snapshots[-1]
        bids, asks = recent_order_book

        total_bid_volume = sum(volume for _, volume in bids) if bids else 0
        total_ask_volume = sum(volume for _, volume in asks) if asks else 0

        print(f"Debug: Total Bid Volume for most recent snapshot: {total_bid_volume}")
        print(f"Debug: Total Ask Volume for most recent snapshot: {total_ask_volume}")

        imbalance = total_bid_volume - total_ask_volume
        ratio = total_bid_volume / total_ask_volume if total_ask_volume != 0 else float('inf')

        print(f"Debug: Imbalance for most recent snapshot: {imbalance}")
        print(f"Debug: Imbalance Ratio for most recent snapshot: {ratio}")

        # Return total bid and ask volume for the most recent snapshot
        return total_bid_volume, total_ask_volume






    def calculate_sophisticated_liquidity_multi_snapshot(self):
        lambda_params = {'alpha': 0.1, 'beta': 0.2, 'gamma': 0.15, 'delta': 0.25, 'epsilon': 0.2, 'zeta': 0.1}

        # Get the latest values from each cached metric, default to 0 if not available
        market_depth = self.cached_market_depth[-1] if self.cached_market_depth else 0
        bid_ask_spread = self.cached_bid_ask_spread[-1] if self.cached_bid_ask_spread else 0
        order_flow = self.cached_order_flow[-1] if self.cached_order_flow else 0
        market_imbalance = self.cached_market_imbalance[-1] if self.cached_market_imbalance else 0
        weighted_price_volatility = self.cached_weighted_price_volatility[-1] if self.cached_weighted_price_volatility else 0
        time_weighted_volume_variability = self.cached_time_weighted_volume_variability[-1] if self.cached_time_weighted_volume_variability else 0

        # Calculate sophisticated liquidity
        L = 1 / (lambda_params['alpha'] * market_depth +
                lambda_params['beta'] * bid_ask_spread +
                lambda_params['gamma'] * order_flow +
                lambda_params['delta'] * market_imbalance +
                lambda_params['epsilon'] * weighted_price_volatility +
                lambda_params['zeta'] * time_weighted_volume_variability)
        return L



    # def calculate_sophisticated_liquidity(self):
    #     if self.cached_order_book is not None:
    #         # Define lambda parameters for liquidity calculation
    #         lambda_params = {
    #             'alpha': 0.1,  # Weight for market depth
    #             'beta': 0.2,   # Weight for bid-ask spread
    #             'gamma': 0.15, # Weight for order flow
    #             'delta': 0.25, # Weight for market imbalance
    #             'epsilon': 0.2,# Weight for weighted price volatility
    #             'zeta': 0.1    # Weight for time-weighted volume variability
    #         }

    #         # Calculate individual components of the liquidity measure
    #         market_depth = self.calculate_market_depth()
    #         bid_ask_spread = self.calculate_bid_ask_spread()
    #         order_flow = self.calculate_order_flow()
    #         market_imbalance = self.calculate_market_imbalance()
    #         weighted_price_volatility = self.calculate_weighted_price_volatility()
    #         time_weighted_volume_variability = self.calculate_time_weighted_volume_variability()

    #         # Combine terms into the liquidity equation
    #         L = 1 / (lambda_params['alpha'] * market_depth +
    #                  lambda_params['beta'] * bid_ask_spread +
    #                  lambda_params['gamma'] * order_flow +
    #                  lambda_params['delta'] * market_imbalance +
    #                  lambda_params['epsilon'] * weighted_price_volatility +
    #                  lambda_params['zeta'] * time_weighted_volume_variability)

    #         return L
    #     return None



    # def calculate_order_flow_rate(self, trades_data, time_interval_seconds):
    #     if trades_data.size == 0:
    #         return 0

    #     sorted_trades = trades_data[trades_data[:, 2].argsort()]
    #     time_diffs = np.diff(sorted_trades[:, 2])
    #     volume_changes = np.diff(sorted_trades[:, 1])

    #     if np.sum(time_diffs) == 0:
    #         return 0

    #     return np.sum(volume_changes) / np.sum(time_diffs)

    def calculate_order_flow_rate_multi_snapshot(self, trades_data_snapshots, time_interval_seconds):
        buy_flow_rates = []
        sell_flow_rates = []

        for i in range(1, len(trades_data_snapshots)):
            if isinstance(trades_data_snapshots[i], pd.Series) and isinstance(trades_data_snapshots[i - 1], pd.Series):
                time_diff = trades_data_snapshots[i]['local_timestamp'] - trades_data_snapshots[i - 1]['local_timestamp']

                if trades_data_snapshots[i]['side'] == 1:  # Buy side
                    buy_volume_change = trades_data_snapshots[i]['volume'] - trades_data_snapshots[i - 1]['volume']
                    buy_flow_rate = buy_volume_change / time_diff if time_diff != 0 else 0
                    buy_flow_rates.append(buy_flow_rate)

                elif trades_data_snapshots[i]['side'] == -1:  # Sell side
                    sell_volume_change = trades_data_snapshots[i]['volume'] - trades_data_snapshots[i - 1]['volume']
                    sell_flow_rate = sell_volume_change / time_diff if time_diff != 0 else 0
                    sell_flow_rates.append(sell_flow_rate)

        # Append to the deques
        self.cached_buy_order_flow_rate.extend(buy_flow_rates)
        self.cached_sell_order_flow_rate.extend(sell_flow_rates)





    # def calculate_order_book_skewness(self):
    #     if self.cached_order_book is not None:
    #         # Assuming order book format: [bid price, bid volume, ask price, ask volume]
    #         bid_price_skewness = skew(self.cached_order_book[:, 0])
    #         ask_price_skewness = skew(self.cached_order_book[:, 2])
    #         return bid_price_skewness, ask_price_skewness
    #     return 0, 0  # Return zeros if there's no order book data

    # def calculate_order_book_slope(self):
    #     if self.cached_order_book is not None:
    #         # Calculate slopes for bid and ask sides
    #         bid_slope = self._calculate_slope(self.cached_order_book[:, 0], self.cached_order_book[:, 1])
    #         ask_slope = self._calculate_slope(self.cached_order_book[:, 2], self.cached_order_book[:, 3])
    #         return bid_slope, ask_slope
    #     return 0, 0  # Return zeros if there's no order book data


    def calculate_order_book_skewness_slope_multi_snapshot(self, order_book_snapshots):
        skewnesses, slopes = [], []

        for order_book in order_book_snapshots:
            if isinstance(order_book, tuple) and len(order_book) == 2:
                bids, asks = order_book
                bid_prices, bid_volumes = zip(*bids) if bids else ([], [])
                ask_prices, ask_volumes = zip(*asks) if asks else ([], [])

                bid_skewness = skew(bid_prices) if bid_prices else 0
                ask_skewness = skew(ask_prices) if ask_prices else 0
                bid_slope = self._calculate_slope(bid_prices, bid_volumes) if bid_prices else 0
                ask_slope = self._calculate_slope(ask_prices, ask_volumes) if ask_prices else 0

                skewnesses.append((bid_skewness, ask_skewness))
                slopes.append((bid_slope, ask_slope))
            else:
                skewnesses.append((0, 0))
                slopes.append((0, 0))

        print("Debugging calculate_order_book_skewness_slope_multi_snapshot method:")
        print(f"Skewnesses: {skewnesses}")
        print(f"Slopes: {slopes}")
        return skewnesses, slopes


    def _calculate_slope(self, prices, volumes):
        # Linear regression to calculate slope
        if len(prices) > 1:
            A = np.vstack([prices, np.ones(len(prices))]).T
            slope, _ = np.linalg.lstsq(A, volumes, rcond=None)[0]
            return slope
        return 0


    # def update_trade_metrics(self):
    #     if self.cached_trades_data is not None:
    #         self.last_traded_price = self.cached_trades_data[-1, 0]
    #         self.last_trade_volume = self.cached_trades_data[-1, 1]

    #         rolling_window_size = 10  # Last 10 trades
    #         self.rolling_trade_volume = np.sum(self.cached_trades_data[-rolling_window_size:, 1])

    #         current_time = time.time()
    #         self.last_trade_timestamp = self.cached_trades_data[-1, 2]
    #         self.time_since_last_trade = current_time - self.last_trade_timestamp

    #         self.order_flow_rate = self.calculate_order_flow_rate(self.cached_trades_data, time_interval_seconds=60)
    def calculate_order_flow_multi_snapshot(self, trades_data_snapshots):
        order_flows = []
        for trades_data in trades_data_snapshots:
            if isinstance(trades_data, pd.DataFrame) and not trades_data.empty:
                # Sum the 'volume' column to get the total order flow for this snapshot
                total_order_flow = trades_data['volume'].sum()
                order_flows.append(total_order_flow)
            else:
                # If there are no trades in this snapshot, the order flow is 0
                order_flows.append(0)
        return order_flows



    # def calculate_historical_returns_multi_snapshot(self, prices_snapshots):
    #     historical_returns = []
    #     for prices in prices_snapshots:
    #         if len(prices) > 1:
    #             returns = np.diff(prices) / prices[:-1]
    #             historical_returns.append(returns)
    #         else:
    #             historical_returns.append(None)
    #     return historical_returns

    # def calculate_var_multi_snapshot(self, historical_returns_snapshots, confidence_level=0.95):
    #     vars = []
    #     for returns in historical_returns_snapshots:
    #         if len(returns) > 0:
    #             sorted_returns = np.sort(returns)
    #             index = int((1 - confidence_level) * len(sorted_returns))
    #             var = sorted_returns[index]
    #             vars.append(var)
    #         else:
    #             vars.append(None)
    #     return vars


    # def update_cached_metrics(self):
    #     self.cached_trades_data = self.order_book_cache.latest_trades_data

    #     if self.is_trade_data_updated:
    #         # Update trade-related metrics
    #         self.cached_vwap = self.calculate_vwap()
    #         self.cached_price_impact = self.calculate_price_impact()
    #         self.cached_var = self.calculate_var(0.95)  # Example: 95% confidence level
    #         self.cached_order_flow_ratio = self.calculate_order_flow_ratio()
    #         self.update_trade_metrics()

    #         # Additional metrics calculations
    #         self.cached_order_flow = self.calculate_order_flow()
    #         self.cached_trading_volumes = self.calculate_trading_volumes()
    #         self.cached_weighted_price_volatility = self.calculate_weighted_price_volatility()
    #         self.cached_time_weighted_volume_variability = self.calculate_time_weighted_volume_variability()

    #         # Reset the flag
    #         self.is_trade_data_updated = False

    def update_trade_metrics(self, trades_data_snapshots):
        # Assuming each item in trades_data_snapshots is a pandas Series representing a trade
        if len(trades_data_snapshots) == 0:
            print("No trades in this batch")
            return 0

        # Process the trade data
        prices = []
        volumes = []
        for trade_data in trades_data_snapshots:
            trade_price = trade_data['price']  # Accessing price directly
            trade_volume = trade_data['volume']  # Accessing volume directly
            prices.append(trade_price)
            volumes.append(trade_volume)

        # Calculate metrics based on prices and volumes
        # ...

        # Example: Calculate metrics based on the last two trades
        if len(prices) >= 2:
            last_price = prices[-1]
            last_volume = volumes[-1]
            second_last_price = prices[-2]
            second_last_volume = volumes[-2]

            if second_last_price != 0 and second_last_volume != 0:
                self.difference_traded_price = ((last_price - second_last_price) / second_last_price) * 100
                self.difference_trade_volume = ((last_volume - second_last_volume) / second_last_volume) * 100
            else:
                self.difference_traded_price = None
                self.difference_trade_volume = None
        else:
            self.difference_traded_price = None
            self.difference_trade_volume = None

        # Calculate and append additional metrics
        # ...
        self.cached_vwap = self.calculate_vwap_multi_snapshot(trades_data_snapshots)
        # price_impacts = self.calculate_price_impact_multi_snapshot(trades_data_snapshots)
        # #order_flow_ratios = self.calculate_order_flow_ratio_multi_snapshot(trades_data_snapshots)
        # time_weighted_volume_variabilities = self.calculate_time_weighted_volume_variability_multi_snapshot(trades_data_snapshots)
        # weighted_price_volatilities = self.calculate_weighted_price_volatility_multi_snapshot(trades_data_snapshots)
        self.calculate_order_flow_rate_features(trades_data_snapshots, time_interval_seconds=60)
        # order_flows = self.calculate_order_flow_multi_snapshot(trades_data_snapshots)

        # # Append to the deques
        # self.cached_vwap.extend(vwaps)
        # self.cached_price_impact.extend(price_impacts)
        # #self.cached_order_flow_ratio.extend(order_flow_ratios)
        # self.cached_time_weighted_volume_variability.extend(time_weighted_volume_variabilities)
        # self.cached_weighted_price_volatility.extend(weighted_price_volatilities)
        # #self.cached_order_flow_rate.extend(order_flow_rates)
        # # self.cached_order_flow.extend(order_flows)

        # Calculate rolling trade volume for a window of 5 snapshots

        # rolling_trade_volumes = []

        # for i in range(len(trades_data_snapshots)):
        #     if i >= 4:  # Ensure there are at least 5 data points
        #         start_index = i - 4  # Fixed window size of 5 data points
        #         rolling_window = trades_data_snapshots[start_index:i + 1]

        #         # Calculate the total trade volume within the rolling window
        #         total_volume = sum(trades_data['volume'] for trades_data in rolling_window if not trades_data.empty)

        #         # Calculate rolling average
        #         rolling_average = total_volume / 5  # Fixed size of the rolling window
        #         rolling_trade_volumes.append(rolling_average)



        #         print(f"Debug: Window {start_index}-{i}, Rolling Average Volume: {rolling_average}")  # Debugging line

        # self.rolling_trade_volume.extend(rolling_trade_volumes)


        print(f"Difference traded price: {self.difference_traded_price}, Difference trade volume: {self.difference_trade_volume}")
        print("Finished updating trade metrics")
        return 1, last_price


    # def update_cached_order_book_metrics(self):
    #     self.cached_order_book = self.order_book_cache.latest_order_book_data

    #     if self.is_order_book_updated:
    #         # Update order book-related metrics
    #         bid_depth, ask_depth = self.calculate_book_depth()
    #         self.cached_bid_depth = bid_depth
    #         self.cached_ask_depth = ask_depth
    #         self.best_bid_price = self.cached_order_book[0, 0] if self.cached_order_book is not None else 0
    #         self.best_ask_price = self.cached_order_book[0, 2] if self.cached_order_book is not None else 0


    #         # Additional market metrics
    #         self.cached_market_imbalance = self.calculate_market_imbalance()
    #         self.cached_market_depth = self.calculate_market_depth()
    #         self.cached_bid_ask_spread = self.calculate_bid_ask_spread()

    #         # Reset the flag
    #         self.is_order_book_updated = False
    import sys
    def update_cached_order_book_metrics(self, order_book_snapshots):

        book_depth_data = self.calculate_most_recent_book_depth(order_book_snapshots)
        # Extracting additional data from book_depth_data
        total_depths = book_depth_data['total_depth']
        bid_depths = book_depth_data['bid_depth']
        ask_depths = book_depth_data['ask_depth']
        best_bid_volumes = book_depth_data['best_bid_volume']
        best_ask_volumes = book_depth_data['best_ask_volume']

        bid_ask_spread_data = self.calculate_most_recent_bid_ask_spread(order_book_snapshots)
        spreads = bid_ask_spread_data['spread']
        best_bids = bid_ask_spread_data['best_bid']
        best_asks = bid_ask_spread_data['best_ask']
        half_spreads = bid_ask_spread_data['half_spread']
        mid_prices = bid_ask_spread_data['mid_price']

        # Additional calculations
        total_bid_volume, total_ask_volume = self.calculate_market_imbalance_for_recent_snapshot(order_book_snapshots)

        top_two_bid_levels_volume, top_two_ask_levels_volume = self.calculate_top_two_volumes(order_book_snapshots)

        # Calculate skewness and slope for bids and asks

        # skewness_slope_output = self.calculate_order_book_skewness_slope_multi_snapshot(order_book_snapshots)

        # # Unpack the skewness and slope data
        # bid_ask_skewnesses, bid_ask_slopes = skewness_slope_output
        # bid_skewnesses, ask_skewnesses = zip(*bid_ask_skewnesses)
        # bid_slopes, ask_slopes = zip(*bid_ask_slopes)

        # # Append the bid and ask skewnesses and slopes to the deques
        # self.cached_bid_skewness.extend(bid_skewnesses)
        # self.cached_ask_skewness.extend(ask_skewnesses)
        # self.cached_bid_slope.extend(bid_slopes)
        # self.cached_ask_slope.extend(ask_slopes)

        # Append to the deques (assuming these deques are defined elsewhere in your class)
        self.cached_market_depth= total_depths
        self.cached_best_bid_volume = best_bid_volumes
        self.cached_best_ask_volume = best_ask_volumes

        self.cached_bid_depth = bid_depths
        self.cached_ask_depth = ask_depths
        self.cached_bid_ask_spread = spreads
        self.cached_total_bid_volume = total_bid_volume
        self.cached_total_ask_volume= total_ask_volume
        #self.cached_order_book_skewness.extend(book_skewnesses)
        #self.cached_order_book_slope.extend(book_slopes)
        self.cached_best_bid_price = best_bids
        self.cached_best_ask_price = best_asks
        self.cached_half_spread = half_spreads
        self.cached_mid_price = mid_prices

        self.cached_top_two_ask_levels_volume = top_two_ask_levels_volume
        self.cached_top_two_bid_levels_volume = top_two_bid_levels_volume


    def update_position(self, trade_volume, trade_price, is_buy):
        if is_buy:
            total_cost = self.average_buy_price * self.inventory_level
            total_cost += trade_volume * trade_price
            self.inventory_level += trade_volume
            self.average_buy_price = total_cost / self.inventory_level if self.inventory_level else 0
        else:
            # Update for sell transaction
            self.inventory_level -= trade_volume
            self.total_sell_revenue += trade_volume * trade_price
            self.average_sell_price = self.total_sell_revenue / (self.original_inventory_level - self.inventory_level) if self.inventory_level < self.original_inventory_level else 0



    def calculate_unrealized_pnl(self, current_market_price):
        market_value = self.inventory_level * current_market_price
        cost_basis = self.inventory_level * self.average_buy_price
        return market_value - cost_basis

    # def calculate_time_since_last_trade(self, trades_data):
    #     if trades_data.size > 0:
    #         # Assuming the timestamp is in the third column
    #         last_trade_timestamp = trades_data[-1, 2]
    #         current_time = time.time()
    #         time_since_last_trade = current_time - last_trade_timestamp
    #         return time_since_last_trade
    #     else:
    #         return None  # Or a suitable default value if there are no trades

    def calculate_time_between_last_two_trades(self, trades_data):
        if trades_data.shape[0] >= 2:
            # Assuming the timestamp is in the third column
            last_trade_timestamp = trades_data[-1, 2]
            second_last_trade_timestamp = trades_data[-2, 2]
            time_diff = last_trade_timestamp - second_last_trade_timestamp
            return time_diff
        else:
            return None  # Return None or a suitable default if there are fewer than two trades

    # def preprocess_feature(self, feature, index):
    #     if index in [8, 9, 10]:  # Price features
    #         return np.log(feature / self.previous_state[index])
    #     elif index in [0, 1, 2, 3, 4, 5, 6, 7, 11, 12, 13, 14, 15, 16, 17]:  # Volume and other features
    #         return (feature - self.state_mean[index]) / self.state_std[index]
    #     else:
    #         return feature  # Other features that don't need preprocessing
    # def preprocess_feature(self, feature):
    #     median = np.median(feature)
    #     mad = np.median([np.abs(val - median) for val in feature])

    #     # Avoid division by zero
    #     if mad != 0:
    #         modified_z_score = [0.6745 * (val - median) / mad for val in feature]
    #     else:
    #         modified_z_score = feature  # or handle as needed

    #     return modified_z_score
    # def preprocess_feature_with_robust_scaling(self, feature, feature_name):

    #     # Check and print if the feature is None
    #     if feature is None:
    #         print(f"Feature '{feature_name}' is None")
    #         return []


    #     # Calculate the median
    #     median = np.median(feature)

    #     # Calculate the interquartile range (IQR)
    #     Q1 = np.percentile(feature, 25)
    #     Q3 = np.percentile(feature, 75)
    #     IQR = Q3 - Q1

    #     # Avoid division by zero in IQR
    #     if IQR != 0:
    #         scaled_feature = [(val - median) / IQR for val in feature]
    #     else:
    #         scaled_feature = feature  # or handle as needed

    #     return scaled_feature
    def calculate_ema(self,data, period):
        alpha = 2 / (period + 1)
        ema = data[0]  # Initialize EMA with the first data point
        for value in data[1:]:
            ema = (value * alpha) + (ema * (1 - alpha))
        return ema

    def calculate_cv(self,data):
        mean = np.mean(data)
        std_dev = np.std(data)
        return std_dev / mean if mean != 0 else 0  # Using 0 instead of float('inf') for practicality

    def calculate_order_flow_rate_features(self, trades_data_snapshots, time_interval_seconds):
        buy_flow_rates = []
        sell_flow_rates = []

        if not trades_data_snapshots:
            print("Debug: trades_data_snapshots is empty")
            return

        print(f"Debug: Processing {len(trades_data_snapshots)} trades data snapshots")

        for i in range(1, len(trades_data_snapshots)):
            time_diff = (trades_data_snapshots[i]['local_timestamp'] - trades_data_snapshots[i - 1]['local_timestamp']) / 1000

            if time_diff == 0:
                print(f"Debug: Time difference is 0 at index {i}")

            if trades_data_snapshots[i]['side'] == 1:  # Buy side
                buy_volume_change = trades_data_snapshots[i]['volume'] - trades_data_snapshots[i - 1].get('volume', 0)
                if time_diff != 0:
                    buy_flow_rate = buy_volume_change / time_diff
                    buy_flow_rates.append(buy_flow_rate)
                    print(f"Debug: Added buy flow rate {buy_flow_rate} at index {i}")

            elif trades_data_snapshots[i]['side'] == -1:  # Sell side
                sell_volume_change = trades_data_snapshots[i]['volume'] - trades_data_snapshots[i - 1].get('volume', 0)
                if time_diff != 0:
                    sell_flow_rate = sell_volume_change / time_diff
                    sell_flow_rates.append(sell_flow_rate)
                    print(f"Debug: Added sell flow rate {sell_flow_rate} at index {i}")

        if not buy_flow_rates:
            print("Debug: buy_flow_rates list is empty")

        if not sell_flow_rates:
            print("Debug: sell_flow_rates list is empty")

        # Proceed with EMA and CV calculations
        self.cached_ema_buy = self.calculate_ema(buy_flow_rates, len(buy_flow_rates)) if buy_flow_rates else None
        self.cached_ema_sell = self.calculate_ema(sell_flow_rates, len(sell_flow_rates)) if sell_flow_rates else None
        self.cached_cv_buy = self.calculate_cv(buy_flow_rates) if buy_flow_rates else None
        self.cached_cv_sell = self.calculate_cv(sell_flow_rates) if sell_flow_rates else None

        print(f"Debug: Cached EMA/CV values calculated for buy and sell flow rates")









    def calculate_avg_entry_price_deviation_ratio(self, avg_entry_price, reference_price, nameoffeature):
        """
        Calculate the ratio of the average entry price deviation to a given reference price.

        :param avg_entry_price: The average entry price of positions.
        :param reference_price: The reference price for comparison.
        :return: Ratio of average entry price deviation to the reference price.
        """
        print(f"Name of Feature: {nameoffeature}")
        if avg_entry_price is None or  avg_entry_price == 0:
            print(f"Average entry price: {avg_entry_price}")
            print(f"Reference price: {reference_price}")
            return 0

        # Calculate the absolute deviation and deviation ratio
        print(f"Average entry price: {avg_entry_price}")
        print(f"Reference price: {reference_price}")
        absolute_deviation = abs(avg_entry_price - reference_price)
        deviation_ratio = absolute_deviation / reference_price if reference_price != 0 else 0
        return deviation_ratio




    def normalized_log_distance(self, x, y, feature_name, min_possible_distance=0, max_possible_distance=20):
        epsilon = 1e-10  # Small positive value to avoid log(0)
        print(f"feature_name: {feature_name}, x: {x}, y: {y}")

        # Ensure x and y are positive for logarithm; use epsilon as a substitute for non-positive values
        x_safe = max(x, epsilon)
        y_safe = max(y, epsilon)
        log_distance = abs(np.log(x_safe) - np.log(y_safe))
        print(f"log_distance: {log_distance}")

        # Normalize and ensure within [0, 1]
        normalized_distance = (log_distance - min_possible_distance) / (max_possible_distance - min_possible_distance)
        normalized_distance = max(0, min(1, normalized_distance))

        return normalized_distance


    def calculate_normalized_weighted_avg_price_spread(self, average_buy_price, average_sell_price, current_mid_price):
        """
        Calculate the normalized weighted spread between the average buy price and average sell price,
        expressed as a percentage of the current mid-price.
        """
        if current_mid_price != 0:
            normalized_buy_spread = (average_buy_price - current_mid_price) / current_mid_price
            normalized_sell_spread = (average_sell_price - current_mid_price) / current_mid_price

            # Calculate the normalized weighted spread as the average of buy and sell spreads
            weighted_spread = self.normalized_log_distance(normalized_buy_spread, normalized_sell_spread, 'normalized_weighted_avg_price_spread')

            # Calculate the weighted spread as a percentage of the current mid-price

        else:
            weighted_spread = float('inf')

        return weighted_spread

    def ema_calculate_aggregated_features(self, data):
        """
        Calculate aggregated statistical features for a given list of data points.
        Return all zeros if any nan values are detected after calculations.
        :param data: List of data points (e.g., price ticks, trade volumes)
        :return: Array of aggregated features with individual quantiles
        """
        data = np.array(data)  # Ensure data is a NumPy array for processing
        ema = self.calculate_ema_for_aggregation(data, .7)
        std_dev = np.std(data)
        cv = std_dev / ema if ema != 0 else 0
        skewness = skew(data, nan_policy='omit')
        kurt = kurtosis(data, nan_policy='omit')
        q25, q50, q75 = np.percentile(data, [25, 50, 75])
        min_value = np.min(data)
        max_value = np.max(data)
        min_max_ratio = min_value / max_value if max_value != 0 else 0

        aggregated_features = [
            ema,
            std_dev,
            cv,
            skewness,
            kurt,
            q25,
            q50,
            q75,
            min_max_ratio
        ]

        # Check if any value in aggregated_features is nan and return zeros if true
        if np.isnan(aggregated_features).any():
            return [0] * 9
        else:
            return aggregated_features


    def calculate_aggregated_features(self,data):
        """
        Calculate aggregated statistical features for a given list of data points.
        :param data: List of data points (e.g., price ticks, trade volumes)
        :return: Array of aggregated features with individual quantiles
        """
        if not data:
            return [0] * 9  # Return an array of None values for each feature

        mean = np.mean(data)
        std_dev = np.std(data)
        cv = std_dev / mean if mean != 0 else 0  # Coefficient of Variation
        skewness = skew(data)
        kurt = kurtosis(data)

        # Calculate individual quantiles
        q25, q50, q75 = np.percentile(data, [25, 50, 75])  # 25th, 50th, 75th percentile
        min_value = np.min(data)
        max_value = np.max(data)
        min_max_ratio = min_value / max_value if max_value != 0 else 0

        return [
            mean,
            std_dev,
            cv,
            skewness,
            kurt,
            q25,
            q50,
            q75,
            min_max_ratio
        ]

    def calculate_ema_for_aggregation(self, data, alpha):
        """Calculate the Exponential Moving Average (EMA) for a given list of data points."""
        ema = data[0]
        for value in data[1:]:
            ema = alpha * value + (1 - alpha) * ema
        return ema

    def preprocess_feature_with_robust_scaling(self, feature, feature_name):
        # Print the entire feature and its name
        print(f"Feature '{feature_name}': {feature}")

        # Check and print if the entire feature is None
        if feature is None:
            print(f"Feature '{feature_name}' is None")
            return []

        # Filter out None values from the feature list
        filtered_feature = [f for f in feature if f is not None]

        # Print the filtered feature
        print(f"Filtered '{feature_name}': {filtered_feature}")

        if len(filtered_feature) > 0:
            # Calculate the median of the filtered feature
            median = np.median(filtered_feature)

            # Calculate the interquartile range (IQR)
            q75, q25 = np.percentile(filtered_feature, [75, 25])
            iqr = q75 - q25

            # Scale features using robust scaling
            scaled_feature = [(f - median) / iqr if iqr != 0 else 0 for f in filtered_feature]
            print(f"Scaled '{feature_name}': {scaled_feature}")
            return scaled_feature
        else:
            # Return an empty list if all values were None or feature was empty
            return []

    def z_score_normalize(self, value, mean, std_dev):
        return (value - mean) / std_dev if std_dev != 0 else 0

    def extend_to_length(self, value, length):
        """Extend a scalar value to a list of a specified length."""
        return [value] * length

    def replace_nans_with_zeros(self, array):
        """
        Replace nan values in the array with zeros.
        """
        clean_array = np.nan_to_num(array)
        return clean_array

    def construct_state_vector(self, current_order_book, current_trades, current_time):
        # Update metrics based on the type of data
        print(f"Current Order_Book: {current_order_book}")
        print(f"Current Trades: {current_trades}")
        print(f"Current Time: {current_time}")
        self.update_cached_order_book_metrics(current_order_book)
        check, last_price = self.update_trade_metrics(current_trades)
        if check == 0:
            print("No trades in this batch")
            return 0, None
        else:

            current_position = self.get_current_position()

            if current_position == 0:
                inventory_side = 0
            elif current_position > 0:
                inventory_side = 1
            else :
                inventory_side = -1

            current_inventory = self.order_manager.inventory
            total_active_order_value = self.order_manager.calculate_combined_inventory_and_orders_value()
            # Calculate the mean and standard deviation of the price
            average_buy_price = self.get_average_buy_price()
            average_sell_price = self.get_average_sell_price()
            #order_flow_ratio = self.calculate_order_flow_ratio(current_trades)

            # unrealized_pnl = self.get_unrealized_pnl()
            rate_percentage = self.rate_limiter.get_usage_percentage()

            # one_minute_mad = self.volatility_manager.get_mad_volatility_measures().get("1m", 0)
            # seven_minute_mad = self.volatility_manager.get_mad_volatility_measures().get("7m", 0)
            # sixty_minute_mad = self.volatility_manager.get_mad_volatility_measures().get("60m", 0)
            # print(f"MADs: {one_minute_mad}, {seven_minute_mad}, {sixty_minute_mad}")
            # Example data retrieval (assuming you have methods to get these values)

            own_best_bid, own_worst_bid, own_best_bid_volume = self.order_manager.get_best_and_worst_bid_with_total_volume()
            if own_best_bid is None:
                own_best_bid = 0
                own_best_bid_volume = 0
            own_best_ask, own_worst_ask, own_best_ask_volume = self.order_manager.get_best_and_worst_ask_with_total_volume()
            if own_best_ask is None:
                own_best_ask = 0
                own_best_ask_volume = 0

            if own_worst_bid is None:
                own_worst_bid = 0
            if own_worst_ask is None:
                own_worst_ask = 0

            # Calculate the ratio


            # Calculate the ratio
            #
            state = []

            AUM_unit_size = self.initial_capital / last_price

            # top_two_ask_levels_volume = self.calculate_top_two_volumes()
            # top_two_bid_levels_volume = self.calculate_top_two_volumes()

            # Calculate the ratios

            current_best_bid = self.cached_best_bid_price
            current_best_ask = self.cached_best_ask_price
            print("Checking DEMAs and MADs:")

            # DEMAs Checks
            one_minute_dema = self.volatility_manager.get_one_minute_dema()
            print(f"Initial one_minute_dema: {one_minute_dema}")
            if not one_minute_dema:  # Checks if one_minute_dema is None or 0
                one_minute_dema = last_price
                print("one_minute_dema was None or 0, replaced with last_price:", last_price)

            seven_minute_dema = self.volatility_manager.get_seven_minute_dema()
            print(f"Initial seven_minute_dema: {seven_minute_dema}")
            if not seven_minute_dema:  # Checks if seven_minute_dema is None or 0
                seven_minute_dema = last_price
                print("seven_minute_dema was None or 0, replaced with last_price:", last_price)

            thirty_five_minute_dema = self.volatility_manager.get_thirty_five_minute_dema()
            print(f"Initial thirty_five_minute_dema: {thirty_five_minute_dema}")
            if not thirty_five_minute_dema:  # Checks if thirty_five_minute_dema is None or 0
                thirty_five_minute_dema = last_price
                print("thirty_five_minute_dema was None or 0, replaced with last_price:", last_price)

            trend_direction = 1 if seven_minute_dema > thirty_five_minute_dema else -1
            deviation =(seven_minute_dema/thirty_five_minute_dema - 1) * 100
            if inventory_side == trend_direction:
                # Example: scale reward by deviation, with a minimum deviation threshold
                min_deviation_threshold = 0.17  # Example threshold for considering the trend
                if deviation > min_deviation_threshold:
                    self.alignment_reward = abs(current_inventory) * last_price * deviation * 0.17  # Example scaling
                else:
                    self.alignment_reward = 0
            else:
                self.alignment_reward = 0

            # MADs Checks
            # Getting MAD volatilities
            mad_volatilities = self.volatility_manager.get_mad_volatility_measures()

            # Print statements for MADs
            print("Retrieved MAD Volatilities:")
            for time_frame, mad in mad_volatilities.items():
                print(f"{time_frame} MAD: {mad if mad is not None else 0}")

            my_best_bid_to_ask_volume_ratio = self.calculate_avg_entry_price_deviation_ratio(own_best_bid_volume, own_best_ask_volume, 'my_best_bid_to_ask_volume_ratio')
            my_best_bid_to_ask_volume_log_distance = self.normalized_log_distance(own_best_bid_volume, own_best_ask_volume, 'my_best_bid_to_ask_volume_log_distance')
            state.append(my_best_bid_to_ask_volume_log_distance)
            state.append(my_best_bid_to_ask_volume_ratio)

            my_best_bid_volume_to_market_best_volume = self.normalized_log_distance(own_best_bid_volume, self.cached_best_bid_volume, 'log_distance_my_best_bid_volume_to_market_best_volume')
            my_best_ask_volume_to_market_best_volume = self.normalized_log_distance(own_best_ask_volume, self.cached_best_ask_volume, 'log_distance_my_best_ask_volume_to_market_best_volume')
            state.append(my_best_bid_volume_to_market_best_volume)
            state.append(my_best_ask_volume_to_market_best_volume)

            my_worst_bid_to_market_best_bid_log_distance = self.normalized_log_distance(own_worst_bid, self.cached_best_bid_price, 'my_worst_bid_to_market_best_bid_log_distance')
            my_worst_ask_to_market_best_ask_log_distance = self.normalized_log_distance(own_worst_ask, self.cached_best_ask_price, 'my_worst_ask_to_market_best_ask_log_distance')
            state.append(my_worst_bid_to_market_best_bid_log_distance)
            state.append(my_worst_ask_to_market_best_ask_log_distance)
            my_worst_bid_to_my_best_bid_log_distance = self.normalized_log_distance(own_worst_bid, own_best_bid, 'my_worst_bid_to_my_best_bid_log_distance')
            my_worst_ask_to_my_best_ask_log_distance = self.normalized_log_distance(own_worst_ask, own_best_ask, 'my_worst_ask_to_my_best_ask_log_distance')
            state.append(my_worst_bid_to_my_best_bid_log_distance)
            state.append(my_worst_ask_to_my_best_ask_log_distance)
            my_best_bid_to_current_log_distance = self.normalized_log_distance(own_best_bid, current_best_bid, 'my_best_bid_to_current_log_distance')
            my_best_ask_to_current_log_distance = self.normalized_log_distance(own_best_ask, current_best_ask, 'my_best_ask_to_current_log_distance')
            state.append(my_best_bid_to_current_log_distance)
            state.append(my_best_ask_to_current_log_distance)

            #
            average_buy_to_best_bid_ratio = self.calculate_avg_entry_price_deviation_ratio(average_buy_price, current_best_bid, 'average_buy_to_best_bid_ratio')
            average_sell_to_best_ask_ratio = self.calculate_avg_entry_price_deviation_ratio(average_sell_price, current_best_ask, 'average_sell_to_best_ask_ratio')
            state.append(average_buy_to_best_bid_ratio)
            state.append(average_sell_to_best_ask_ratio)

            #
            average_buy_to_best_bid_log_distance = self.normalized_log_distance(average_buy_price, current_best_bid, 'average_buy_to_best_bid_log_distance')
            average_sell_to_best_ask_log_distance = self.normalized_log_distance(average_sell_price, current_best_ask, 'average_sell_to_best_ask_log_distance')
            state.append(average_buy_to_best_bid_log_distance)
            state.append(average_sell_to_best_ask_log_distance)

            #


            #
            average_sell_to_worst_ask_log_distance = self.normalized_log_distance(own_worst_ask, average_sell_price, 'average_sell_to_worst_ask_log_distance')
            average_buy_to_worst_bid_log_distance= self.normalized_log_distance(own_worst_bid, average_buy_price, 'average_buy_to_worst_bid_log_distance')
            state.append(average_sell_to_worst_ask_log_distance)
            state.append(average_buy_to_worst_bid_log_distance)


            weighted_avg_price_spread = self.calculate_normalized_weighted_avg_price_spread(average_buy_price, average_sell_price, self.cached_mid_price)
            active_orders_to_aum_ratio = self.calculate_avg_entry_price_deviation_ratio(total_active_order_value, self.initial_capital, 'active_orders_to_aum_ratio')
            state.append(weighted_avg_price_spread)
            state.append(active_orders_to_aum_ratio)
            active_orders_to_aum_log_distance = self.normalized_log_distance(total_active_order_value, self.initial_capital, 'active_orders_to_aum_log_distance')
            state.append(active_orders_to_aum_log_distance)
            #
            inventory_target = self.order_manager.get_target_inventory()

            inventory_target_ratio = self.calculate_avg_entry_price_deviation_ratio(current_inventory, inventory_target, 'inventory_target_ratio')
            inventory_target_log_distance = self.normalized_log_distance(current_inventory, inventory_target, 'inventory_target_log_distance')
            state.append(inventory_target_ratio)
            state.append(inventory_target_log_distance)

            #
            dema_1_7_ratio = self.calculate_avg_entry_price_deviation_ratio(one_minute_dema, seven_minute_dema, 'dema_1_7_ratio')
            dema_1_7_log_distance = self.normalized_log_distance(one_minute_dema, seven_minute_dema, 'dema_1_7_log_distance')
            state.append(dema_1_7_ratio)
            state.append(dema_1_7_log_distance)
            dema_1_35_ratio = self.calculate_avg_entry_price_deviation_ratio(one_minute_dema, thirty_five_minute_dema, 'dema_1_35_ratio')
            dema_1_35_log_distance = self.normalized_log_distance(one_minute_dema, thirty_five_minute_dema, 'dema_1_35_log_distance')
            state.append(dema_1_35_ratio)
            state.append(dema_1_35_log_distance)
            #
            #
            #

            #

            dema_7_35_ratio = self.calculate_avg_entry_price_deviation_ratio(seven_minute_dema, thirty_five_minute_dema, 'dema_7_35_ratio')
            dema_7_35_log_distance = self.normalized_log_distance(seven_minute_dema, thirty_five_minute_dema, 'dema_7_35_log_distance')
            state.append(dema_7_35_ratio)
            state.append(dema_7_35_log_distance)


            depth_ratio = self.calculate_avg_entry_price_deviation_ratio(self.cached_bid_depth, self.cached_ask_depth, 'depth_ratio')
            depth_log_distance = self.normalized_log_distance(self.cached_bid_depth, self.cached_ask_depth, 'depth_log_distance')
            state.append(depth_ratio)
            state.append(depth_log_distance)

            #


            total_volume_ratio = self.calculate_avg_entry_price_deviation_ratio(self.cached_total_ask_volume, self.cached_total_bid_volume, 'total_volume_ratio')
            total_volume_log_distance = self.normalized_log_distance(self.cached_total_ask_volume, self.cached_total_bid_volume, 'total_volume_log_distance')
            state.append(total_volume_ratio)
            state.append(total_volume_log_distance)

            #






            #


            own_best_bid_to_inventory_log_distance = self.normalized_log_distance(own_best_bid_volume, current_inventory, 'own_best_bid_to_inventory_log_distance')
            own_best_ask_to_inventory_log_distance = self.normalized_log_distance(own_best_ask_volume, current_inventory, 'own_best_ask_to_inventory_log_distance')
            state.append(own_best_bid_to_inventory_log_distance)
            state.append(own_best_ask_to_inventory_log_distance)
            own_best_ask_volume_to_total_active_order_value = self.normalized_log_distance(own_best_ask_volume, total_active_order_value, 'own_best_ask_volume_to_total_active_order_value')
            own_best_bid_volume_to_total_active_order_value = self.normalized_log_distance(own_best_bid_volume, total_active_order_value, 'own_best_bid_volume_to_total_active_order_value')
            state.append(own_best_ask_volume_to_total_active_order_value)
            state.append(own_best_bid_volume_to_total_active_order_value)




            top_two_ask_levels_volume_log_distance_AUM = self.normalized_log_distance(self.cached_top_two_ask_levels_volume, AUM_unit_size, 'top_two_ask_levels_volume_log_distance_AUM')
            top_two_bid_levels_volume_log_distance_AUM = self.normalized_log_distance(self.cached_top_two_bid_levels_volume, AUM_unit_size, 'top_two_bid_levels_volume_log_distance_AUM')
            top_two_ask_levels_volume_ratio_AUM  = self.calculate_avg_entry_price_deviation_ratio(self.cached_top_two_ask_levels_volume, AUM_unit_size, 'top_two_ask_levels_volume_ratio_AUM')
            top_two_bid_levels_volume_ratio_AUM = self.calculate_avg_entry_price_deviation_ratio(self.cached_top_two_bid_levels_volume, AUM_unit_size, 'top_two_bid_levels_volume_ratio_AUM')

            state.append(top_two_ask_levels_volume_log_distance_AUM)
            state.append(top_two_bid_levels_volume_log_distance_AUM)
            state.append(top_two_ask_levels_volume_ratio_AUM)
            state.append(top_two_bid_levels_volume_ratio_AUM)

            top_two_ask_levels_volume_log_distance_inventory = self.normalized_log_distance(self.cached_top_two_ask_levels_volume, current_inventory, 'top_two_ask_levels_volume_log_distance_inventory')
            top_two_bid_levels_volume_log_distance_inventory = self.normalized_log_distance(self.cached_top_two_bid_levels_volume, current_inventory, 'top_two_bid_levels_volume_log_distance_inventory')
            top_two_ask_levels_volume_ratio_inventory = self.calculate_avg_entry_price_deviation_ratio(self.cached_top_two_ask_levels_volume, current_inventory, 'top_two_ask_levels_volume_ratio_inventory')
            top_two_bid_levels_volume_ratio_inventory = self.calculate_avg_entry_price_deviation_ratio(self.cached_top_two_bid_levels_volume, current_inventory, 'top_two_bid_levels_volume_ratio_inventory')
            state.append(top_two_ask_levels_volume_log_distance_inventory)
            state.append(top_two_bid_levels_volume_log_distance_inventory)
            state.append(top_two_ask_levels_volume_ratio_inventory)
            state.append(top_two_bid_levels_volume_ratio_inventory)

            top_two_bid_levels_volume_log_distance_own_best_bid = self.normalized_log_distance(own_best_bid_volume, self.cached_top_two_bid_levels_volume, 'top_two_bid_levels_volume_log_distance_own_best_bid')
            top_two_ask_levels_volume_log_distance_own_best_ask = self.normalized_log_distance(own_best_ask_volume, self.cached_top_two_ask_levels_volume, 'top_two_ask_levels_volume_log_distance_own_best_ask')
            state.append(top_two_bid_levels_volume_log_distance_own_best_bid)
            state.append(top_two_ask_levels_volume_log_distance_own_best_ask)


            top_two_bid_levels_volume_log_distance_top_two_ask_levels_volume = self.normalized_log_distance(self.cached_top_two_bid_levels_volume, self.cached_top_two_ask_levels_volume, 'top_two_bid_levels_volume_log_distance_top_two_ask_levels_volume')
            top_two_bid_levels_volume_top_two_ask_levels_volume_ratio = self.calculate_avg_entry_price_deviation_ratio(self.cached_top_two_bid_levels_volume, self.cached_top_two_ask_levels_volume, 'top_two_bid_levels_volume_top_two_ask_levels_volume_ratio')
            state.append(top_two_bid_levels_volume_log_distance_top_two_ask_levels_volume)
            state.append(top_two_bid_levels_volume_top_two_ask_levels_volume_ratio)


            vwap_top_two_ask_ratio = self.calculate_avg_entry_price_deviation_ratio(self.cached_vwap , self.cached_top_two_ask_levels_volume, 'vwap_top_two_ask_ratio')
            vwap_top_two_ask_log_distance = self.normalized_log_distance(self.cached_vwap , self.cached_top_two_ask_levels_volume, 'vwap_top_two_ask_log_distance')
            vwap_top_two_bid_log_distance =self.normalized_log_distance(self.cached_vwap , self.cached_top_two_bid_levels_volume, 'vwap_top_two_bid_log_distance')
            vwap_top_two_bid_ratio = self.calculate_avg_entry_price_deviation_ratio(self.cached_vwap , self.cached_top_two_bid_levels_volume, 'vwap_top_two_bid_ratio')


            state.append(vwap_top_two_ask_ratio)
            state.append(vwap_top_two_ask_log_distance)
            state.append(vwap_top_two_bid_log_distance)
            state.append(vwap_top_two_bid_ratio)


            ratio_average_buy_to_vwap = self.calculate_avg_entry_price_deviation_ratio(average_buy_price, self.cached_vwap , 'ratio_average_buy_to_vwap')
            log_diff_buy_vwap = self.normalized_log_distance(average_buy_price, self.cached_vwap , 'log_diff_buy_vwap')
            ratio_average_sell_to_vwap = self.calculate_avg_entry_price_deviation_ratio(average_sell_price, self.cached_vwap , 'ratio_average_sell_to_vwap')
            log_diff_sell_vwap = self.normalized_log_distance(average_sell_price, self.cached_vwap , 'log_diff_sell_vwap')
            state.append(ratio_average_buy_to_vwap)
            state.append(log_diff_buy_vwap)
            state.append(ratio_average_sell_to_vwap)
            state.append(log_diff_sell_vwap)



            ratio_own_best_bid_to_vwap = self.calculate_avg_entry_price_deviation_ratio(own_best_bid, self.cached_vwap , 'ratio_own_best_bid_to_vwap')
            log_diff_own_best_bid_vwap = self.normalized_log_distance(own_best_bid, self.cached_vwap , 'log_diff_own_best_bid_vwap')
            state.append(ratio_own_best_bid_to_vwap)
            state.append(log_diff_own_best_bid_vwap)

            ratio_own_best_ask_to_vwap = self.calculate_avg_entry_price_deviation_ratio(own_best_ask, self.cached_vwap , 'ratio_own_best_ask_to_vwap')
            log_diff_own_best_ask_vwap = self.normalized_log_distance(own_best_ask, self.cached_vwap , 'log_diff_own_best_ask_vwap')
            state.append(ratio_own_best_ask_to_vwap)
            state.append(log_diff_own_best_ask_vwap)




            cv_buy_sell_ratio = self.calculate_avg_entry_price_deviation_ratio(self.cached_cv_buy, self.cached_cv_sell, 'cv_buy_sell_ratio')
            cv_buy_sell_log_distance = self.normalized_log_distance(self.cached_cv_buy, self.cached_cv_sell , 'cv_buy_sell_log_distance')
            state.append(cv_buy_sell_ratio)
            state.append(cv_buy_sell_log_distance)
            ema_buy_sell_ratio = self.calculate_avg_entry_price_deviation_ratio(self.cached_ema_buy, self.cached_ema_sell , 'ema_buy_sell_ratio')
            ema_buy_sell_log_distance = self.normalized_log_distance(self.cached_ema_buy, self.cached_ema_sell , 'ema_buy_sell_log_distance')
            state.append(ema_buy_sell_ratio)
            state.append(ema_buy_sell_log_distance)


            # one_seven_ratio= self.calculate_avg_entry_price_deviation_ratio(one_minute_mad, seven_minute_mad, 'one_seven_ratio')
            # one_sixty_ratio = self.calculate_avg_entry_price_deviation_ratio(one_minute_mad, sixty_minute_mad, 'one_sixty_ratio')
            # sixty_seven_ratio = self.calculate_avg_entry_price_deviation_ratio(sixty_minute_mad, seven_minute_mad, 'sixty_seven_ratio')
            # state.append(one_seven_ratio)
            # state.append(one_sixty_ratio)
            # state.append(sixty_seven_ratio)

            # one_seven_log_distance = self.normalized_log_distance(one_minute_mad, seven_minute_mad, 'one_seven_log_distance')
            # one_sixty_log_distance = self.normalized_log_distance(one_minute_mad, sixty_minute_mad, 'one_sixty_log_distance')
            # sixty_seven_log_distance = self.normalized_log_distance(sixty_minute_mad, seven_minute_mad, 'sixty_seven_log_distance')
            # state.append(one_seven_log_distance)
            # state.append(one_sixty_log_distance)
            # state.append(sixty_seven_log_distance)



            state.append(rate_percentage)


            state.append(inventory_side)





            state.append(self.cached_bid_ask_spread)

            # normalized_buy_price_z_score = self.z_score_normalize(average_buy_price, mean_price, std_dev_price)
            # normalized_sell_price_z_score = self.z_score_normalize(average_sell_price, mean_price, std_dev_price)
            # Specify the target length
            # target_length = 9

            # # Extend scalar values to the target length
            # extended_current_position = self.extend_to_length(inventory_side, target_length)

            # extend_active_orders_to_aum_ratio = self.extend_to_length(active_orders_to_aum_ratio, target_length)
            # extended_my_best_bid_to_current_ratio = self.extend_to_length(my_best_bid_to_current_ratio, target_length)
            # extended_my_best_ask_to_current_ratio = self.extend_to_length(my_best_ask_to_current_ratio, target_length)
            # extended_average_buy_to_best_bid_ratio = self.extend_to_length(average_buy_to_best_bid_ratio, target_length)
            # extended_average_sell_to_best_ask_ratio = self.extend_to_length(average_sell_to_best_ask_ratio, target_length)
            # extended_weighted_avg_price_spread = self.extend_to_length(weighted_avg_price_spread, target_length)
            # extended_inventory_target_ratio = self.extend_to_length(inventory_target_ratio, target_length)
            # extended_dema_ratio = self.extend_to_length(dema_ratio, target_length)

            # # extended_avg_buy_price = self.extend_to_length(self.get_average_buy_price(), target_length)
            # # extended_avg_sell_price = self.extend_to_length(self.get_average_sell_price(), target_length)

            # extended_order_flow_ratio = self.extend_to_length(self.calculate_trade_flow_ratio(current_trades), target_length)

            # extended_unrealized_pnl = self.extend_to_length(self.get_unrealized_pnl(), target_length)
            # extended_rate_percentage = self.extend_to_length(rate_percentage, target_length)
            # extended_liquidity = self.extend_to_length(self.liquidity, target_length)
            #extended_liquidity = self.extend_to_length(self.calculate_sophisticated_liquidity_multi_snapshot(), target_length)

            # Calculate mean and standard deviation for z-score normalization
            # mean_price = (extended_avg_buy_price[0] + extended_avg_sell_price[0]) / 2
            # std_dev_price = math.sqrt(((extended_avg_buy_price[0] - mean_price)**2 + (extended_avg_sell_price[0] - mean_price)**2) / 1)

            # # Constructing the state vector
            # state = [self.preprocess_feature_with_robust_scaling(feature) for feature in [
            #     self.cached_bid_depth, self.cached_ask_depth,
            #     self.cached_market_imbalance, self.cached_bid_ask_spread,
            #     self.cached_vwap, self.cached_price_impact, self.cached_order_book_skewness, self.cached_order_book_slope,
            #     self.cached_order_flow_ratio, self.cached_order_flow, self.cached_best_bid_price, self.cached_best_ask_price, self.cached_half_spread, self.cached_mid_price,
            #     self.rolling_trade_volume,
            #     self.cached_order_flow_rate,
            #     self.cached_trading_volumes, self.cached_weighted_price_volatility, self.cached_time_weighted_volume_variability,

            # ]]
            # feature_names = [ 'market_imbalance_ratio',  'vwap', 'price_impact',  'rolling_trade_volume',   'weighted_price_volatility', 'time_weighted_volume_variability'] #'bid_skewness',  'ask_skewness', 'bid_slope', 'ask_slope',

            # features = [ self.cached_imbalance_ratio,   self.cached_vwap, self.cached_price_impact,     self.rolling_trade_volume,  self.cached_weighted_price_volatility, self.cached_time_weighted_volume_variability]#self.cached_bid_skewness, self.cached_ask_skewness, self.cached_bid_slope, self.cached_ask_slope,

            # # Constructing the state vector
            # state = [self.preprocess_feature_with_robust_scaling(feature, name) for feature, name in zip(features, feature_names)]
            # one_minute_mad = volatility_manager.get_mad_volatility_measures().get("1m", 0)
            # seven_minute_mad = volatility_manager.get_mad_volatility_measures().get("7m", 0)
            # sixty_minute_mad = volatility_manager.get_mad_volatility_measures().get("60m", 0)

            # Names of the features for debugging or reference
            # feature_names2 = [
            #     'bid_depth',
            #     'ask_depth',
            #     'bid_ask_spread',
            #     'best_bid_price',
            #     'best_ask_price',
            #     'best_ask_volume',
            #     'best_bid_volume',
            #     'half_spread',
            #     'mid_price'
            # ]
            # # Features to normalize
            # features_to_normalize = [
            #     self.cached_bid_depth,
            #     self.cached_ask_depth,
            #     self.cached_bid_ask_spread,
            #     self.cached_best_bid_price,
            #     self.cached_best_ask_price,
            #     self.cached_best_ask_volume,
            #     self.cached_best_bid_volume,
            #     self.cached_half_spread,
            #     self.cached_mid_price
            # ]

            # for feature, name in zip(features_to_normalize, feature_names2):
            #     # Convert the queue to a list for calculation
            #     feature_list = list(feature)
            #     mean = np.mean(feature_list)
            #     std_dev = np.std(feature_list)

            #     normalized_feature = [self.z_score_normalize(value, mean, std_dev) for value in feature_list]
            #     print(f'{name}: {normalized_feature}')

            #     # Append the normalized feature to the state vector
            #     state.append(normalized_feature)

            # Integrate MAD volatilities into the state vector
            # Integrate MAD volatilities into the state vector using aggregated features
            # mad_values_tuple = tuple(mad_volatility for mad_volatility in mad_volatilities.values())

            # # for key, mad_volatility in mad_volatilities.items():
            #     # Ensure to handle None values or defaults if mad_volatility is not calculated
            #     print(f'mad_volatility: {mad_volatility}')

            #     if mad_volatility is not None:
            #         # Calculate aggregated features for the mad_volatility data
            #         aggregated_features = self.calculate_aggregated_features(mad_volatility)

            #         # Append the aggregated features to the state vector
            #         state.append(aggregated_features)
            #     else:
            #         # If MAD volatility is None, append default values (zeros) for each feature
            #         state.append([0.0] * 9)  # 9 features: mean, std_dev, cv, skewness, kurtosis, q25, q50, q75, min_max_ratio

            # cached_buy_order_flow_rate = self.preprocess_feature_with_robust_scaling(self.cached_buy_order_flow_rate, 'buy_order_flow_rate')
            # cached_sell_order_flow_rate = self.preprocess_feature_with_robust_scaling(self.cached_sell_order_flow_rate, 'sell_order_flow_rate')

            # # Assuming 'state' is a list to which you're appending features
            # ageregated_buy_order_flow = self.ema_calculate_aggregated_features(cached_buy_order_flow_rate)
            # ageregated_sell_order_flow = self.ema_calculate_aggregated_features(cached_sell_order_flow_rate)

            # print(f"ageregated_buy_order_flow: {ageregated_buy_order_flow}")
            # #
            # state.append(ageregated_buy_order_flow)
            # print(f"ageregated_sell_order_flow: {ageregated_sell_order_flow}")
            # state.append(ageregated_sell_order_flow)
            # # print(f"difference_traded_volume: {self.difference_traded_volume}")
            # # state.append(self.difference_traded_volume)

            # print(f"extended_order_flow_ratio: {extended_order_flow_ratio}")
            # state.append(extended_order_flow_ratio)

            # # print(f"difference_traded_price: {self.difference_traded_price}")
            # # state.append(self.difference_traded_price)

            # print(f"extended_rate_percentage: {extended_rate_percentage}")
            # state.append(extended_rate_percentage)

            # print(f"extended_unrealized_pnl: {extended_unrealized_pnl}")
            # state.append(extended_unrealized_pnl)

            # print(f"extended_current_position: {extended_current_position}")
            # state.append(extended_current_position)

            # print(f"extended_my_best_bid_to_current_ratio: {extended_my_best_bid_to_current_ratio}")
            # state.append(extended_my_best_bid_to_current_ratio)

            # print(f"extended_my_best_ask_to_current_ratio: {extended_my_best_ask_to_current_ratio}")
            # state.append(extended_my_best_ask_to_current_ratio)

            # print(f"extended_average_buy_to_best_bid_ratio: {extended_average_buy_to_best_bid_ratio}")
            # state.append(extended_average_buy_to_best_bid_ratio)

            # print(f"extended_average_sell_to_best_ask_ratio: {extended_average_sell_to_best_ask_ratio}")
            # state.append(extended_average_sell_to_best_ask_ratio)

            # print(f"extended_weighted_avg_price_spread: {extended_weighted_avg_price_spread}")
            # state.append(extended_weighted_avg_price_spread)

            # print(f"extended_inventory_target_ratio: {extended_inventory_target_ratio}")
            # state.append(extended_inventory_target_ratio)

            # # print(f"extended_inventory_target_ratio: {extended_inventory_target_ratio}")
            # # state.append(extended_inventory_target_ratio)
            # print(f"extended_active_orders_ratio: {extend_active_orders_to_aum_ratio}")
            # state.append(extend_active_orders_to_aum_ratio)


            # print(f"extended_dema_ratio: {extended_dema_ratio}")
            # state.append(extended_dema_ratio)

            # # This line seems to be a duplicate of the inventory target ratio.
            # # If this is intentional, append it again, otherwise, you may want to remove or replace this line.



            # Convert the state list to a tensor
            print("State vector:", state)
            state_array = np.array(state)
            print(f"State size: {len(state)}")


            # Convert the flattened
            state_tensor = torch.tensor(state_array, dtype=torch.float32)
            actor_state = state_tensor.view(-1)  # Flatten to 1D
            flattened_state_with_batch = actor_state.unsqueeze(0)

            print("State tensor:", flattened_state_with_batch.shape)


            return 1, flattened_state_with_batch






















In [ ]:
import pandas as pd
import numpy as np
import heapq
import queue
import random
import math
import glob
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors

import uuid

TIME_FACTOR_CONSTANT = 2
class OrderBook:
    def __init__(self, depth=10):
        self.depth = depth
        self.bids = []  # List of tuples (price, volume)
        self.asks = []  # List of tuples (price, volume)
        print("OrderBook initialized with depth:", self.depth)

        self.active_orders = {}  # Active orders placed by the strategy
        self.order_insertion_times = {}  # Insertion times of orders
        # self.queue_position_estimate = {}  # Estimated position in the queue
        self.my_trades = []  # Store trades executed by the strategy


    def update_book(self, bids, asks):
        """ Update the order book with new bid and ask data. """
        # Sort and trim the bids and asks to maintain the depth
        print("Updating order book with new bids and asks")
        self.bids = sorted(bids, key=lambda x: x[0], reverse=True)[:self.depth]
        self.asks = sorted(asks, key=lambda x: x[0])[:self.depth]

    def display_book(self):
        """ Display the current state of the order book. """
        print("Current state of the Order Book:")
        print("Bids:")
        for price, volume in self.bids:
            print(f"Price: {price}, Volume: {volume}")
        print("\nAsks:")
        for price, volume in self.asks:
            print(f"Price: {price}, Volume: {volume}")

    # def place_limit_order(self, order_id, price, volume, side, current_time):
    #     """ Place a limit order in the order book. """
    #     self.active_orders[order_id] = {'price': price, 'volume': volume, 'side': side}
    #     self.order_insertion_times[order_id] = current_time
    #     self.queue_position_estimate[order_id] = self.get_initial_queue_position(price, volume, side)

    #     # Check for potential matches before adding the order to the book
    #     remaining_volume, matched_orders = self.match_order(price, volume, side)

    #     # If there's remaining volume after matching, add it to the order book
    #     if remaining_volume > 0:
    #         if side == 'buy':
    #             self.bids.append((price, remaining_volume))
    #         else:
    #             self.asks.append((price, remaining_volume))
    #         self.bids.sort(key=lambda x: x[0], reverse=True)
    #         self.asks.sort(key=lambda x: x[0])

    #     self.trim_order_book()
    def place_limit_order(self, order_id, price, volume, side, current_time):
        """ Place a limit order in the order book. """
        print(f"Placing limit order: Order ID {order_id}, Price {price}, Volume {volume}, Side {side}")

        # Check for potential immediate matches
        remaining_volume, matched_trades = self.match_order(price, volume, side)

        # Add remaining volume to the order book
        if remaining_volume > 0:
            self.add_order_to_book(order_id, price, remaining_volume, side, current_time)

        # Process matched trades
        for trade  in matched_trades:
            self.execute_my_trade(trade, order_id, side)

        self.trim_order_book()

    def execute_my_trade(self, trade, order_id, matched_side):
        """ Execute a trade for the strategy and store it. """
        matched_price, matched_volume = trade
        matched_price = matched_price[0]
        print(f"Executing matched trade: Order ID {order_id}, Price {matched_price}, Volume {matched_volume}, Side {matched_side}")

        # Logic to process the matched trade
        # For example, updating positions, PnL calculations, etc.
        # self.position_manager.update_position(...)  # Example

        # Record the trade
        recorded_trade = {
            'order_id': order_id,
            'price': matched_price,
            'volume': matched_volume,
            'side': matched_side
        }
        self.my_trades.append(recorded_trade)

    def get_my_recent_trades(self):
        """ Get the list of strategy's trades executed since the last update. """
        recent_trades_copy = self.my_trades.copy()
        self.my_trades.clear()  # Clear the list after copying
        return recent_trades_copy

    def add_order_to_book(self, order_id, price, volume, side, current_time):
        print(f"Adding order to book: Order ID {order_id}, Price {price}, Volume {volume}, Side {side}, Time {current_time}")
        self.order_insertion_times[order_id] = current_time
        five_minutes_in_microseconds = 5 * 60 * 1_000_000  # 5 minutes in microseconds
        expiration_time = current_time + five_minutes_in_microseconds
        self.active_orders[order_id] = {'price': price, 'volume': volume, 'side': side, 'expiration': expiration_time}

        order_book = self.bids if side == 'buy' else self.asks

        # Find existing price level and update volume or append new price-volume pair
        found = False
        for i, (p, v) in enumerate(order_book):
            if p == price:
                order_book[i] = (price, v + volume)  # Update total volume at this price level
                found = True
                break

        if not found:
            order_book.append((price, volume))  # Append new price level
            order_book.sort(key=lambda x: x[0], reverse=(side == 'buy'))

        # Truncate the order book to maintain the specified depth
        if len(order_book) > self.depth:
            order_book = order_book[:self.depth]

    # def add_order_to_book(self, order_id, price, volume, side, current_time):
    #     """ Add or update an order in the order book and maintain sorting. """
    #     print(f"Adding order to book: Order ID {order_id}, Price {price}, Volume {volume}, Side {side}, Time {current_time}")
    #     expiration_time = current_time + 360000  # Add 5 minutes in milliseconds
    #     self.active_orders[order_id] = {'price': price, 'volume': volume, 'side': side, 'expiration': expiration_time}
    #     self.order_insertion_times[order_id] = current_time
    #     self.queue_position_estimate[order_id] = self.get_initial_queue_position(price, volume, side)

    #     order_book = self.bids if side == 'buy' else self.asks

    #     # Find existing price and update volume or append new price-volume pair
    #     for i, (p, v) in enumerate(order_book):
    #         if p == price:
    #             order_book[i] = (price, v + volume)
    #             break
    #     else:
    #         order_book.append((price, volume))
    #         order_book.sort(key=lambda x: x[0], reverse=(side == 'buy'))


    # def match_order(self, price, volume, side):
    #     """Attempt to match the incoming order with existing orders in the book."""
    #     book_side = self.asks if side == 'buy' else self.bids
    #     remaining_volume = volume
    #     matched_orders = []

    #     for order in book_side[:]:
    #         if (side == 'buy' and order[0] <= price) or (side == 'sell' and order[0] >= price):
    #             matched_volume = min(order[1], remaining_volume)
    #             remaining_volume -= matched_volume
    #             new_order_volume = order[1] - matched_volume

    #             if new_order_volume > 0:
    #                 order_index = book_side.index(order)
    #                 book_side[order_index] = (order[0], new_order_volume)
    #             else:
    #                 book_side.remove(order)

    #             matched_orders.append((order, matched_volume))
    #             if remaining_volume <= 0:
    #                 break


    #     # Adjust queue positions at the end of matching process
    #     for matched_order, matched_vol in matched_orders:
    #         matched_order_price, matched_order_side = matched_order
    #         self.adjust_queue_positions(matched_order, matched_vol)

    #     return remaining_volume, matched_orders
    def match_order(self, price, volume, side):
        """Attempt to match the incoming order with existing orders in the book."""
        book_side = self.asks if side == 'buy' else self.bids
        remaining_volume = volume
        matched_orders = []

        for order in book_side[:]:
            if (side == 'buy' and order[0] <= price) or (side == 'sell' and order[0] >= price):
                matched_volume = min(order[1], remaining_volume)
                remaining_volume -= matched_volume
                new_order_volume = order[1] - matched_volume

                if new_order_volume > 0:
                    order_index = book_side.index(order)
                    book_side[order_index] = (order[0], new_order_volume)
                else:
                    book_side.remove(order)

                matched_orders.append((order, matched_volume))
                if remaining_volume <= 0:
                    break

        return remaining_volume, matched_orders


    def find_order_id_by_price(self, price, trade_side):
    # Determine the opposite side for matching
      opposite_side = 'sell' if trade_side == 'buy' else 'buy'

      for order_id, order_details in self.active_orders.items():
          if order_details['price'] == price and order_details['side'] == opposite_side:
              return order_id
      return None  # Return None if no matching order is found

    # def calculate_execution_probability(self, order_id):
    #     """Calculate the probability of an order's execution based solely on its queue position."""
    #     print(f"Calculating execution probability for Order ID {order_id}")

    #     position_in_queue = self.queue_position_estimate.get(order_id, 0)

    #     # The simplest model might assume a linear decrease in probability
    #     # the further back in the queue an order is.
    #     # For example, the first order in the queue might have a near-100% probability of execution,
    #     # while the last order in a queue of depth 20 might have only a 5% chance.
    #     if self.depth > 0:
    #         probability = max(1 - (position_in_queue / self.depth), 0)
    #     else:
    #         probability = 0  # If the depth is 0 or undefined, set probability to 0
    #     return probability



    # def get_initial_queue_position(self, price, volume, side):
    #     """ Estimate the initial position of the order in the queue. """
    #     current_volume = sum(vol for p, vol in (self.bids if side == 'buy' else self.asks) if p == price)
    #     return current_volume
    def handle_trade_execution(self, trade_execution_event):
        """ Handle trade execution events for market orders. """
        trade_side = trade_execution_event['side']
        trade_volume = trade_execution_event['volume']
        print('======================================================================')
        print(f"Handling trade execution event: side: {trade_side}, volume: {trade_volume}")
        print(f"Current book: {self.bids}, {self.asks}")
        print('======================================================================')
        # Choose the correct side of the order book for matching

        # Choose the correct side of the order book for matching
        book_side = self.asks if trade_side == 'buy' else self.bids
        remaining_volume = trade_volume
        matched_orders = []

        for order in book_side:
            if remaining_volume <= 0:
                break

            # Determine the opposite side for finding the matching order
            opposite_side = 'sell' if trade_side == 'buy' else 'buy'
            order_id = self.find_order_id_by_price(order[0], opposite_side)

            if order_id is not None and self.check_order_execution(order_id):
                matched_volume = min(order[1], remaining_volume)
                remaining_volume -= matched_volume

                # Update the order volume or remove the order if fully executed
                new_order_volume = order[1] - matched_volume
                if new_order_volume > 0:
                    order_index = book_side.index(order)
                    book_side[order_index] = (order[0], new_order_volume)
                else:
                    book_side.remove(order)

                # Execute the trade for each matched order
                self.execute_my_trade((order[0], matched_volume), order_id, trade_side)




        # Note: The method to update orders and adjust queue positions needs to be adjusted or removed
        # as per the new implementation approach.
        # self.update_order_and_queue() and self.trim_order_book() may need revision or removal.

    # def handle_trade_execution(self, trade_execution_event):
    #     """ Handle trade execution events. """


    #     trade_side = trade_execution_event['side']
    #     trade_price = trade_execution_event['price']
    #     trade_volume = trade_execution_event['volume']
    #     print('======================================================================')
    #     print(f"Handling trade execution event: side: {trade_side}, price: {trade_price}, volume: {trade_volume}")
    #     print(f"Current book: {self.bids}, {self.asks}")
    #     print('======================================================================')
    #     book_side = self.asks if trade_side == 'buy' else self.bids
    #     remaining_volume = trade_volume
    #     matched_orders = []

    #     for order in book_side[:]:
    #         # Check if order price is compatible
    #         if (trade_side == 'buy' and order[0] <= trade_price) or (trade_side == 'sell' and order[0] >= trade_price):
    #             order_id = self.find_order_id_by_price(order[0], trade_side)
    #             execution_probability = self.calculate_execution_probability(order_id)

    #             # Only match if execution probability is above a certain threshold
    #             if execution_probability >= 0.5:  # Example threshold
    #                 print(f"Matched order {order} with execution probability {execution_probability}")
    #                 matched_volume = min(order[1], remaining_volume)
    #                 remaining_volume -= matched_volume
    #                 new_order_volume = order[1] - matched_volume

    #                 # Update the book and queue if partially or fully matched
    #                 if new_order_volume > 0:
    #                     order_index = book_side.index(order)
    #                     book_side[order_index] = (order[0], new_order_volume)
    #                 else:
    #                     book_side.remove(order)

    #                 matched_orders.append((order, matched_volume))
    #                 # If there's still remaining volume, find the next price level
    #                 if remaining_volume > 0:
    #                     next_price = self.find_next_price(trade_price, trade_side)
    #                     if next_price != trade_price:
    #                         trade_price = next_price
    #                     else:
    #                         # If no next price is found or it's the same as the current price, exit the loop
    #                         break

    #     # Update active orders and adjust queue positions based on matched orders
    #     for matched_order, matched_vol in matched_orders:
    #         matched_order_price, matched_order_side = matched_order
    #         self.update_order_and_queue(( matched_order_price, matched_order_side), matched_vol)

    #     self.trim_order_book()

    # def adjust_queue_positions(self, matched_order_info, matched_volume):
    #     """Adjust queue positions after a trade."""
    #     matched_order_price, matched_order_side = matched_order_info

    #     print(f"Adjusting queue positions: Price {matched_order_price}, Side {matched_order_side}, Matched Volume {matched_volume}")

    #     # Iterate over all orders in the queue
    #     for order_id, order_details in self.active_orders.items():
    #         # Check if the order is at the same price level and side as the matched order
    #         if order_details['price'] == matched_order_price and order_details['side'] == matched_order_side:
    #             # Adjust the estimated queue position for each affected order
    #             self.queue_position_estimate[order_id] = max(self.queue_position_estimate[order_id] - matched_volume, 0)

    # def update_order_and_queue(self, matched_order_info, matched_volume):
    #     """ Update the active orders and queue positions after a trade. """
    #     print(f"Updating order and queue: Matched Order Info {matched_order_info}, Matched Volume {matched_volume}")

    #     matched_order_price, matched_order_side = matched_order_info

    #     # Find the order ID by price and side
    #     order_id = self.find_order_id_by_price(matched_order_price, matched_order_side)
    #     if order_id is None:
    #         print(f"No matching order found for price {matched_order_price} and side {matched_order_side}")
    #         return

    #     # Update the volume in the active order or remove if fully matched
    #     if order_id in self.active_orders:
    #         order = self.active_orders[order_id]
    #         new_volume = order['volume'] - matched_volume
    #         if new_volume > 0:
    #             order['volume'] = new_volume
    #         else:
    #             # Remove the order if fully matched
    #             del self.active_orders[order_id]
    #             del self.order_insertion_times[order_id]
    #             del self.queue_position_estimate[order_id]

    #     # Adjust the queue positions for remaining orders
    #     for other_order_id, position in self.queue_position_estimate.items():
    #         other_order = self.active_orders.get(other_order_id)
    #         if other_order and other_order['price'] == matched_order_price and other_order['side'] == matched_order_side:
    #             if position > self.queue_position_estimate.get(order_id, 0):
    #                 self.queue_position_estimate[other_order_id] -= matched_volume

    # def revise_queue_positions(self, market_update_event):
    #     """ Revise the queue position estimates based on market update. """
    #     print("Revising queue positions based on market update event")

    #     for order_id in self.queue_position_estimate:
    #         current_position = self.queue_position_estimate[order_id]
    #         self.queue_position_estimate[order_id] = self.revise_queue_position_estimate(order_id, current_position, market_update_event)

    # def revise_queue_position_estimate(self, order_id, current_position, market_update):
    #     """ Revise the queue position estimate based on a single market update. """
    #     print("Revising queue position estimate based on market update")

    #     updated_position = current_position
    #     order_details = self.active_orders[order_id]

    #     # Check if the update is relevant to the order's price level
    #     if market_update['price'] == order_details['price']:
    #         front = self.get_volume_in_front_of_order(order_id)
    #         back = self.get_volume_behind_order(order_id)
    #         probability_of_impact = self.estimate_probability_of_impact(front, back)
    #         position_change = self.calculate_position_change(market_update, probability_of_impact)
    #         updated_position = max(updated_position + position_change, 0)

    #     return updated_position

    #     #return updated_position

    # def get_volume_in_front_of_order(self, order_id):
    #     """ Get the volume of orders in front of the specified order. """
    #     order_details = self.active_orders[order_id]
    #     front_volume = 0
    #     insertion_time = self.order_insertion_times[order_id]

    #     for other_order_id, other_order_details in self.active_orders.items():
    #         if other_order_details['price'] == order_details['price'] and \
    #            self.order_insertion_times[other_order_id] < insertion_time:
    #             front_volume += other_order_details['volume']

    #     return front_volume

    # def get_volume_behind_order(self, order_id):
    #     """ Get the volume of orders behind the specified order. """
    #     order_details = self.active_orders[order_id]
    #     back_volume = 0
    #     insertion_time = self.order_insertion_times[order_id]

    #     for other_order_id, other_order_details in self.active_orders.items():
    #         if other_order_details['price'] == order_details['price'] and \
    #            self.order_insertion_times[other_order_id] > insertion_time:
    #             back_volume += other_order_details['volume']

    #     return back_volume

    # def estimate_probability_of_impact(self, front, back):
    #     """ Estimate the probability that an update affects the queue position. """
    #     if front == 0 and back == 0:
    #         # If both front and back volumes are zero, return a default low probability
    #         return 0.9  # or some other default value of your choice

    #     f = lambda x: math.log(1 + x)  # Example: logarithmic function
    #     return f(back) / (f(front) + f(back))


    # def calculate_position_change(self, update, probability_of_impact):
    #     """ Calculate the change in position based on update and probability. """
    #     # Assuming 'size_change' is a column in your DataFrame
    #     if 'size_change' in update and update['size_change'] < 0:
    #         return update['size_change'] * probability_of_impact
    #     return 0

    # def get_order_execution_probability(self, order_id):
    #     """ Calculate the probability of an order's execution based on its volume percentage at the price level. """
    #     order_details = self.active_orders[order_id]
    #     book_side = self.bids if order_details['side'] == 'buy' else self.asks

    #     total_volume_at_price = sum(v for p, v in book_side if p == order_details['price'])
    #     if total_volume_at_price == 0:
    #         return 0

    #     volume_percentage = order_details['volume'] / total_volume_at_price
    #     return volume_percentage

    def get_order_execution_probability(self, order_id, current_time):
        """ Calculate the probability of an order's execution based on volume percentage and the age of the order for Level 2 data. """
        order_details = self.active_orders[order_id]
        book_side = self.bids if order_details['side'] == 'buy' else self.asks

        total_volume_at_price = sum(v for p, v in book_side if p == order_details['price'])
        if total_volume_at_price == 0:
            return 0

        volume_percentage = order_details['volume'] / total_volume_at_price

        # Time factor based on order age

        order_age = current_time - self.order_insertion_times[order_id]
        time_factor = 1 + math.log(1 + order_age / TIME_FACTOR_CONSTANT)  # TIME_FACTOR_CONSTANT is a predefined constant

        # Adjust probability based on volume percentage and time factor
        adjusted_probability = volume_percentage * time_factor

        return adjusted_probability

    def check_order_execution(self, order_id, current_time):
        """ Randomly determine if an order is executed based on its execution probability. """
        order_details = self.active_orders[order_id]
        book_side = self.bids if order_details['side'] == 'buy' else self.asks

        # Check if your order is the only order at that price level
        total_volume_at_price = sum(v for p, v in book_side if p == order_details['price'])
        if total_volume_at_price == order_details['volume']:
            # If only your volume is present, execute the order
            return True
        probability = self.get_order_execution_probability(order_id, current_time)
        return random.random() < probability

    def process_market_data(self, market_data):
        """ Simulate the arrival of new market data and update the order book. """
        print(f"Simulating market data: {market_data}")
        ## Check for and execute any trades
        updated_market_data = self.check_for_trade_executions(market_data)

        # Update order book with the updated market data
        if updated_market_data['side'] == 1.0:  # Bids
            self.update_order_book_side(self.bids, updated_market_data, descending=True)
        elif updated_market_data['side'] == -1.0:  # Asks
            self.update_order_book_side(self.asks, updated_market_data, descending=False)


        # Trim the order book and revise queue positions after updates
        self.trim_order_book()
        #self.revise_queue_positions(updated_market_data)

        print("Finished simulating market data update")

    def check_for_trade_executions(self, update):
        print("Checking for trade executions based on market update")
        trade_side = 'sell' if update['side'] == -1 else 'buy'  # Assuming '-1' represents sell in your system
        remaining_volume = update['volume']
        trade_price = update['price']
        print(f"Market Update - Side: {trade_side}, Price: {trade_price}, Volume: {remaining_volume}")

        # Choose the opposite book side to match trades
        book_side = self.bids if trade_side == 'sell' else self.asks

        for i, order in enumerate(book_side):
            #print(f"Checking Order {i} - Price: {order[0]}, Volume: {order[1]}")
            if (trade_side == 'sell' and order[0] >= trade_price) or (trade_side == 'buy' and order[0] <= trade_price):
                #print(f"Order meets trade conditions - Order Price: {order[0]}, Trade Price: {trade_price}")
                order_id = self.find_order_id_by_price(order[0], trade_side)
                if order_id:
                    #print(f"Found Order ID {order_id} matching price {order[0]}")
                    if self.check_order_execution(order_id, update['local_timestamp']):
                        matched_volume = min(order[1], remaining_volume)
                        #print(f"Matched Volume: {matched_volume} for Order ID {order_id}")
                        remaining_volume -= matched_volume

                        # Update the order volume or remove the order if fully executed
                        new_order_volume = order[1] - matched_volume
                        if new_order_volume > 0:
                            order_index = book_side.index(order)
                            book_side[order_index] = (order[0], new_order_volume)
                        else:
                            book_side.remove(order)
                            #print(f"Order fully executed and removed from book - Order ID: {order_id}")

                        # Execute trade
                        self.execute_trade(order_id, matched_volume, order[0])
                        #print(f"Executed trade for Order ID {order_id}, Volume: {matched_volume}, Price: {order[0]}")

                        if remaining_volume <= 0:
                            #print("All volume matched, exiting loop")
                            break
                    else:
                        #print(f"Order ID {order_id} did not meet execution criteria")
                        typee =0
                else:
                    #print(f"No matching Order ID found for price {order[0]}")
                    typee =2
            else:
                #print(f"Order at price {order[0]} does not meet trade conditions")
                typee =1

        update['volume'] = remaining_volume
        print(f"Remaining volume after matching: {remaining_volume}")
        return update







    def find_next_price(self, current_price, side):
        """ Find the next available price in the order book. """
        book_side = self.bids if side == 'sell' else self.asks  # Opposite side of the book
        for price, volume in book_side:
            if side == 'sell' and price < current_price:
                return price
            elif side == 'buy' and price > current_price:
                return price
        return current_price  # Return the same price if no next price is found

    def should_execute_order(self, probability, order, update):
        """ Determine if an order should be executed based on probability and update. """
        if (order['side'] == 'buy' and order['price'] >= update['price']) or \
           (order['side'] == 'sell' and order['price'] <= update['price']):
            return probability > .5  # Define this threshold as needed
        return False

    def execute_trade(self, order_id, matched_volume, trade_price):
        """ Execute a trade and return trade details. """
        # ... existing logic to execute the trade ...
        side = 'side'
        print(f'executing trade: orderId: {order_id}, side:  {self.active_orders[order_id][side]}, price: {trade_price}, volume: {matched_volume} ')
        # Remove or update the order in the order book

      # Append trade to my_trades
        trade = {
            'order_id': order_id,
            'price': trade_price,
            'volume': matched_volume,
            'side': self.active_orders[order_id]['side']
        }
        self.my_trades.append(trade)  #

    def calculate_trade_pnl(self, volume, price):
        """ Calculate the profit or loss for a trade. """
        # Example profit or loss calculation, modify as needed
        return volume * price  # Placeholder for PnL calculation logic

    def update_order_book_side(self, book_side, update, descending):
        price = update['price']
        volume = update['volume']

        # Calculate the total volume of your own orders at this price
        my_volume_at_price = sum(order['volume'] for order_id, order in self.active_orders.items() if order['price'] == price)

        # Find existing price and update or remove order
        for i, (p, v) in enumerate(book_side):
            if p == price:
                # Update the total volume, ensuring to include your own volume
                new_volume = max(volume + my_volume_at_price, 0)
                if new_volume > 0:
                    book_side[i] = (price, new_volume)
                else:
                    del book_side[i]  # Remove the price level if the total volume is zero
                break
        else:
            # Add new price level if it doesn't exist and volume is non-zero
            if volume > 0:
                book_side.append((price, volume + my_volume_at_price))
                book_side.sort(key=lambda x: x[0], reverse=descending)

        # Truncate the order book to maintain the specified depth
        if len(book_side) > self.depth:
            book_side = book_side[:self.depth]



    # No need to return anything as the book_side and sorted_prices are updated in place

    def get_current_price(self):
        """ Get the current price from the order book. """
        if self.bids and self.asks:
            top_bid = self.bids[0][0]
            top_ask = self.asks[0][0]
            return (top_bid + top_ask) / 2  # Mid-price as the current price
        return None  # Return None if the order book is empty

    def get_current_book(self):
        """ Returns the current bids and asks. """
        return self.bids[:self.depth], self.asks[:self.depth]


    def get_best_bid_ask(self):
        if self.bids and self.asks:
            best_bid = self.bids[0][0]
            best_ask = self.asks[0][0]
            return best_bid, best_ask
        return None, None

    def find_order_index(self, book_side, price):
        """ Find the index of an order in the order book side based on price. """
        for i, (p, _) in enumerate(book_side):
            if p == price:
                return i
        return None

    def trim_order_book(self):
        """ Trim the order book to maintain the specified depth. """
        self.bids = self.bids[:self.depth]
        self.asks = self.asks[:self.depth]

    def remove_order(self, order_id):
        """ Remove an order from the order book and return its details. """
        print(f"Removing order: Order ID {order_id}")
        removed_order = None

        if order_id in self.active_orders:
            removed_order = self.active_orders[order_id]
            book_side = self.bids if removed_order['side'] == 'buy' else self.asks
            index = self.find_order_index(book_side, removed_order['price'])
            if index is not None:
                _, volume = book_side[index]
                new_volume = volume - removed_order['volume']
                if new_volume > 0:
                    book_side[index] = (removed_order['price'], new_volume)
                else:
                    del book_side[index]
            del self.active_orders[order_id]
            # del self.order_insertion_times[order_id]
            # del self.queue_position_estimate[order_id]

        return removed_order

    def remove_expired_orders(self, current_exchange_time):
        """ Remove orders that have exceeded their lifetime and return their details. """
        expired_order_details = []
        for order_id in list(self.active_orders.keys()):  # Use a list of keys to avoid runtime error
            order_details = self.active_orders[order_id]
            if order_details['expiration'] <= current_exchange_time:
                removed_order = self.remove_order(order_id)
                expired_order_details.append(removed_order)
        return expired_order_details

class PositionManager:
    def __init__(self):
        self.positions = {}  # Dictionary to store positions by asset
        self.maker_fee_percent = 0.0016  # Maker fee as a percentage (0.16%)
        self.realized_pnl = 0  # Realized profit and loss

    def update_position(self, trade):
        print(f"Received trade: {trade}")  # Debugging
        asset = trade['asset']  # Assuming the trade dict has an 'asset' key
        side, volume, trade_price = trade['side'], trade['volume'], trade['price']

        # Ensure volume and trade_price are numerical
        if not isinstance(volume, (int, float)) or not isinstance(trade_price, (int, float)):
            print("Error: Volume or trade_price is not a number.")
            return 0

        if asset not in self.positions:
            self.positions[asset] = {'size': 0, 'avg_price': 0, 'unrealized_pnl': 0}

        current_position = self.positions[asset]
        trade_pnl = 0  # Initialize trade PnL

        # Calculate the maker fee
        maker_fee = trade_price * volume * self.maker_fee_percent

        # Calculate PnL based on the side of the trade
        if side == 'buy':
            # If buying, PnL is realized if it's closing or reducing a short position
            if current_position['size'] < 0:
                trade_pnl = (current_position['avg_price'] - trade_price) * min(volume, -current_position['size']) - maker_fee
        else:  # side == 'sell'
            # If selling, PnL is realized if it's closing or reducing a long position
            if current_position['size'] > 0:
                trade_pnl = (trade_price - current_position['avg_price']) * min(volume, current_position['size']) - maker_fee

        # Update position size and average price
        if side == 'buy':
            new_size = current_position['size'] + volume
            new_avg_price = ((current_position['size'] * current_position['avg_price']) + (volume * trade_price)) / new_size if new_size != 0 else 0
        else:  # side == 'sell'
            new_size = current_position['size'] - volume
            new_avg_price = ((-current_position['size'] * current_position['avg_price']) + (volume * trade_price)) / -new_size if new_size != 0 else 0

        current_position['size'] = new_size
        current_position['avg_price'] = new_avg_price

        # Update realized PnL
        self.realized_pnl += trade_pnl

        # Update unrealized PnL if there's an open position
        if new_size != 0:
            current_position['unrealized_pnl'] = (trade_price - new_avg_price) * new_size
        else:
            current_position['unrealized_pnl'] = 0

        print(f"New position size: {current_position['size']}, New average price: {current_position['avg_price']}, Realized PnL: {self.realized_pnl}")
        return trade_pnl


    # Accessor methods...
    def get_realized_pnl(self):
        return self.realized_pnl

    def get_inventory(self, asset='BTC'):
        return self.positions.get(asset, {'size': 0})['size']

    def get_position(self, asset='BTC'):
        return self.positions.get(asset, {'size': 0, 'avg_price': 0})


    def get_position_size(self):
        """ Get the current position size for BTC. """
        # Access 'BTC' position size from the positions dictionary
        btc_position = self.positions.get('BTC', {'size': 0})
        return btc_position['size']

    def get_average_entry_price(self):
        """ Get the average entry price for BTC. """
        # Access 'BTC' average price from the positions dictionary
        btc_position = self.positions.get('BTC', {'avg_price': 0})
        return btc_position['avg_price']


class StatisticsManager:
    def __init__(self):
        self.total_pnl = 0
        self.total_volume = 0
        self.trades = []
        self.winning_trades = 0
        self.daily_returns = []  # Assuming daily returns are stored here

        self.losing_trades = 0
        self.total_win_amount = 0
        self.total_loss_amount = 0

    def record_trade(self, trade):
        """ Record a trade and update statistics. """
        self.trades.append(trade)
        self.calculate_pnl(trade)
        self.total_volume += trade['volume']

        # Count wins and losses
        if trade['profit_loss'] > 0:
            self.winning_trades += 1
            self.total_win_amount += trade['profit_loss']
        elif trade['profit_loss'] < 0:
            self.losing_trades += 1
            self.total_loss_amount += trade['profit_loss']

    def calculate_pnl(self, trade):
        """ Update total PnL based on the trade. """
        self.total_pnl += trade['profit_loss']

    def get_pnl(self):
        """ Get the total PnL. """
        return self.total_pnl

    def get_average_win(self):
        """ Calculate the average win amount per winning trade. """
        return self.total_win_amount / self.winning_trades if self.winning_trades > 0 else 0

    def get_average_loss(self):
        """ Calculate the average loss amount per losing trade. """
        return self.total_loss_amount / self.losing_trades if self.losing_trades > 0 else 0

    def get_win_loss_ratio(self):
        """ Calculate the win/loss ratio. """
        return self.winning_trades / self.losing_trades if self.losing_trades > 0 else float('inf')

    def get_total_volume(self):
        """ Get the total traded volume. """
        return self.total_volume

    # Additional methods for other statistics can be added here.
    def calculate_sharpe_ratio(self, risk_free_rate=0):
        """ Calculate the Sharpe ratio. """
        excess_returns = np.array(self.daily_returns) - risk_free_rate
        return np.mean(excess_returns) / np.std(excess_returns) if np.std(excess_returns) != 0 else 0

    def calculate_sortino_ratio(self, risk_free_rate=0):
        """ Calculate the Sortino ratio. """
        excess_returns = np.array(self.daily_returns) - risk_free_rate
        negative_returns = [r for r in excess_returns if r < 0]
        return np.mean(excess_returns) / np.std(negative_returns) if np.std(negative_returns) != 0 else 0

    def calculate_risk_return_ratio(self):
        """ Calculate the risk-return ratio. """
        if np.std(self.daily_returns) != 0:
            return np.mean(self.daily_returns) / np.std(self.daily_returns)
        return 0

    def calculate_annualized_return(self, trading_days=252):
        """ Calculate the annualized return. """
        cumulative_return = np.prod([1 + r for r in self.daily_returns]) - 1
        return (1 + cumulative_return) ** (trading_days / len(self.daily_returns)) - 1

    def calculate_max_drawdown(self):
        """ Calculate the maximum drawdown. """
        cumulative_returns = np.cumprod([1 + r for r in self.daily_returns])
        peak = np.maximum.accumulate(cumulative_returns)
        drawdown = (peak - cumulative_returns) / peak
        return np.max(drawdown)


    def calculate_current_return(self, initial_capital):
        """ Calculate the current return as a percentage of the initial capital. """
        if initial_capital <= 0:
            raise ValueError("Initial capital must be greater than zero.")

        current_return = (self.total_pnl / initial_capital) * 100
        return current_return

    def print_summary(self, capital=None):
        """ Print a summary of the trading statistics. """
        # Assuming the necessary metrics are calculated and stored in the class attributes
        print('=========== Summary ===========')
        print(f'Sharpe ratio: {self.sharpe_ratio:.1f}')
        print(f'Sortino ratio: {self.sortino_ratio:.1f}')
        print(f'Risk-return ratio: {self.risk_return_ratio:.1f}')

        if capital is not None:
            annualised_return_percentage = self.annualised_return / capital * 100
            max_drawdown_percentage = self.max_drawdown / capital * 100
            print(f'Annualised return: {annualised_return_percentage:.2f} %')
            print(f'Max. draw down: {max_drawdown_percentage:.2f} %')
        else:
            print(f'Annualised return: {self.annualised_return:.2f}')
            print(f'Max. draw down: {self.max_drawdown:.2f}')

        print(f'The number of trades per day: {self.trades_per_day}')
        print(f'Avg. daily trading volume: {self.avg_daily_volume}')
        print(f'Avg. daily trading amount: {self.avg_daily_amount}')




import matplotlib.pyplot as plt
import matplotlib.patches as patches

import matplotlib.pyplot as plt

class TradingSimulation:
    def __init__(self):
        self.price = []  # Store price data
        self.posi_arr = []  # Store positions data
        self.inventory = []  # Store inventory data
        self.pnl_arr = []  # Store PnL data
        self.trade_data = []  # Store trade execution data
        self.render_on = False

    def _plot_trading(self):
        # Ensure that there is data to plot and check data types
        if not self.price:
            print("Price data not available to plot.")
            return

        # Debugging: Print data types
        print(f"Data types - Price: {type(self.price)}, Position: {type(self.posi_arr)}, Inventory: {type(self.inventory)}, PnL: {type(self.pnl_arr)}")

        # Check if data are lists of numbers and plot only if they are valid
        if not all(isinstance(x, (int, float)) for x in self.price):
            print("Price data contains non-numeric values.")
            return

        # Time axis (x-axis)
        time_axis = list(range(len(self.price)))

        # Clear previous plots if necessary
        self.ax.clear()
        self.ax2.clear()
        self.ax4.clear()
        self.ax5.clear()

        # Plotting trade executions
        buy_trades = [trade for trade in self.trade_data if trade['side'] == 'buy']
        sell_trades = [trade for trade in self.trade_data if trade['side'] == 'sell']

        # Plot trades on the price chart
        self.ax.scatter([trade['time'] for trade in buy_trades], [trade['price'] for trade in buy_trades], color='green', marker='^', label='Buy Trades', s=50)
        self.ax.scatter([trade['time'] for trade in sell_trades], [trade['price'] for trade in sell_trades], color='red', marker='v', label='Sell Trades', s=50)

        # Plotting price data
        self.ax.plot(time_axis, self.price, color='cyan', label='Price', linewidth=2)

        # Plotting positions if available and valid
        if isinstance(self.posi_arr, dict):
            # Convert dictionary to list or handle it as per your logic
            positions_list = [self.posi_arr.get(i, None) for i in time_axis]  # Example conversion
            self.ax2.plot(time_axis, positions_list, color='blue', label='Position', linewidth=2)

        # Plotting inventory if available and valid
        if isinstance(self.inventory, dict):
            # Convert dictionary to list or handle it as per your logic
            inventory_list = [self.inventory.get(i, None) for i in time_axis]  # Example conversion
            self.ax4.plot(time_axis, inventory_list, color='purple', label='Inventory', linewidth=2)

        # Plotting PnL if available and valid
        if self.pnl_arr and all(isinstance(x, (int, float)) for x in self.pnl_arr):
            self.ax5.plot(time_axis, self.pnl_arr, color='orange', label='PnL', linewidth=2)

        # Adding legends and labels
        self.ax.legend(loc='upper left')
        self.ax2.legend(loc='upper right')
        self.ax4.legend(loc='upper left')
        self.ax5.legend(loc='upper left')

        self.ax.set_xlabel('Time Steps')
        self.ax.set_ylabel('Price')
        self.ax2.set_ylabel('Positions')
        self.ax4.set_ylabel('Inventory')
        self.ax5.set_ylabel('PnL')

        # Adjust and display the plot
        plt.draw()
        plt.pause(0.1)

    def render(self, save=False):
        if not self.render_on:
            plt.style.use('dark_background')
            self.render_on = True

            # Define the layout for subplots
            left, width = 0.1, 0.8
            rect_price = [left, 0.4, width, 0.2]
            rect_positions = [left, 0.2, width, 0.15]
            rect_inventory = [left, 0.6, width, 0.2]
            rect_pnl = [left, 0.75, width, 0.15]

            self.fig = plt.figure(figsize=(15, 9))
            self.fig.suptitle('Trading Simulation', fontsize=14, fontweight='bold')

            # Create subplots
            self.ax = self.fig.add_axes(rect_price)  # Price chart
            self.ax2 = self.fig.add_axes(rect_positions, sharex=self.ax)  # Positions chart
            self.ax4 = self.fig.add_axes(rect_inventory, sharex=self.ax)  # Inventory chart
            self.ax5 = self.fig.add_axes(rect_pnl, sharex=self.ax)  # PnL chart

            # Set grid and styling for each subplot
            for ax in [self.ax, self.ax2, self.ax4, self.ax5]:
                ax.grid(color='gray', linestyle='-', linewidth=0.5)

            self._plot_trading()

        if save:
            self.fig.savefig(f'fig/{self.t_index}.png')


    # Other methods as needed


# Corrected BacktestEngine with event processing and handler methods

class BacktestEngine:
    def __init__(self, market_data_stream, population_size, num_generations, actor_critic_hyperparams, ppo_hyperparams, rollout_buffer_params, evolutionary_training_config):
        self.market_data_stream = market_data_stream
        self.population_size = population_size
        self.num_generations = num_generations
        self.actor_critic_hyperparams = actor_critic_hyperparams
        self.ppo_hyperparams = ppo_hyperparams
        self.rollout_buffer_params = rollout_buffer_params
        self.initial_capital = 500
        self.current_capital = self.initial_capital
        #self.env = gym.make(env_name)  # Replace with your specific environment
        self.penalty_threshold = -2000
        # Initialize components for backtesting
        self.lob = OrderBook()  # Limit Order Book
        self.volatility_manager = VolatilityManager()
        self.portfolio_manager = PortfolioManager(self.current_capital)
        self.trading_manager = TradeUpdateManager()

        self.evolutionary_training_config = evolutionary_training_config
        self.evolutionary_model = EvolutionaryModel( self.actor_critic_hyperparams, self.ppo_hyperparams, self.rollout_buffer_params,  self.lob,  self.evolutionary_training_config, self.volatility_manager, self.portfolio_manager, self.current_capital, self.trading_manager)#self.env,
        #self.statistics_manager = StatisticsManager()
        #self.trading_simulation = TradingSimulation()
        #self.strategy = SimpleTwoSidedQuoteStrategy(max_position=100, order_qty=20)

        self.current_time = 0
        self.train_counter = 0
        self.drawdown_max = 0.007
        # Initialize timing for training and evolution
        self.TRAINING_INTERVAL = 5 * 60 * 1000000  # 30 minutes in seconds
        self.EVOLUTION_INTERVAL = 10 * 60 * 1000000  # 2 hours in seconds
        self.TRAINING_FREQUENCY = 4
        self.training_interval = self.EVOLUTION_INTERVAL / self.TRAINING_FREQUENCY
        self.next_training_time = 0
        self.next_evolution_time = self.EVOLUTION_INTERVAL
        self.current_generation = 0

        # ... Other initialization ...


    def calculate_next_training_time(self, current_time):
        # Logic to calculate the next training time
        return current_time + self.TRAINING_INTERVAL

    def calculate_next_evolution_time(self, current_time):
        # Logic to calculate the next evolution time
        return current_time + self.EVOLUTION_INTERVAL

    def update_next_training_time(self):
        # Update the next training time based on the current local exchange time
        if self.train_counter % TRAINING_FREQUENCY == 0:
            self.next_training_time = self.current_time + EVOLUTION_INTERVAL
            self.next_evolution_time = self.next_training_time
        else:
            self.next_training_time += EVOLUTION_INTERVAL / TRAINING_FREQUENCY

        self.train_counter += 1


    def run_backtest(self):
        print("Starting backtest...")
        warm_up_duration =  8 * 60 * 1000000    # 30 minutes in seconds
        warm_up_start_time = None
        warm_up_complete = False
        pop_reduced = False

        with tqdm(total=self.num_generations, desc="Processing Generations") as pbar:
            for index, market_data in self.market_data_stream.iterrows():
                #if isinstance(market_data, dict) and 'local_timestamp' in market_data:
                self.current_time = market_data['local_timestamp']
                    # ... rest of the code ...
                #else:
                    # Handle the case where market_data is not as expected
                    #print(f"Unexpected market_data format: {market_data}")
                            # Start warm-up timer on first data point
                if warm_up_start_time is None:
                    warm_up_start_time = self.current_time
                    print(warm_up_start_time)
                    print("Warm-up period started.")

                # Check if warm-up period is complete
                if not warm_up_complete and self.current_time - warm_up_start_time >= warm_up_duration:
                    warm_up_complete = True
                    self.next_training_time = self.calculate_next_training_time(self.current_time)
                    self.next_evolution_time = self.calculate_next_evolution_time(self.current_time)
                    print("Warm-up period complete. Starting main process.")

                # Process market and trade data
                print("market_data type:", type(market_data))
                print("market_data content:", market_data)

                if market_data['event'] == 1:  # Market data update
                    print("Processing market data update")
                    self.simulate_market_data(market_data)
                    self.process_market_data(market_data, self.evolutionary_model, self.rollout_buffer_params, warm_up_complete) #self.process_market_data(market_data)
                    print("Finished processing market data update")
                elif market_data['event'] == 2:  # Trade execution event
                    print("Processing trade data update")
                    self.process_trade_execution(market_data)
                    self.process_trade_data(market_data, self.evolutionary_model, self.rollout_buffer_params, warm_up_complete)
                    print("Finished processing trade data update")

                # Train models and evolve population after warm-up is complete
                if warm_up_complete :
                    if self.current_time >= self.next_training_time:
                        self.train_and_update_models()
                        self.next_training_time = self.calculate_next_training_time(self.current_time)

                    if self.current_time >= self.next_evolution_time and pop_reduced == False:
                        self.evolutionary_model.evolve_population()
                        self.next_evolution_time = self.calculate_next_evolution_time(self.current_time)

                        for model in self.evolutionary_model.population:
                            model.ppo.target_entropy =  model.ppo.target_entropy - 0.05
                        self.current_generation += 1

                        pbar.update(1)
                        tqdm.write(f"Generation {self.current_generation}/{self.num_generations} completed.")

                    # Check if all generations are concluded
                    if pop_reduced == False:
                        if self.current_generation >= self.num_generations:
                            top_performers = self.evolutionary_model.select_top_performers(3)
                            pop_reduced = True

                            self.TRAINING_INTERVAL = 2 * 60 * 60 # 2 hours in seconds
                            self.calculate_next_training_time(self.current_time)
                            print("All generations concluded. Continuing training with top performers.")


                # Additional backtesting logic if needed
        print("Backtest completed.")

    def round_price(self, price):
        """
        Rounds the price to a maximum of 5 decimal places.
        """
        return round(price, 5)

    def adjust_order_size(self, size, price, aum):
        """
        Ensures the order size is at least the minimum required size, does not exceed available capital,
        and is within 5% of the AUM.
        """
        if size <= 0:
            return -1
        MIN_ORDER_SIZE = 10  # Minimum order size
        MAX_SIZE_BASED_ON_AUM = (0.05 * aum) / price  # Maximum size as 5% of AUM

        # Choose the smaller of the two maximum sizes


        adjusted_size = max(round(MAX_SIZE_BASED_ON_AUM, 0), MIN_ORDER_SIZE)
        return adjusted_size

    def place_order(self, order_size, order_price, is_buy):
        # Logic to place an order
        order_id = self.generate_order_id()
        order = {'order_id': order_id, 'price': order_price, 'size': order_size, 'timestamp': self.current_time, 'side': is_buy}
        # Logic to add order to the system
        # ...
        return  order_id,  order_price,  order_size, is_buy

    def process_action(self, action, model):
        print("Starting process_action...")
        # Unpack the action array
        action = action.flatten()
        bid_price_adjustment, ask_price_adjustment, buy_order_size, sell_order_size, hold, target_inventory_level, cancellation_intensity, cancellation_strategy = action
        print(f"Action unpacked: {action}")
        # Retrieve current inventory and mid-price
        current_inventory = model.environment.order_manager.inventory
        current_mid_price = model.environment.market_data_transformation.cached_mid_price if model.environment.market_data_transformation.cached_mid_price else 0
        print(f"Current inventory: {current_inventory}, Current mid-price: {current_mid_price}")

        # Initialize a list to keep track of orders
        orders = []
        t_i_l = (model.environment.initial_capital * target_inventory_level)
        print(f"Target inventory level: {t_i_l}")
        model.environment.order_manager.update_target_inventory(t_i_l)
        # Process 'hold' action
        print(f"Hold action value: {hold}")
        if hold < 0.5:
            # Calculate bid and ask prices
            bid_price = self.round_price(current_mid_price * (1 + bid_price_adjustment))
            ask_price = self.round_price(current_mid_price * (1 + ask_price_adjustment))
            print(f"Bid price: {bid_price}, Ask price: {ask_price}")

            current_time = self.current_time

            # Adjust order sizes
            adjusted_buy_order_size = self.adjust_order_size(buy_order_size, bid_price, model.environment.initial_capital)

            # Check against target inventory level before placing buy orders
            if adjusted_buy_order_size > 0 :
                if model.environment.cash_balance >= adjusted_buy_order_size * bid_price:
                    order_id, price, volume, side = self.place_order(adjusted_buy_order_size, bid_price, is_buy='buy')
                    model.lob_instance.place_limit_order(order_id, price, volume, side, current_time)
                    model.environment.order_manager.place_order(order_id, volume, price, side, current_time)
                    model.environment.rate_limiter.update_on_order_placement(current_time)

                    #ratelimiter

            adjusted_sell_order_size = self.adjust_order_size(sell_order_size, ask_price, model.environment.initial_capital)

            # Check against target inventory level before placing sell orders

            if adjusted_sell_order_size > 0 :
                # Check if there's enough cash balance to place the sell order
                if model.environment.cash_balance >= adjusted_sell_order_size * ask_price:
                    # Logic to place the sell order, assuming short selling is always allowed
                    order_id, price, volume, side = self.place_order(adjusted_sell_order_size, ask_price, is_buy='sell')
                    model.lob_instance.place_limit_order(order_id, price, volume, side, current_time)
                    model.environment.order_manager.place_order(order_id, volume, price, side, current_time)
                    # Increment the rate limiter
                    model.environment.rate_limiter.update_on_order_placement(current_time)


        # Update target inventory level regardless of 'hold' action


        # Handle order cancellation if necessary
        if cancellation_strategy > 0.6:
            model.environment.process_cancellation(model, cancellation_intensity, cancellation_strategy, self.current_time)
            print("Order cancellation logic executed")



        # Update the rate limiter based on the actions taken
        # model.rate_limiter.update_rate_limit_status()

        # return orders




    # def process_market_data(self, market_data, evolutionary_model, hyperparams):
    #     for i, model in enumerate(evolutionary_model.population):
    #         # Step 1: Update model's LOB instance with market data
    #         model.lob_instance.process_market_data(market_data)
    #         #order_book_lob = model.lob_instance.get_order_book()
    #         # Step 2: Construct State Vector
    #         current_time = market_data['local_timestamp']
    #         #model.environment.orderbook.update_order_book(market_data) THIS FUCNTION IS IMPORTANT FOR LIVE
    #         model.environment.order_book_manager.update_from_lob_instance(model.lob_instance)


    #         if  self.model.decision_time < current_time or self.model.decision_time == None:
    #             start_time = time.time()
    #             order_book_snapshots = model.environment.orderbook.get_order_book_history()
    #             current_trades = model.environment.trade_update_manager.get_latest_trades()


    #             state = model.environment.market_data_transformation.construct_state_vector(order_book_snapshots, current_trades)

    #             # Step 3: Sample Action from the model
    #             action, log_prob, value = model.actor_critic.act(state)
    #             action = action.cpu().numpy()

    #             # Step 4: Process Action in Market Environment
    #             self.process_action(action, model)
    #             end_time = time.time()
    #             print(f'Time taken to process action: {end_time-start_time}')
    #             calc_time = end_time - start_time
    #             self.model.decision_time = current_time + calc_time
    #         else:
    #             self.rate_limiter.simulate_time_step(current_time)

    #         book = model.lob_istance.get_order_book()

    #         # Step 5: Retrieve recent trades executed by the model
    #         recent_trades = model.lob_instance.get_my_recent_trades()

    #         # Step 6: Update Order Management with Trade Data
    #         self.update_order_manager(model.environment.order_manager, recent_trades)


    #         # Step 7: Calculate Reward
    #         reward = self.get_reward(model, market_data)
    #         model.performance mulated_reward += reward  # Accumulate reward for the model

    #         print(f'Reward: {reward}')
    #         # Step 8: Check if Episode is Done
    #         done, termination_reason = self.check_if_done(model, market_data)

    #         # Step 9: Update the Rollout Buffer
    #         model.rollout_buffer.add(state, action, reward, log_prob, value, done)

    #         if done:
    #             # Reset the model's state for a new training session
    #             self.train_model(model, hyperparams)
    #             self.reset_model_state(model)

    def process_market_data(self, market_data, evolutionary_model, hyperparams, warm_up_complete):
        print("Starting process_market_data...")
        for i, model in enumerate(evolutionary_model.population):
            print(f'Processing model {i}')

            # Step 1: Update model's LOB instance with market data
            model.lob_instance.process_market_data(market_data)
            print(f'Market data updated for model {i}')

            # Step 2: Construct State Vector
            current_time = market_data['local_timestamp']
            model.environment.orderbook.update_from_lob_instance(model.lob_instance)

            print(f'State vector constructed for model {i}')

            # Only process actions if warm-up is complete
            if warm_up_complete:
                print("Warm-up complete, processing actions")
                if model.decision_time is None or model.decision_time < current_time:

                    start_time = time.time()
                    order_book_snapshots = model.environment.orderbook.get_order_book_history()
                    current_trades = model.environment.trade_update_manager.get_latest_trades()
                    print(f"Order book snapshots and current trades retrieved for model {i}")
                    model.lob_instance.display_book()
                    num, state = model.environment.market_data_transformation.construct_state_vector(order_book_snapshots, current_trades, current_time)
                    if num == 1:
                        action, log_prob, value = model.actor_critic.act(state)
                        #action = action.cpu().numpy() GPU
                        action = action.detach().cpu().numpy()

                        print(f"Action sampled from model {i}: {action}")

                        # Step 4: Process Action in Market Environment
                        self.process_action(action, model)
                        end_time = time.time()
                        calc_time = end_time - start_time
                        print(f"Action processed for model {i}. Time taken: {calc_time}")
                        model.decision_time = current_time + calc_time
                    else:
                      continue
                else:
                    self.rate_limiter.simulate_time_step(current_time)
                    #rint("Rate limiter simulated time step")

                # Step 5: Retrieve recent trades executed by the model
                recent_trades = model.lob_instance.get_my_recent_trades()
                print(f"Recent trades retrieved for model {i}")
                if recent_trades:
                      # Step 7: Update Order Management with Trade Data
                      self.update_order_manager(model, recent_trades, current_time)
                      print(f"Order manager updated for model {i}")
                else:
                      print(f"No recent trades for model {i}, skipping order manager update")
                print(f'Current time: {current_time}')
                print(f'Decision time: {model.decision_time}')


                # Step 7: Calculate Reward
                reward = self.get_reward(model, market_data)
                model.performance += reward
                print(f"Reward calculated for model {i}: {reward}")

                # Step 8: Check if Episode is Done
                done, termination_reason = self.check_if_done(model)
                print(f"Checked if episode is done for model {i}. Done: {done}, Reason: {termination_reason}")

                # Step 9: Update the Rollout Buffer
                model.rollout_buffer.add(state, action, reward, log_prob, value, done)
                print(f"Rollout buffer updated for model {i}")

                if done:
                    # Reset the model's state for a new training session
                    self.train_model(model,hyperparams)
                    self.reset_model_state(model)
                    print(f"Model {i} state reset for new training session")

        print("Finished process_market_data")


            # Additional checks or updates can be added here if necessary

# ... [Rest of the methods]

    def get_reward(self, model, market_data):
        # Calculate the reward for the given model using its reward manager
        reward = model.environment.reward_manager._get_reward(self.current_time)#model, market_data
        return reward

    def reset_model_state(self, model):
        # Reset the model's LOB to the base state
        model.lob_instance = copy.deepcopy(self.lob)
        model.trading_manager = copy.deepcopy(self.trading_manager)

        model.portfolio_manager = copy.deepcopy(self.portfolio_manager)
        model.volatility_manager = copy.deepcopy(self.volatility_manager)

        model.environment = ContinuousTradingEnvironment(model.volatility_manager, model.portfolio_manager, self.initial_capital, model.trading_manager)
        # Reset the accumulated reward
        model.performance = 0
        # Reset the model's internal state
        # model.reset()

    def check_if_done(self, model):
        # Check various conditions from the Reward class
        if self.check_drawdown_condition(model):
            return True, "Drawdown limit exceeded"
        if self.check_penalty_threshold(model):
            return True, "Penalty threshold exceeded"
        if self.check_capital_depletion(model):
            return True, "Capital depleted"

        # Other conditions...

        return False, ""

    # Example helper methods for different conditions
    def check_drawdown_condition(self, model):
        # Check if the drawdown exceeds a certain threshold
        drawdown_penalty = self.get_drawdown_penalty(model)
        return drawdown_penalty > self.drawdown_max

    def check_penalty_threshold(self, model):
        # Check if the total penalty exceeds a certain threshold
        rate_limit_penalty = model.environment.rate_limiter.get_usage_percentage()

        percentage = rate_limit_penalty / 100

        return percentage > 1

    def check_capital_depletion(self, model):
        # Check for capital depletion using the capital stored in the model's environment
        current_capital = model.environment.portfolio_manager.current_capital
        return current_capital < 250

    def select_top_performers(self, num_top_performers, model):
        sorted_population = sorted(self.population, key=lambda model: model.performance, reverse=True)
        return sorted_population[:num_top_performers]


    def get_drawdown_penalty(self, model):
        # Calculate the drawdown penalty using the drawdown manager
        drawdown_penalty = model.environment.reward_manager.get_drawdown_penalty()
        return drawdown_penalty

    def process_trade_data(self, market_data, evolutionary_model, hyperparams, warm_up_complete):
        print("Starting process_trade_data...")
        for i, model in enumerate(evolutionary_model.population):
            print(f'Processing trade data for model {i}')

            # Step 1: Update model's LOB instance with market data
            current_time = market_data['local_timestamp']
            model.lob_instance.handle_trade_execution(market_data)
            self.trading_manager.update_trades(market_data)
            model.environment.trade_update_manager.update_trades(market_data)
            model.volatility.update_tick_data(market_data, current_time)
            print(f'Market data and trade execution updated for model {i}')

            if warm_up_complete:
                print("Warm-up complete, processing actions")
                if model.decision_time is None or model.decision_time < current_time:

                    start_time = time.time()

                    # Step 2: Retrieve the entire order book history and current trades
                    order_book_history = model.environment.orderbook.get_order_book_history()
                    current_trades = model.environment.trade_update_manager.get_latest_trades()
                    print(f"Order book history and current trades retrieved for model {i}")

                    # Step 3: Construct State Vector
                    num, state = model.environment.market_data_transformation.construct_state_vector(order_book_history, current_trades, current_time)
                    print(f"State vector constructed for model {i}")
                    if num == 1:

                        # Step 4: Sample Action from the model
                        action, log_prob, value = model.actor_critic.act(state)
                        #action = action.cpu().numpy() # GPU
                        action = action.detach().cpu().numpy()

                        print(f"Action sampled from model {i}: {action}")

                        # Step 5: Process Action in Market Environment
                        self.process_action(action, model)
                        end_time = time.time()
                        calc_time = end_time - start_time
                        print(f"Action processed for model {i}. Time taken: {calc_time}")
                        model.decision_time = current_time + calc_time
                    else:
                        continue
                else:
                    self.rate_limiter.simulate_time_step(current_time)
                    print("Rate limiter simulated time step")

                # Step 6: Retrieve recent trades executed by the model
                recent_trades = model.lob_instance.get_my_recent_trades()
                print(f"Recent trades retrieved for model {i}")
                if recent_trades:
                      # Step 7: Update Order Management with Trade Data
                      self.update_order_manager(model, recent_trades, current_time)
                      print(f"Order manager updated for model {i}")
                else:
                      print(f"No recent trades for model {i}, skipping order manager update")


                # Step 8: Calculate Reward
                reward = self.get_reward(model, market_data)
                model.performance += reward
                print(f"Reward calculated for model {i}: {reward}")

                # Step 9: Check if Episode is Done
                done, termination_reason = self.check_if_done(model)
                print(f"Checked if episode is done for model {i}. Done: {done}, Reason: {termination_reason}")

                # Step 10: Update the Rollout Buffer
                model.rollout_buffer.add(state, action, reward, log_prob, value, done)
                print(f"Rollout buffer updated for model {i}")

                if done:
                    # Reset the model's state for a new training session
                    self.train_model(model, hyperparams)
                    self.reset_model_state(model)
                    print(f"Model {i} state reset for new training session")

        print("Finished process_trade_data")


            # Additional checks or updates can be added here if necessary

    def update_order_manager(self, model, trades, current_time):
        #model.environment.trade_update_manager.update_trades(trades)
        model.environment.trade_update_manager.format_and_append_trade_data(trades, current_time)
        model.volatility.update_tick_data(trades, current_time)
        for trade in trades:
            order_id = trade['order_id']
            matched_price = trade['price']
            matched_volume = trade['volume']
            is_buy = trade['side'] == 'buy'

            # Update the OrderManager within the model's environment with trade execution details
            model.environment.order_manager.process_order_execution(order_id, matched_volume, matched_price)


# ... [Rest of the methods]

    def train_and_update_models(self):
        # Loop through each model in the population
        for model in self.evolutionary_model.population:
            # Check if the rollout buffer has enough data to train
            if len(model.rollout_buffer.states) >= self.hyperparams['training_batch_size']:
                print(f"Training Model ID {id(model)}")

                # Perform training
                last_value = model.actor_critic.critic_forward(model.rollout_buffer.states[-1])
                model.rollout_buffer.compute_returns_and_advantage(last_value, self.hyperparams['gamma'], self.hyperparams['lambda_gae'])
                avg_actor_loss, avg_critic_loss, avg_entropy_loss = model.ppo.update(model.rollout_buffer)

                # Reset the rollout buffer for the next training session
                model.rollout_buffer.reset()

                # Debugging print statements
                print(f"Model ID {id(model)}, Actor Loss: {avg_actor_loss}, Critic Loss: {avg_critic_loss}, Entropy: {avg_entropy_loss}")

                # Optional: Logging
                # logging.info(f'Model ID {id(model)}, Actor Loss: {avg_actor_loss}, Critic Loss: {avg_critic_loss}, Entropy: {avg_entropy_loss}')
            else:
                print(f"Not enough data to train Model ID {id(model)}")

    # def train_model(self,model, hyperparams):
    #     # Ensure there's enough data for training
    #     # if rollout_buffer.ptr >= hyperparams['min_batch_size']:
    #     # Prepare the buffer for training, e.g., computing returns and advantages
    #     last_state_tensor = torch.tensor(model.rollout_buffer.states[-1], dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    #     last_value = model.actor_critic.forward(last_state_tensor).detach().item()  # Assuming critic_forward returns a tensor

    #     # Call compute_returns_and_advantage with gamma and lambda_gae from hyperparams
    #     model.rollout_buffer.compute_returns_and_advantage(last_value, hyperparams['gamma'], hyperparams['lambda_gae'])

    #     # Collect all batches into a list or generator
    #     sequential_batches = list(model.rollout_buffer.get_sequential_mini_batches(hyperparams['mini_batch_size']))
    #     random_batches = list(model.rollout_buffer.get_random_mini_batches(hyperparams['mini_batch_size']))

    #     # First Epoch: Sequential
    #     model.ppo.update(sequential_batches)

    #     # Second Epoch: Random
    #     model.ppo.update(random_batches)

    #     # Reset the buffer after training
    #     model.rollout_buffer.reset()
    def train_model(self, model, hyperparams):
        print("Starting training model...")

        # Debug: Check the size of the states buffer
        print(f"Number of states in buffer: {len(model.rollout_buffer.states)}")

        # Assuming the last state is correctly shaped, let's print its shape
        if len(model.rollout_buffer.states) > 0:
            print(f"Shape of the last state: {model.rollout_buffer.states[-1].shape}")

        # Prepare the buffer for training, e.g., computing returns and advantages
        last_state_tensor = torch.tensor(model.rollout_buffer.states[-1], dtype=torch.float32)
        print(f"Shape of last_state_tensor for model input: {last_state_tensor.shape}")

        # Passing last_state_tensor through the model
         # Correctly handling the model's forward method when it returns multiple outputs
        mean, std, nu, value_tensor = model.actor_critic.forward(last_state_tensor)
        last_value = value_tensor.detach().item()
        print(f"Last value from critic: {last_value}")
        # Call compute_returns_and_advantage with gamma and lambda_gae from hyperparams
        model.rollout_buffer.compute_returns_and_advantage(last_value, hyperparams['gamma'], hyperparams['lambda_gae'])

        # Debug: Check if returns and advantages are computed
        print("Returns and advantages computed.")

        # Collect all batches into a list or generator
        sequential_batches = list(model.rollout_buffer.get_sequential_mini_batches(hyperparams['mini_batch_size']))
        random_batches = list(model.rollout_buffer.get_random_mini_batches(hyperparams['mini_batch_size']))

        print(f"Number of sequential batches: {len(sequential_batches)}")
        print(f"Number of random batches: {len(random_batches)}")

        # First Epoch: Sequential
        print("Updating model with sequential batches...")
        model.ppo.update(sequential_batches)  # Pass mean, std, nu for policy update

        # Second Epoch: Random
        print("Updating model with random batches...")
        model.ppo.update(random_batches)  # Pass mean, std, nu for policy update


        # Reset the buffer after training
        model.rollout_buffer.reset()
        print("Training completed. Buffer reset.")


    # def check_capital_depletion(self, model):
    #     # Check for capital depletion using the capital stored in the model's environment
    #     current_capital = model.portfolio_manager.current_capital
    #     return current_capital < self.min_capital_threshold

    #     # Evolve the population and reset rewards after each generation
    #     self.evolutionary_model.evolve_population()
    #     for model in self.evolutionary_model.population:
    #         model.accumulated_reward = 0  # Reset rewards for the new generation

            # Other backtesting logic: plotting, statistics updating, etc.

#     def run_backtest(self, market_data_stream):
#         print("Starting backtest...")
#         self.trading_simulation.render()
#         market_data_index = 0
#         total_market_data_points = len(market_data_stream)
#         count=0
#         for index, market_data in market_data_stream.iterrows():
#     # Process each row of market data
#             count+=1
#             print(count)
#            #market_data = market_data_stream[market_data_index]
#             print(f"Processing market data: {market_data}")
#             # Calculate ingestion latency and schedule market event
#             ingestion_latency = self.calculate_ingestion_latency(market_data)
#             #self.schedule_event(market_data, ingestion_latency)

#             # Update the current time to the maximum of current time and market data's timestamp plus latency
#             # Set current time to the local timestamp of the market data
#             self.current_time = market_data['local_timestamp']
#             price = self.lob.get_current_price()
#             expired_orders = self.lob.remove_expired_orders(self.current_time)
# #             for expired_order in expired_orders:
# #                 self.position_manager.update_position_on_expiry(expired_order, price)
#             self.lob.display_book()
#             print(f"Current simulation time: {self.current_time}")

#             if market_data['event'] == 1:  # Market data update
#                 print("Processing market data update")
#                 self.simulate_market_data(market_data)
#                 print("Finished processing market data update")
#             elif market_data['event'] == 2:  # Trade execution event
#                 print("Processing trade data update")
#                 #trade_execution_event = TradeExecutionEvent(market_data['trade'])  # Assuming market_data['trade'] contains trade details
#                 self.process_trade_execution(market_data)
#                 print("Finished processing trade data update")
#             # Check if it's time to make a new decision
#             if self.current_time >= self.next_decision_time:
#                 print('Making Decision')
#                 # Handle any orders generated as a result of the decision
#                 start_time = time.time()  # Start time
#                 current_position_size = self.position_manager.get_position_size()
#                 print(f"Current BTC Position Size: {current_position_size}")  # Get current position size
#                 best_bid, best_ask = self.lob.get_best_bid_ask()
#                 if best_bid != None or best_ask != None:
#                     # Get the decision from the strategy
#                     bid_price, ask_price = self.strategy.make_decision(best_bid, best_ask, current_position_size,self.lob.active_orders)

#                     end_time = time.time()
#                     execution_time = end_time - start_time
#                     print(execution_time)
#                     self.next_decision_time = self.current_time + execution_time #+ ingestion_latency / 1e6
#                     if bid_price != None:
#                         # Place orders based on strategy's decision
#                         buy_order_id = self.generate_order_id()
#                         self.lob.place_limit_order(buy_order_id, bid_price, volume=self.strategy.order_qty, side='buy', current_time=self.current_time)
#                         #self.strategy.submit_order(buy_order_id,'buy', bid_price)
#                     if ask_price != None:
#                         sell_order_id = self.generate_order_id()
#                         self.lob.place_limit_order(sell_order_id, ask_price, volume=self.strategy.order_qty, side='sell', current_time=self.current_time)
#                         #self.strategy.submit_order(sell_order_id,'buy', bid_price)
#             # Process all scheduled events (market updates and orders)
#             #self.process_scheduled_events()
#             print(f"Next decision time: {self.next_decision_time}")

#             trade_data = self.get_trade_data()
#             self.trading_simulation.trade_data = trade_data
#             for trade in trade_data:
#                 print(f"Processing trade: {trade}")
#                 pnl = self.position_manager.update_position(trade)
#                 trade['profit_loss'] = pnl  # Add PnL to the trade dictionary

#                 # Record the trade with PnL in the StatisticsManager
#                 self.statistics_manager.record_trade(trade)



#             # Update data for plotting
#            # Update data for plotting
#             new_price = self.get_current_price()  # Example method to get current price
#             if new_price is not None:
#                 self.trading_simulation.price.append(new_price)
#             #self.trading_simulation.reward_arr = self.get_rewards()
#             self.trading_simulation.posi_arr = self.get_positions()
#             self.trading_simulation.inventory = self.get_inventory()
#             self.trading_simulation.trade_data = self.get_trade_data()
#             #self.trading_simulation.obs_features = self.get_features()

#             # Call the plotting function
#             self.trading_simulation._plot_trading()

#             # Increment market data index to process the next market data point
#             market_data_index += 1
#             print(f'PNL : {self.statistics_manager.get_pnl()}')
#             current_return = self.statistics_manager.calculate_current_return(self.initial_capital)
#             print(f"Current Return: {current_return:.2f}%")

#         # Print summary of statistics at the end of the backtest
#         self.statistics_manager.print_summary(capital=self.initial_capital)
#         print("Backtest completed.")

    def generate_order_id(self):
        return str(uuid.uuid4())

    def calculate_ingestion_latency(self, market_data):
        """ Calculate the latency between exchange and local timestamps. """
        latency = market_data['local_timestamp'] - market_data['exch_timestamp']

        # Example calculation (adjust based on your data format)
        print(f"Calculated ingestion latency: {latency}")
        return latency

    def schedule_event(self, market_data, ingestion_latency):
        """ Schedule a market event based on ingestion latency. """
        event_time = market_data.exch_timestamp + ingestion_latency
        self.event_queue.put((event_time, market_data))

    def process_scheduled_events(self):
        """ Process events that are scheduled to occur up to the current time. """
        while not self.event_queue.empty():
            event_time, event = self.event_queue.queue[0]  # Peek at the next event
            if event_time <= self.current_time:
                _, event = self.event_queue.get()
                self.process_event(event)
            else:
                break

    def calculate_decision_time(self):
        """ Calculate the time taken for decision-making. """
        # Decision time can be a fixed value or dynamically calculated
        return random.uniform(0.001, 0.005)  # Random time between 1ms and 5ms for example



    def process_trade_execution(self, trade_execution_event):
        """ Process a trade execution event. """
        self.lob.handle_trade_execution(trade_execution_event)


    def simulate_market_data(self, market_data):
        """ Simulate the arrival of new market data and update the order book. """
        print(f"Simulating market data: {market_data}")
        executed_trades = self.lob.process_market_data(market_data)
#         for trade in executed_trades:
#             # Update position and record the trade
#             self.position_manager.update_position(trade['asset'], trade['side'], trade['volume'])
#             self.statistics_manager.record_trade(trade)

#             # Optional: Update daily returns and other statistics
#             pnl_change = trade['price'] * trade['volume']
#             self.statistics_manager.daily_returns.append(pnl_change / self.initial_capital)

#         return executed_trades

    def execute_simple_strategy(self):
        print("Executing simple strategy")
        end_time=0
        best_bid, best_ask = self.lob.get_best_bid_ask()


        if best_bid and best_ask:
            best_bid=best_bid+0.00001
            best_ask=best_ask-0.00001
            end_time = time.time()

            # Place buy order at best bid
            buy_order_id = self.generate_order_id()
            self.lob.place_limit_order(buy_order_id, best_bid, volume=2, side='buy', current_time=self.current_time)
            self.strategy.submit_order(buy_order_id,'buy', bid_price)
            # Place sell order at best ask
            sell_order_id = self.generate_order_id()
            self.lob.place_limit_order(sell_order_id, best_ask, volume=2, side='sell', current_time=self.current_time)
        return end_time

    def handle_generated_orders(self):
        """ Handle any orders generated from decision making. """
        # Simulate a trading strategy decision
        print("Handling generated orders")
        new_order = self.generate_new_order_based_on_strategy()
        if new_order:
            self.handle_order(new_order)

    def generate_new_order_based_on_strategy(self):
        """ Generate a new order based on a trading strategy. """
        # Example trading strategy logic
        # This is a placeholder - replace with your strategy logic
        if random.choice([True, False]):  # Randomly decide to place an order or not
            order_side = random.choice(['buy', 'sell'])
            order_price = self.get_current_market_price()  # Placeholder for market price
            order_volume = random.uniform(1, 10)  # Random volume
            order = {'side': order_side, 'price': order_price, 'volume': order_volume}
            return OrderEvent(order)
        else:
            return None

    def handle_order(self, order_event):
        """ Handle new order events with added latency. """
        order = order_event.order
        execution_time = self.current_time + self.calculate_order_latency()
        order = order_event.order
        order_side = order['side']
        order_price = order['price']
        order_volume = order['volume']
        order_expiration = order['expiration']
        trade_execution_event = TradeExecutionEvent(order)
        print(f"Handling order: {order_event.order}")
        self.event_queue.put(execution_time,trade_execution_event)
        #self.order_execution_queue.put((execution_time, order))

    def calculate_order_latency(self):
        """ Calculate the latency for order execution. """
        # Fixed or dynamic latency
        return self.order_execution_latency


    def get_current_price(self):
        price = self.lob.get_current_price()
        print(f"Current market price: {price}")
        return price

    def get_positions(self):
        # Retrieve current positions from PositionManager
        return self.position_manager.positions


    def get_inventory(self):
        """ Retrieve inventory data for BTC. """
        return {'BTC': self.position_manager.get_position_size()}

    def get_trade_data(self):
        # Assuming you have a method or a log that records trade data
        return self.lob.get_my_recent_trades()



# # Event Types
# class Event:
#     """ Base class for all events. """
#     pass

# import pandas as pd

# class MarketEvent(Event):
#     """ Event for market updates and trade executions. """
#     LIMIT_ORDER = 1
#     TRADE_EXECUTION = 2

#     def __init__(self, event_type, exch_timestamp, local_timestamp, side, price, qty):
#         self.event_type = event_type
#         self.exch_timestamp = exch_timestamp
#         self.local_timestamp = local_timestamp
#         self.side = side
#         self.price = price
#         self.qty = qty

# def load_data(data):
#     """ Load data and create MarketEvent instances. """
#     events = []
#     for index, row in data.iterrows():
#         event_type = MarketEvent.LIMIT_ORDER if row['event'] == 1.0 else MarketEvent.TRADE_EXECUTION
#         event = MarketEvent(event_type, row['exch_timestamp'], row['local_timestamp'], row['side'], row['price'], row['qty'])
#         events.append(event)
#     return events

# # Example usage
# data = pd.DataFrame([
#     # Your data rows here
#     # Example: [1.0, 1.702079e+15, 1.702079e+15, -1.0, 0.66856, 4531.652925],
#     # ...
# ])

# market_events = load_data(data)

# class TradeExecutionEvent(Event):
#     """ Event for trade executions. """
#     def __init__(self, trade):
#         self.trade = trade

# class OrderEvent(Event):
#     """ Event for new orders. """
#     def __init__(self, order):
#         self.order = order

# class TimerEvent(Event):
#     """ Event for simulating the passage of time. """
#     pass

# class SimpleTwoSidedQuoteStrategy:

#     def __init__(self, max_position=100, order_qty= 20):
#         self.max_position = max_position

#         self.skew = 0.01  # Skew per unit of position
#         self.max_sigfigs = 5  # Maximum number of significant figures

#         self.order_qty = order_qty
#         self.order_id = 0
#         self.position = 0  # Current position
#         self.active_orders = {}  # Store active orders

#     def make_decision(self, best_bid, best_ask, current_position, active_orders):
#         # Calculate mid_price and reservation price
#         mid_price = (best_bid + best_ask) / 2.0
#         #reservation_price = mid_price - self.skew * current_position

#         # Calculate spread and adjust half spread
#         spread = best_ask - best_bid
#         half_spread = spread / 2
#         self.half_spread = max(half_spread * 0.9, 0.00001)  # Adjusted and bounded by min price precision
#         self.half_spread = round(self.half_spread, self.max_sigfigs)  # Round to max price precision

#         # Calculate bid and ask prices based on reservation price and half spread
#         #bid_price = max(reservation_price - self.half_spread, 0)  # Ensure bid price is non-negative
#         #ask_price = reservation_price + self.half_spread
#         ask_price = mid_price + self.half_spread
#         bid_price = mid_price - self.half_spread
#         ask_price = round(ask_price, self.max_sigfigs)  # Round to max price precision
#         bid_price = round(bid_price, self.max_sigfigs)  # Round to max price precision

#         # Check the count of active buy and sell orders
#         buy_order_count = sum(1 for order in active_orders.values() if order['side'] == 'buy')
#         sell_order_count = sum(1 for order in active_orders.values() if order['side'] == 'sell')
#         print(f"Buy order count: {buy_order_count}, Sell order count: {sell_order_count}")
#         if buy_order_count < 1:
#             ask_price = None
#         # Adjust decision based on the count of active orders
#         if buy_order_count >= 5:
#             bid_price = None  # Do not place a new buy order
#         if sell_order_count >= 5:
#             ask_price = None  # Do not place a new sell order

#         # Return the calculated bid and ask prices
#         return bid_price, ask_price


#     def submit_order(self, order_id, side, price):

#         self.active_orders[order_id] = {'side': side, 'price': price}

#         print(f"Order submitted: ID {order_id}, Side {side}, Price {price}")

#     def cancel_order(self, order_id):
#         """
#         Cancel an existing order.

#         :param order_id: ID of the order to cancel
#         """
#         # Logic to cancel order
#         # ...
#         del self.active_orders[order_id]
#         print(f"Order cancelled: ID {order_id}")

#     def update_position(self, trade):
#         """
#         Update the current position based on executed trades.

#         :param trade: Trade data
#         """
#         if trade['side'] == 'buy':
#             self.position += trade['volume']
#         else:
#             self.position -= trade['volume']
#         print(f"Position updated: {self.position}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def load_csv_data(file_path):
    try:
        # Read the CSV file
        data = pd.read_csv(file_path)

        # Print the first few rows of the DataFrame
        print(data.head())

        return data
    except Exception as e:
        print(f"An error occurred while reading the CSV file: {e}")
        return None


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
marketdata_df = pd.read_csv('/content/drive/MyDrive/filenamefile.csv')

# # Convert DataFrame to list of MarketEvent objects
# marketdata_events = load_data(marketdata_df)

# Initialize the BacktestEngine
population_size =6  # Number of agents in the population
num_generations = 10
engine = BacktestEngine(marketdata_df, population_size, num_generations, actor_critic_hyperparams, ppo_hyperparams, environment_training_settings, evolutionary_training_config)

# Run the backtest
engine.run_backtest()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
OrderBook initialized with depth: 10


/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Starting backtest...


Processing Generations:   0%|          | 0/10 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
feature_name: ema_buy_sell_log_distance, x: 439.3941612295557, y: -0.022889600041385353
log_distance: 29.11124780174978
State vector: [0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0.00013973141573454532, 6.986082710477848e-06, 0.0003897618617258173, 1.948429621511927e-05, 0.00024999551376420645, 1.249821350464142e-05, 0.0, 0, 0.10164061322360649, 0.005359254137513325, 0, 0, 0, 0, 0.163028543655666, 0.06762387895196391, 25.06441236841673, 2.8669944563273684, 1, 1, 0, 0, 1, 1, 0.09540466470370208, 0.8516369983075791, 0.9999657051148472, 0.514025716791165, 0.41862105208746303, 0.9997688447607455, 0, 1, 0, 1, 0, 1, 0, 1, -5.246346799477658, 1, -19197.235864109145, 1, 0.0, 0, 7.00000000000145e-05]
State size: 66
State tensor: torch.Size([1, 66])
Action sampled from model 13: [[-5.99999999e-04  5.99999999e-04  2.64999996e-02  2.64999996e-02
   0.00000000e+00 -3.49999994e-01  1.00000000e+00  0.00000000e+00]]
Starting process

<ipython-input-25-b23c665de96a>:1017: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.returns[t] = last_advantage + self.values[t]
<ipython-input-25-b23c665de96a>:885: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  value_loss = F.mse_loss(state_values, returns)
Processing Generations:   0%|          | 0/10 [06:48<?, ?it/s]

Debug: Total Bid Volume for most recent snapshot: 109159.58287046
Debug: Total Ask Volume for most recent snapshot: 84591.51698504
Debug: Imbalance for most recent snapshot: 24568.06588542
Debug: Imbalance Ratio for most recent snapshot: 1.2904317922300041
trades_data_snapshots: [event              2.000000e+00
exch_timestamp     1.702076e+15
local_timestamp    1.702076e+15
side               1.000000e+00
price              6.687600e-01
volume             1.400747e+02
Name: 1310, dtype: float64, event              2.000000e+00
exch_timestamp     1.702076e+15
local_timestamp    1.702076e+15
side               1.000000e+00
price              6.685000e-01
volume             1.544844e+03
Name: 2641, dtype: float64, event              2.000000e+00
exch_timestamp     1.702076e+15
local_timestamp    1.702076e+15
side               1.000000e+00
price              6.683400e-01
volume             1.591396e+03
Name: 3543, dtype: float64, event              2.000000e+00
exch_timestamp     1.702076

AssertionError: NaN detected in shared_output after shared_layers

In [ ]:
import pandas as pd

def read_csv_files(update_files, trade_files):
    all_update_dfs = []
    all_trade_dfs = []

    for update_file in update_files:
        try:
            update_df = pd.read_csv(update_file, names=["Local Timestamp", "Price", "Volume", "Exchange Timestamp", "Type"], header=0)
            update_df['Exchange Timestamp'] = (update_df['Exchange Timestamp'] * 1e9).astype('int64')
            all_update_dfs.append(update_df)
        except FileNotFoundError:
            print(f"File not found: {update_file}")

    for trade_file in trade_files:
        try:
            trade_df = pd.read_csv(trade_file, names=["Local Time", "Exchange Time", "Price", "Volume", "Side"], header=0)
            trade_df['Exchange Time'] = (trade_df['Exchange Time'] * 1e9).astype('int64')
            all_trade_dfs.append(trade_df)
        except FileNotFoundError:
            print(f"File not found: {trade_file}")

    combined_update_df = pd.concat(all_update_dfs, ignore_index=True) if all_update_dfs else None
    combined_trade_df = pd.concat(all_trade_dfs, ignore_index=True) if all_trade_dfs else None

    return combined_update_df, combined_trade_df

# Use glob to find all update and trade files
update_files = glob.glob("update_snapshot_*.csv")
trade_files = glob.glob("trade_snapshot_*.csv")

# Process data
processed_data = read_csv_files(update_files, trade_files)
update_df, trade_df = processed_data

# Print DataFrames (optional, for checking)
if update_df is not None:
    print("Update DataFrame:")
    print(update_df.head())

if trade_df is not None:
    print("\nTrade DataFrame:")
    print(trade_df.head())

In [ ]:

# Constants for event types
DEPTH_EVENT = 1
TRADE_EVENT = 2

# Continue with the rest of your code...

# Immediately after loading the data
print(update_df.head())
print(trade_df.head())

# Rename columns to standardize
update_df.rename(columns={'Local Timestamp': 'local_timestamp', 'Price': 'price', 'Volume': 'volume', 'Exchange Timestamp': 'exch_timestamp', 'Type': 'side'}, inplace=True)
trade_df.rename(columns={'Local Time': 'local_timestamp', 'Exchange Time': 'exch_timestamp', 'Price': 'price', 'Volume': 'volume', 'Side': 'side'}, inplace=True)
print("Unique values in 'Type' column of update_df:", update_df['side'].unique())

# Check unique values in the 'Side' column of trade_df
print("Unique values in 'Side' column of trade_df:", trade_df['side'].unique())

# Assuming the renaming is done as before

# Correct mapping for 'side' based on the inspected unique values
update_df['side'] = update_df['side'].map({'ask': -1, 'bid': 1})
trade_df['side'] = trade_df['side'].map({'s': -1, 'b': 1})

# Add event type column and concatenate dataframes as before
# ...

# Add event type column
update_df['event'] = DEPTH_EVENT
trade_df['event'] = TRADE_EVENT

# Concatenate dataframes
combined_df = pd.concat([update_df, trade_df], ignore_index=True)

# Sort if necessary
combined_df.sort_values(by=[ 'local_timestamp'], inplace=True)

# Reorder columns
column_order = ['event', 'exch_timestamp', 'local_timestamp', 'side', 'price', 'volume']
combined_df = combined_df[column_order]

# Print the combined DataFrame
print(combined_df.head(10))

In [ ]:
import pandas as pd
prev_exch_timestamp = 0
prev_local_timestamp = 0
correct_exch_timestamp = True  # Enable timestamp correction

def correct_timestamp(row):
    global prev_exch_timestamp, prev_local_timestamp
    if correct_exch_timestamp:
        # If exch_timestamp is out of order, mark it as -1
        if row['exch_timestamp'] < prev_exch_timestamp:
            row['exch_timestamp'] = -1

        # Update prev_exch_timestamp if it's not -1
        if row['exch_timestamp'] != -1:
            prev_exch_timestamp = row['exch_timestamp']

        # Ensure local_timestamp is always greater or equal to exch_timestamp
        # If not, adjust local_timestamp to be equal to exch_timestamp
        # Also, ensure local_timestamp is non-decreasing
        if row['local_timestamp'] < max(prev_local_timestamp, row['exch_timestamp']):
            row['local_timestamp'] = max(prev_local_timestamp, row['exch_timestamp'])
        else:
            prev_local_timestamp = row['local_timestamp']

    return row

# Convert timestamps from nanoseconds to microseconds
combined_df['exch_timestamp'] = combined_df['exch_timestamp'] // 1000
combined_df['local_timestamp'] = combined_df['local_timestamp'] // 1000

# Apply the correct_timestamp function
combined_df = combined_df.apply(correct_timestamp, axis=1)

# Sorting the DataFrame
combined_df.sort_values(by=['local_timestamp', 'exch_timestamp'], inplace=True)


# Print the sorted DataFrame (optional)
print(combined_df.head(15))

combined_df.to_csv('filename.csv', index=False)  # Set index=False to exclude row indices
